<a href="https://colab.research.google.com/github/IllgamhoDuck/Quantum-Circuit-Optimization-with-Deep-learning/blob/master/Generate_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate dataset for deep learning model training
**Deep learning model will be seperated to 3 parts**

    Datasets will be created for 3 models each

- lstm autoencoder to find the original circuit manifold
- layout prediction
- optimized circuit prediction

**Only 5 qubit dataset will be used**

Because of data generating time and the data size 5 qubit will be fit for this education notebook.

    1 data of 20 qubit tooks approximately more than 40 minutes 

**The layout and circuit optimized by tranpiler level 3 will be used as a dataset**

    layout and optimized circuit dataset will be generated by transpiler level 3

We will use the dataset to train the model to predict the optimized circuits same as the circuit optimized by transpiler level 3. After the training is done we will train the optimized circuit model by reinforcement learning.


**Only Ourense quantum computer information will be used**

The code includes all noise information of Quantum computer below 20 qubits but for convenience we will only use `ourense` quantum computer inforamtion and others are ignored. It is possible to fix easily to generate dataset for every quantum computer

**One dataset**

We will create dataset in one file for convenience. Originally it was devided by 3 parts but will merge to one dataset

**Encoding method**

There are two ways considered to encode currently.
The encoded dataset will be following format

**`[gate type, control, target, angle1, angle2, angle3]`**

For the following colabs we are going to use **2nd method**

     1. Encode directly as number (Total 6 blocks)
        gate - 1 block (empty will be 0)
        control - 1 block (empty will be 0)
        target - 1 block (empty will be 0)
        angle - 3 blocks (one block for each angle)
        cx q[0],q[1] => [4, 1, 2, 0, 0, 0]

     2. Encode as one hot encoding (Total 29 blocks)
        gate - 8 block (empty will be the 1 at last block)
        control - 6 block (empty will be the 1 at 1st block)
        target - 6 block (empty will be the 1 at 1st block)
        measure - 6 block (empty will be the 1 at 1st block)
        angle - 3 blocks (one block for each angle)
        cx q[0],q[1] => [0, 0, 0, 1, 0, 0, 0, 0,
                         0, 1, 0, 0, 0, 0,
                         0, 0, 1, 0, 0, 0,
                         1, 0, 0, 0, 0, 0,
                         0, 0, 0]

### Setting environment
- package install
- google drive mount
- path configuration

In [0]:
!pip install --quiet qiskit nxpd

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# TODO - modify path to fit your computer
# dataset_path - where to save the dataset including circuit information
# model_path = where to save the deep learning model you created
# weight_path = Where to save the model weight you trained

dataset_path = '/content/drive/My Drive/QUANTUM/qaward/dataset'
model_path = '/content/drive/My Drive/QUANTUM/qaward/model and weight'
weight_path = '/content/drive/My Drive/QUANTUM/qaward/model and weight'

## Generate Data set with real quantum device noise model
- Quantum computer noise information
- Random circuit generator

In [0]:
#@title Real Quantum device hardware information
LOCAL_BASE_DIR = "/usr/local/lib/python3.6/site-packages/qiskit/test/mock/"
DATAFLOW_BASE_DIR = "/usr/local/lib/python3.6/site-packages/qiskit/test/mock/"
COLAB_BASE_DIR = "/usr/local/lib/python3.6/dist-packages/qiskit/test/mock/"

base_dir = COLAB_BASE_DIR

props_almaden_target_filepath = base_dir + "props_almaden.json"
props_almaden_str = '{"general": [], "qubits": [[{"date": "2019-10-08T08:19:48+00:00", "unit": "\u00b5s", "value": 113.3795751321217, "name": "T1"}, {"date": "2019-10-08T08:20:26+00:00", "unit": "\u00b5s", "value": 150.28477205442593, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 4.857281983598487, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.05499999999999994, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.08333333333333337, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.02666666666666667, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:19:48+00:00", "unit": "\u00b5s", "value": 105.66416929095271, "name": "T1"}, {"date": "2019-10-08T08:21:00+00:00", "unit": "\u00b5s", "value": 18.38410012677542, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 4.971793150372549, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.04833333333333334, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.06666666666666665, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.03, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:19:48+00:00", "unit": "\u00b5s", "value": 99.7234930105941, "name": "T1"}, {"date": "2019-10-08T08:20:26+00:00", "unit": "\u00b5s", "value": 21.462023649422687, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 5.071149476055804, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.03500000000000003, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.06333333333333335, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.006666666666666667, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:19:48+00:00", "unit": "\u00b5s", "value": 87.86876354457614, "name": "T1"}, {"date": "2019-10-08T08:21:00+00:00", "unit": "\u00b5s", "value": 55.001267752549474, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 4.926937969330278, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.036666666666666625, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.046666666666666634, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.02666666666666667, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T08:27:00+00:00", "unit": "\u00b5s", "value": 101.51294276467692, "name": "T1"}, {"date": "2019-10-08T08:20:26+00:00", "unit": "\u00b5s", "value": 23.343141049316454, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 4.719739345221021, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.025000000000000022, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.040000000000000036, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.01, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:19:48+00:00", "unit": "\u00b5s", "value": 83.04658147471106, "name": "T1"}, {"date": "2019-10-08T08:20:26+00:00", "unit": "\u00b5s", "value": 117.99853405913042, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 4.775029072397078, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.04500000000000004, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.08333333333333337, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.006666666666666667, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:19:48+00:00", "unit": "\u00b5s", "value": 124.94090943840628, "name": "T1"}, {"date": "2019-10-08T08:21:33+00:00", "unit": "\u00b5s", "value": 41.53937920218258, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 4.906384120317435, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.07333333333333325, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.09999999999999998, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.04666666666666667, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:19:48+00:00", "unit": "\u00b5s", "value": 33.22869346596583, "name": "T1"}, {"date": "2019-10-08T08:20:26+00:00", "unit": "\u00b5s", "value": 13.675961047735335, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 4.79662359453284, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.04499999999999993, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.06333333333333335, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.02666666666666667, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:19:48+00:00", "unit": "\u00b5s", "value": 64.93841415181943, "name": "T1"}, {"date": "2019-10-08T08:21:33+00:00", "unit": "\u00b5s", "value": 87.61256223452315, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 4.8576605551920276, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.023333333333333317, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.033333333333333326, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.013333333333333334, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:19:48+00:00", "unit": "\u00b5s", "value": 93.7867700394655, "name": "T1"}, {"date": "2019-10-08T08:20:26+00:00", "unit": "\u00b5s", "value": 192.50000251062167, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 4.659275115145499, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.020000000000000018, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.036666666666666625, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.0033333333333333335, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:19:48+00:00", "unit": "\u00b5s", "value": 114.73977844647561, "name": "T1"}, {"date": "2019-10-08T08:21:00+00:00", "unit": "\u00b5s", "value": 11.691873206915197, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 4.979151194108445, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.05166666666666675, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.09666666666666668, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.006666666666666667, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:19:48+00:00", "unit": "\u00b5s", "value": 46.82592471456916, "name": "T1"}, {"date": "2019-10-08T08:20:26+00:00", "unit": "\u00b5s", "value": 83.69092323968601, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 4.922140269634184, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.021666666666666723, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.030000000000000027, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.013333333333333334, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T08:27:00+00:00", "unit": "\u00b5s", "value": 52.7244604478663, "name": "T1"}, {"date": "2019-10-08T08:21:00+00:00", "unit": "\u00b5s", "value": 46.41161763294124, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 4.834130717092513, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.02833333333333332, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.046666666666666634, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.01, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:19:48+00:00", "unit": "\u00b5s", "value": 104.84280045120985, "name": "T1"}, {"date": "2019-10-08T08:20:26+00:00", "unit": "\u00b5s", "value": 172.6443272130343, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 4.953395340841351, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.021666666666666723, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.036666666666666625, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.006666666666666667, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:19:48+00:00", "unit": "\u00b5s", "value": 107.09968131373412, "name": "T1"}, {"date": "2019-10-08T08:21:00+00:00", "unit": "\u00b5s", "value": 53.72517902560914, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 4.75043432810736, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.04166666666666663, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.07999999999999996, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.0033333333333333335, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:19:48+00:00", "unit": "\u00b5s", "value": 119.81914190803826, "name": "T1"}, {"date": "2019-10-08T08:20:26+00:00", "unit": "\u00b5s", "value": 170.75473030671495, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 4.656960806445796, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.023333333333333317, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.033333333333333326, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.013333333333333334, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:19:48+00:00", "unit": "\u00b5s", "value": 85.78589261348652, "name": "T1"}, {"date": "2019-10-08T08:21:00+00:00", "unit": "\u00b5s", "value": 88.93183950008954, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 4.8493325715400335, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.02833333333333332, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.046666666666666634, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.01, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:19:48+00:00", "unit": "\u00b5s", "value": 105.45362547865902, "name": "T1"}, {"date": "2019-10-08T08:20:26+00:00", "unit": "\u00b5s", "value": 88.03702871710159, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 4.7850052011828605, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.050000000000000044, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.08666666666666667, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.013333333333333334, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T08:27:00+00:00", "unit": "\u00b5s", "value": 97.13741598514389, "name": "T1"}, {"date": "2019-10-08T08:21:00+00:00", "unit": "\u00b5s", "value": 108.71810987380148, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 4.826352274844338, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.050000000000000044, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.07666666666666666, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.023333333333333334, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:19:48+00:00", "unit": "\u00b5s", "value": 132.39063671088658, "name": "T1"}, {"date": "2019-10-08T08:20:26+00:00", "unit": "\u00b5s", "value": 212.7728386342524, "name": "T2"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "GHz", "value": 4.713156080438663, "name": "frequency"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.04166666666666663, "name": "readout_error"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.07999999999999996, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:19:25+00:00", "unit": "", "value": 0.0033333333333333335, "name": "prob_meas1_prep0"}]], "gates": [{"qubits": [0], "gate": "id", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0005295247303964942, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_0"}, {"qubits": [0], "gate": "u1", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_0"}, {"qubits": [0], "gate": "u2", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0005295247303964942, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_0"}, {"qubits": [0], "gate": "u3", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0010590494607929885, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_0"}, {"qubits": [1], "gate": "id", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0003306101893089778, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_1"}, {"qubits": [1], "gate": "u1", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_1"}, {"qubits": [1], "gate": "u2", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0003306101893089778, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_1"}, {"qubits": [1], "gate": "u3", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0006612203786179556, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_1"}, {"qubits": [2], "gate": "id", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0005406845644677221, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_2"}, {"qubits": [2], "gate": "u1", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_2"}, {"qubits": [2], "gate": "u2", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0005406845644677221, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_2"}, {"qubits": [2], "gate": "u3", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0010813691289354441, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_2"}, {"qubits": [3], "gate": "id", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0009069681997705605, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_3"}, {"qubits": [3], "gate": "u1", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_3"}, {"qubits": [3], "gate": "u2", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0009069681997705605, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_3"}, {"qubits": [3], "gate": "u3", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.001813936399541121, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_3"}, {"qubits": [4], "gate": "id", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0011366551187410982, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_4"}, {"qubits": [4], "gate": "u1", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_4"}, {"qubits": [4], "gate": "u2", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0011366551187410982, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_4"}, {"qubits": [4], "gate": "u3", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0022733102374821964, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_4"}, {"qubits": [5], "gate": "id", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0003913702412179343, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_5"}, {"qubits": [5], "gate": "u1", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_5"}, {"qubits": [5], "gate": "u2", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0003913702412179343, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_5"}, {"qubits": [5], "gate": "u3", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0007827404824358686, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_5"}, {"qubits": [6], "gate": "id", "parameters": [{"date": "2019-10-08T08:23:45+00:00", "unit": "", "value": 0.0005301138939858822, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_6"}, {"qubits": [6], "gate": "u1", "parameters": [{"date": "2019-10-08T08:23:45+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_6"}, {"qubits": [6], "gate": "u2", "parameters": [{"date": "2019-10-08T08:23:45+00:00", "unit": "", "value": 0.0005301138939858822, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_6"}, {"qubits": [6], "gate": "u3", "parameters": [{"date": "2019-10-08T08:23:45+00:00", "unit": "", "value": 0.0010602277879717644, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_6"}, {"qubits": [7], "gate": "id", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0010217987258482762, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_7"}, {"qubits": [7], "gate": "u1", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_7"}, {"qubits": [7], "gate": "u2", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0010217987258482762, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_7"}, {"qubits": [7], "gate": "u3", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0020435974516965524, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_7"}, {"qubits": [8], "gate": "id", "parameters": [{"date": "2019-10-08T08:23:45+00:00", "unit": "", "value": 0.0008059908037876117, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_8"}, {"qubits": [8], "gate": "u1", "parameters": [{"date": "2019-10-08T08:23:45+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_8"}, {"qubits": [8], "gate": "u2", "parameters": [{"date": "2019-10-08T08:23:45+00:00", "unit": "", "value": 0.0008059908037876117, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_8"}, {"qubits": [8], "gate": "u3", "parameters": [{"date": "2019-10-08T08:23:45+00:00", "unit": "", "value": 0.0016119816075752234, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_8"}, {"qubits": [9], "gate": "id", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0003753304106910418, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_9"}, {"qubits": [9], "gate": "u1", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_9"}, {"qubits": [9], "gate": "u2", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0003753304106910418, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_9"}, {"qubits": [9], "gate": "u3", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0007506608213820836, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_9"}, {"qubits": [10], "gate": "id", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0005205904954868159, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_10"}, {"qubits": [10], "gate": "u1", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_10"}, {"qubits": [10], "gate": "u2", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0005205904954868159, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_10"}, {"qubits": [10], "gate": "u3", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0010411809909736318, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_10"}, {"qubits": [11], "gate": "id", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.00048084747779095487, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_11"}, {"qubits": [11], "gate": "u1", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_11"}, {"qubits": [11], "gate": "u2", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.00048084747779095487, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_11"}, {"qubits": [11], "gate": "u3", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0009616949555819097, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_11"}, {"qubits": [12], "gate": "id", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0011724890409250469, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_12"}, {"qubits": [12], "gate": "u1", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_12"}, {"qubits": [12], "gate": "u2", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0011724890409250469, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_12"}, {"qubits": [12], "gate": "u3", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0023449780818500937, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_12"}, {"qubits": [13], "gate": "id", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.000597846450074657, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_13"}, {"qubits": [13], "gate": "u1", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_13"}, {"qubits": [13], "gate": "u2", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.000597846450074657, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_13"}, {"qubits": [13], "gate": "u3", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.001195692900149314, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_13"}, {"qubits": [14], "gate": "id", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0005726914344263584, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_14"}, {"qubits": [14], "gate": "u1", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_14"}, {"qubits": [14], "gate": "u2", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0005726914344263584, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_14"}, {"qubits": [14], "gate": "u3", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.001145382868852717, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_14"}, {"qubits": [15], "gate": "id", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0006331167468516876, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_15"}, {"qubits": [15], "gate": "u1", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_15"}, {"qubits": [15], "gate": "u2", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0006331167468516876, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_15"}, {"qubits": [15], "gate": "u3", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0012662334937033751, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_15"}, {"qubits": [16], "gate": "id", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0005300176749858641, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_16"}, {"qubits": [16], "gate": "u1", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_16"}, {"qubits": [16], "gate": "u2", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0005300176749858641, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_16"}, {"qubits": [16], "gate": "u3", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0010600353499717282, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_16"}, {"qubits": [17], "gate": "id", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0010868413863705075, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_17"}, {"qubits": [17], "gate": "u1", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_17"}, {"qubits": [17], "gate": "u2", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0010868413863705075, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_17"}, {"qubits": [17], "gate": "u3", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.002173682772741015, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_17"}, {"qubits": [18], "gate": "id", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0013081972476988721, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_18"}, {"qubits": [18], "gate": "u1", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_18"}, {"qubits": [18], "gate": "u2", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0013081972476988721, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_18"}, {"qubits": [18], "gate": "u3", "parameters": [{"date": "2019-10-08T08:22:57+00:00", "unit": "", "value": 0.0026163944953977443, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_18"}, {"qubits": [19], "gate": "id", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.00050044063455184, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_19"}, {"qubits": [19], "gate": "u1", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_19"}, {"qubits": [19], "gate": "u2", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.00050044063455184, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_19"}, {"qubits": [19], "gate": "u3", "parameters": [{"date": "2019-10-08T08:22:03+00:00", "unit": "", "value": 0.00100088126910368, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_19"}, {"qubits": [0, 1], "gate": "cx", "parameters": [{"date": "2019-10-08T08:24:59+00:00", "unit": "", "value": 0.010649169010966214, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 298.66666666666663, "name": "gate_length"}], "name": "cx0_1"}, {"qubits": [1, 0], "gate": "cx", "parameters": [{"date": "2019-10-08T08:24:59+00:00", "unit": "", "value": 0.010649169010966214, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 298.66666666666663, "name": "gate_length"}], "name": "cx1_0"}, {"qubits": [1, 2], "gate": "cx", "parameters": [{"date": "2019-10-08T08:30:17+00:00", "unit": "", "value": 0.012231293393077186, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 341.3333333333333, "name": "gate_length"}], "name": "cx1_2"}, {"qubits": [1, 6], "gate": "cx", "parameters": [{"date": "2019-10-08T08:27:46+00:00", "unit": "", "value": 0.009538476217756053, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 277.3333333333333, "name": "gate_length"}], "name": "cx1_6"}, {"qubits": [2, 1], "gate": "cx", "parameters": [{"date": "2019-10-08T08:30:17+00:00", "unit": "", "value": 0.012231293393077186, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 341.3333333333333, "name": "gate_length"}], "name": "cx2_1"}, {"qubits": [2, 3], "gate": "cx", "parameters": [{"date": "2019-10-08T08:32:57+00:00", "unit": "", "value": 0.029249833657427865, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 497.77777777777777, "name": "gate_length"}], "name": "cx2_3"}, {"qubits": [3, 2], "gate": "cx", "parameters": [{"date": "2019-10-08T08:32:57+00:00", "unit": "", "value": 0.029249833657427865, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 497.77777777777777, "name": "gate_length"}], "name": "cx3_2"}, {"qubits": [3, 4], "gate": "cx", "parameters": [{"date": "2019-10-08T08:38:12+00:00", "unit": "", "value": 0.028813294663382394, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 568.8888888888889, "name": "gate_length"}], "name": "cx3_4"}, {"qubits": [3, 8], "gate": "cx", "parameters": [{"date": "2019-10-08T08:35:31+00:00", "unit": "", "value": 0.013091620718991448, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 277.3333333333333, "name": "gate_length"}], "name": "cx3_8"}, {"qubits": [4, 3], "gate": "cx", "parameters": [{"date": "2019-10-08T08:38:12+00:00", "unit": "", "value": 0.028813294663382394, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 568.8888888888889, "name": "gate_length"}], "name": "cx4_3"}, {"qubits": [5, 6], "gate": "cx", "parameters": [{"date": "2019-10-08T08:40:44+00:00", "unit": "", "value": 0.02142360836276308, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 547.5555555555555, "name": "gate_length"}], "name": "cx5_6"}, {"qubits": [5, 10], "gate": "cx", "parameters": [{"date": "2019-10-08T08:51:35+00:00", "unit": "", "value": 0.014759397002595542, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 497.77777777777777, "name": "gate_length"}], "name": "cx5_10"}, {"qubits": [6, 1], "gate": "cx", "parameters": [{"date": "2019-10-08T08:27:46+00:00", "unit": "", "value": 0.009538476217756053, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 277.3333333333333, "name": "gate_length"}], "name": "cx6_1"}, {"qubits": [6, 5], "gate": "cx", "parameters": [{"date": "2019-10-08T08:40:44+00:00", "unit": "", "value": 0.02142360836276308, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 547.5555555555555, "name": "gate_length"}], "name": "cx6_5"}, {"qubits": [6, 7], "gate": "cx", "parameters": [{"date": "2019-10-08T08:43:38+00:00", "unit": "", "value": 0.02270468880297208, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 455.1111111111111, "name": "gate_length"}], "name": "cx6_7"}, {"qubits": [7, 6], "gate": "cx", "parameters": [{"date": "2019-10-08T08:43:38+00:00", "unit": "", "value": 0.02270468880297208, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 455.1111111111111, "name": "gate_length"}], "name": "cx7_6"}, {"qubits": [7, 8], "gate": "cx", "parameters": [{"date": "2019-10-08T08:46:28+00:00", "unit": "", "value": 0.021679080407588175, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 369.77777777777777, "name": "gate_length"}], "name": "cx7_8"}, {"qubits": [7, 12], "gate": "cx", "parameters": [{"date": "2019-10-08T09:02:27+00:00", "unit": "", "value": 0.02654598416338272, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 320.0, "name": "gate_length"}], "name": "cx7_12"}, {"qubits": [8, 3], "gate": "cx", "parameters": [{"date": "2019-10-08T08:35:31+00:00", "unit": "", "value": 0.013091620718991448, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 277.3333333333333, "name": "gate_length"}], "name": "cx8_3"}, {"qubits": [8, 7], "gate": "cx", "parameters": [{"date": "2019-10-08T08:46:28+00:00", "unit": "", "value": 0.021679080407588175, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 369.77777777777777, "name": "gate_length"}], "name": "cx8_7"}, {"qubits": [8, 9], "gate": "cx", "parameters": [{"date": "2019-10-08T08:49:01+00:00", "unit": "", "value": 0.023184292522825223, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 625.7777777777777, "name": "gate_length"}], "name": "cx8_9"}, {"qubits": [9, 8], "gate": "cx", "parameters": [{"date": "2019-10-08T08:49:01+00:00", "unit": "", "value": 0.023184292522825223, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 625.7777777777777, "name": "gate_length"}], "name": "cx9_8"}, {"qubits": [9, 14], "gate": "cx", "parameters": [{"date": "2019-10-08T09:08:07+00:00", "unit": "", "value": 0.008601183314818767, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 270.22222222222223, "name": "gate_length"}], "name": "cx9_14"}, {"qubits": [10, 5], "gate": "cx", "parameters": [{"date": "2019-10-08T08:51:35+00:00", "unit": "", "value": 0.014759397002595542, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 497.77777777777777, "name": "gate_length"}], "name": "cx10_5"}, {"qubits": [10, 11], "gate": "cx", "parameters": [{"date": "2019-10-08T08:54:08+00:00", "unit": "", "value": 0.013647151734394514, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 348.4444444444444, "name": "gate_length"}], "name": "cx10_11"}, {"qubits": [11, 10], "gate": "cx", "parameters": [{"date": "2019-10-08T08:54:08+00:00", "unit": "", "value": 0.013647151734394514, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 348.4444444444444, "name": "gate_length"}], "name": "cx11_10"}, {"qubits": [11, 12], "gate": "cx", "parameters": [{"date": "2019-10-08T08:56:39+00:00", "unit": "", "value": 0.015917239757574114, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 327.1111111111111, "name": "gate_length"}], "name": "cx11_12"}, {"qubits": [11, 16], "gate": "cx", "parameters": [{"date": "2019-10-08T08:59:19+00:00", "unit": "", "value": 0.01475356313288248, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 320.0, "name": "gate_length"}], "name": "cx11_16"}, {"qubits": [12, 7], "gate": "cx", "parameters": [{"date": "2019-10-08T09:02:27+00:00", "unit": "", "value": 0.02654598416338272, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 320.0, "name": "gate_length"}], "name": "cx12_7"}, {"qubits": [12, 11], "gate": "cx", "parameters": [{"date": "2019-10-08T08:56:39+00:00", "unit": "", "value": 0.015917239757574114, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 327.1111111111111, "name": "gate_length"}], "name": "cx12_11"}, {"qubits": [12, 13], "gate": "cx", "parameters": [{"date": "2019-10-08T09:05:00+00:00", "unit": "", "value": 0.018841209791559038, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 355.55555555555554, "name": "gate_length"}], "name": "cx12_13"}, {"qubits": [13, 12], "gate": "cx", "parameters": [{"date": "2019-10-08T09:05:00+00:00", "unit": "", "value": 0.018841209791559038, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 355.55555555555554, "name": "gate_length"}], "name": "cx13_12"}, {"qubits": [13, 14], "gate": "cx", "parameters": [{"date": "2019-10-08T09:10:45+00:00", "unit": "", "value": 0.011703615402504003, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 234.66666666666666, "name": "gate_length"}], "name": "cx13_14"}, {"qubits": [13, 18], "gate": "cx", "parameters": [{"date": "2019-10-08T09:19:00+00:00", "unit": "", "value": 0.01982481359304153, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 526.2222222222222, "name": "gate_length"}], "name": "cx13_18"}, {"qubits": [14, 9], "gate": "cx", "parameters": [{"date": "2019-10-08T09:08:07+00:00", "unit": "", "value": 0.008601183314818767, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 270.22222222222223, "name": "gate_length"}], "name": "cx14_9"}, {"qubits": [14, 13], "gate": "cx", "parameters": [{"date": "2019-10-08T09:10:45+00:00", "unit": "", "value": 0.011703615402504003, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 234.66666666666666, "name": "gate_length"}], "name": "cx14_13"}, {"qubits": [15, 16], "gate": "cx", "parameters": [{"date": "2019-10-08T09:13:19+00:00", "unit": "", "value": 0.016639771737897047, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 362.66666666666663, "name": "gate_length"}], "name": "cx15_16"}, {"qubits": [16, 11], "gate": "cx", "parameters": [{"date": "2019-10-08T08:59:19+00:00", "unit": "", "value": 0.01475356313288248, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 320.0, "name": "gate_length"}], "name": "cx16_11"}, {"qubits": [16, 15], "gate": "cx", "parameters": [{"date": "2019-10-08T09:13:19+00:00", "unit": "", "value": 0.016639771737897047, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 362.66666666666663, "name": "gate_length"}], "name": "cx16_15"}, {"qubits": [16, 17], "gate": "cx", "parameters": [{"date": "2019-10-08T09:16:00+00:00", "unit": "", "value": 0.012589810300146581, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 256.0, "name": "gate_length"}], "name": "cx16_17"}, {"qubits": [17, 16], "gate": "cx", "parameters": [{"date": "2019-10-08T09:16:00+00:00", "unit": "", "value": 0.012589810300146581, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 256.0, "name": "gate_length"}], "name": "cx17_16"}, {"qubits": [17, 18], "gate": "cx", "parameters": [{"date": "2019-10-08T09:21:33+00:00", "unit": "", "value": 0.02873879493425166, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 611.5555555555555, "name": "gate_length"}], "name": "cx17_18"}, {"qubits": [18, 13], "gate": "cx", "parameters": [{"date": "2019-10-08T09:19:00+00:00", "unit": "", "value": 0.01982481359304153, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 526.2222222222222, "name": "gate_length"}], "name": "cx18_13"}, {"qubits": [18, 17], "gate": "cx", "parameters": [{"date": "2019-10-08T09:21:33+00:00", "unit": "", "value": 0.02873879493425166, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 611.5555555555555, "name": "gate_length"}], "name": "cx18_17"}, {"qubits": [18, 19], "gate": "cx", "parameters": [{"date": "2019-10-08T09:24:09+00:00", "unit": "", "value": 0.014819088647043482, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 320.0, "name": "gate_length"}], "name": "cx18_19"}, {"qubits": [19, 18], "gate": "cx", "parameters": [{"date": "2019-10-08T09:24:09+00:00", "unit": "", "value": 0.014819088647043482, "name": "gate_error"}, {"date": "2019-10-08T09:24:09+00:00", "unit": "ns", "value": 320.0, "name": "gate_length"}], "name": "cx19_18"}], "backend_version": "1.2.7", "backend_name": "ibmq_almaden", "last_update_date": "2019-10-08T09:24:09+00:00"}'
with open(props_almaden_target_filepath, 'w') as f:
  f.write(props_almaden_str)

props_boeblingen_target_filepath = base_dir + "props_boeblingen.json"
props_boeblingen_str = '{"general": [], "qubits": [[{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 102.85416493848709, "name": "T1"}, {"date": "2019-10-08T10:52:39+00:00", "unit": "\u00b5s", "value": 163.5967764003543, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 5.046903257383382, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.1166666666666667, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.1333333333333333, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.1, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 79.14579023442666, "name": "T1"}, {"date": "2019-10-08T10:54:47+00:00", "unit": "\u00b5s", "value": 131.90631923866155, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 4.847301793214777, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.020000000000000018, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.023333333333333334, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.01666666666666672, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 94.91194437776132, "name": "T1"}, {"date": "2019-10-08T10:52:39+00:00", "unit": "\u00b5s", "value": 145.93512916784837, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 4.703657144760766, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.025000000000000022, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.033333333333333326, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.016666666666666666, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 73.7568384768635, "name": "T1"}, {"date": "2019-10-08T10:54:47+00:00", "unit": "\u00b5s", "value": 47.30578667690049, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 4.77433318770615, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.020000000000000018, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.02666666666666667, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.013333333333333308, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 187.85891843367634, "name": "T1"}, {"date": "2019-10-08T10:52:39+00:00", "unit": "\u00b5s", "value": 112.12520047407418, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 4.366180085609712, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.036666666666666625, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.05666666666666664, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.016666666666666666, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 65.3543237183781, "name": "T1"}, {"date": "2019-10-08T10:52:39+00:00", "unit": "\u00b5s", "value": 138.4630186765929, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 4.911579162611726, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.09499999999999997, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.05, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.14, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 32.54991563480177, "name": "T1"}, {"date": "2019-10-08T11:02:37+00:00", "unit": "\u00b5s", "value": 82.99388294139078, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 4.726872639487108, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.043333333333333335, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.043333333333333335, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.043333333333333335, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 83.83547839075025, "name": "T1"}, {"date": "2019-10-08T10:52:39+00:00", "unit": "\u00b5s", "value": 68.20810008553288, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 4.545987790976865, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.03166666666666673, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.04666666666666667, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.01666666666666672, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 53.12941203803107, "name": "T1"}, {"date": "2019-10-08T11:02:37+00:00", "unit": "\u00b5s", "value": 90.78113285364125, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 4.658588162065793, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.05166666666666664, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.06, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.043333333333333335, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 43.94955821389211, "name": "T1"}, {"date": "2019-10-08T10:52:39+00:00", "unit": "\u00b5s", "value": 88.0260463486044, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 4.766744956760172, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.03499999999999992, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.043333333333333335, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.02666666666666667, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 60.55309292536068, "name": "T1"}, {"date": "2019-10-08T10:54:47+00:00", "unit": "\u00b5s", "value": 66.72119293364145, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 4.95530399400922, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.06833333333333336, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.09333333333333338, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.043333333333333335, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 70.77889689328188, "name": "T1"}, {"date": "2019-10-08T10:52:39+00:00", "unit": "\u00b5s", "value": 135.3797000375796, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 4.521699757167218, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.043333333333333335, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.04, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.046666666666666634, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 42.15032330713362, "name": "T1"}, {"date": "2019-10-08T10:54:47+00:00", "unit": "\u00b5s", "value": 63.7709606807059, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 4.732550716941763, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.04833333333333334, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.07999999999999996, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.016666666666666666, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 48.11152465903231, "name": "T1"}, {"date": "2019-10-07T09:39:43+00:00", "unit": "\u00b5s", "value": 53.75664081104812, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 4.6355517858485875, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.043333333333333335, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.06333333333333335, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.023333333333333334, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 88.82211529410327, "name": "T1"}, {"date": "2019-10-08T10:54:47+00:00", "unit": "\u00b5s", "value": 139.52734840443173, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 4.555081132676964, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.02833333333333332, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.040000000000000036, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.016666666666666666, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 99.84794001465994, "name": "T1"}, {"date": "2019-10-08T10:52:39+00:00", "unit": "\u00b5s", "value": 113.30408098299273, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 4.5822646287843725, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.013333333333333308, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.023333333333333334, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.0033333333333332993, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 69.5885504108954, "name": "T1"}, {"date": "2019-10-08T10:54:47+00:00", "unit": "\u00b5s", "value": 64.0082971274578, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 4.755628079180518, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.016666666666666607, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.013333333333333308, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.02, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 125.61840752599393, "name": "T1"}, {"date": "2019-10-08T10:52:39+00:00", "unit": "\u00b5s", "value": 191.19933784261426, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 4.589824421320106, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.023333333333333317, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.036666666666666625, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.01, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 35.100068289725094, "name": "T1"}, {"date": "2019-10-08T10:54:47+00:00", "unit": "\u00b5s", "value": 61.102028000691035, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 4.752417207215866, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.04499999999999993, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.06666666666666667, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.023333333333333317, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T10:44:43+00:00", "unit": "\u00b5s", "value": 81.53242053408343, "name": "T1"}, {"date": "2019-10-08T10:52:39+00:00", "unit": "\u00b5s", "value": 76.33138775481778, "name": "T2"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "GHz", "value": 5.035097390471422, "name": "frequency"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.04833333333333334, "name": "readout_error"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.043333333333333335, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T10:42:42+00:00", "unit": "", "value": 0.05333333333333334, "name": "prob_meas1_prep0"}]], "gates": [{"qubits": [0], "gate": "id", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.000325653286886848, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_0"}, {"qubits": [0], "gate": "u1", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_0"}, {"qubits": [0], "gate": "u2", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.000325653286886848, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_0"}, {"qubits": [0], "gate": "u3", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.000651306573773696, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_0"}, {"qubits": [1], "gate": "id", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0003616314031393863, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_1"}, {"qubits": [1], "gate": "u1", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_1"}, {"qubits": [1], "gate": "u2", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0003616314031393863, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_1"}, {"qubits": [1], "gate": "u3", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0007232628062787726, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_1"}, {"qubits": [2], "gate": "id", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0006892012573486157, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_2"}, {"qubits": [2], "gate": "u1", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_2"}, {"qubits": [2], "gate": "u2", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0006892012573486157, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_2"}, {"qubits": [2], "gate": "u3", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0013784025146972314, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_2"}, {"qubits": [3], "gate": "id", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0004332493074768526, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_3"}, {"qubits": [3], "gate": "u1", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_3"}, {"qubits": [3], "gate": "u2", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0004332493074768526, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_3"}, {"qubits": [3], "gate": "u3", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0008664986149537052, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_3"}, {"qubits": [4], "gate": "id", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.00037100559395040615, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_4"}, {"qubits": [4], "gate": "u1", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_4"}, {"qubits": [4], "gate": "u2", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.00037100559395040615, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_4"}, {"qubits": [4], "gate": "u3", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0007420111879008123, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_4"}, {"qubits": [5], "gate": "id", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0006900752085232459, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_5"}, {"qubits": [5], "gate": "u1", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_5"}, {"qubits": [5], "gate": "u2", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0006900752085232459, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_5"}, {"qubits": [5], "gate": "u3", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0013801504170464919, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_5"}, {"qubits": [6], "gate": "id", "parameters": [{"date": "2019-10-08T11:26:07+00:00", "unit": "", "value": 0.0004620010020043001, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_6"}, {"qubits": [6], "gate": "u1", "parameters": [{"date": "2019-10-08T11:26:07+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_6"}, {"qubits": [6], "gate": "u2", "parameters": [{"date": "2019-10-08T11:26:07+00:00", "unit": "", "value": 0.0004620010020043001, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_6"}, {"qubits": [6], "gate": "u3", "parameters": [{"date": "2019-10-08T11:26:07+00:00", "unit": "", "value": 0.0009240020040086002, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_6"}, {"qubits": [7], "gate": "id", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0002776491690771715, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_7"}, {"qubits": [7], "gate": "u1", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_7"}, {"qubits": [7], "gate": "u2", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0002776491690771715, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_7"}, {"qubits": [7], "gate": "u3", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.000555298338154343, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_7"}, {"qubits": [8], "gate": "id", "parameters": [{"date": "2019-10-08T11:26:07+00:00", "unit": "", "value": 0.0004164794205428041, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_8"}, {"qubits": [8], "gate": "u1", "parameters": [{"date": "2019-10-08T11:26:07+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_8"}, {"qubits": [8], "gate": "u2", "parameters": [{"date": "2019-10-08T11:26:07+00:00", "unit": "", "value": 0.0004164794205428041, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_8"}, {"qubits": [8], "gate": "u3", "parameters": [{"date": "2019-10-08T11:26:07+00:00", "unit": "", "value": 0.0008329588410856082, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_8"}, {"qubits": [9], "gate": "id", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0005300125196208912, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_9"}, {"qubits": [9], "gate": "u1", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_9"}, {"qubits": [9], "gate": "u2", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0005300125196208912, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_9"}, {"qubits": [9], "gate": "u3", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0010600250392417824, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_9"}, {"qubits": [10], "gate": "id", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0020816424289898648, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_10"}, {"qubits": [10], "gate": "u1", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_10"}, {"qubits": [10], "gate": "u2", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0020816424289898648, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_10"}, {"qubits": [10], "gate": "u3", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0041632848579797295, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_10"}, {"qubits": [11], "gate": "id", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0004274221676876212, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_11"}, {"qubits": [11], "gate": "u1", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_11"}, {"qubits": [11], "gate": "u2", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0004274221676876212, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_11"}, {"qubits": [11], "gate": "u3", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0008548443353752424, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_11"}, {"qubits": [12], "gate": "id", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.00043340491454847344, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_12"}, {"qubits": [12], "gate": "u1", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_12"}, {"qubits": [12], "gate": "u2", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.00043340491454847344, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_12"}, {"qubits": [12], "gate": "u3", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0008668098290969469, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_12"}, {"qubits": [13], "gate": "id", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0006036562398736551, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_13"}, {"qubits": [13], "gate": "u1", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_13"}, {"qubits": [13], "gate": "u2", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0006036562398736551, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_13"}, {"qubits": [13], "gate": "u3", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0012073124797473101, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_13"}, {"qubits": [14], "gate": "id", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0003157735478063037, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_14"}, {"qubits": [14], "gate": "u1", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_14"}, {"qubits": [14], "gate": "u2", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0003157735478063037, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_14"}, {"qubits": [14], "gate": "u3", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0006315470956126074, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_14"}, {"qubits": [15], "gate": "id", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.00043352679749160004, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_15"}, {"qubits": [15], "gate": "u1", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_15"}, {"qubits": [15], "gate": "u2", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.00043352679749160004, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_15"}, {"qubits": [15], "gate": "u3", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0008670535949832001, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_15"}, {"qubits": [16], "gate": "id", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.00036203332628612935, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_16"}, {"qubits": [16], "gate": "u1", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_16"}, {"qubits": [16], "gate": "u2", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.00036203332628612935, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_16"}, {"qubits": [16], "gate": "u3", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0007240666525722587, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_16"}, {"qubits": [17], "gate": "id", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0003191963235549212, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_17"}, {"qubits": [17], "gate": "u1", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_17"}, {"qubits": [17], "gate": "u2", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0003191963235549212, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_17"}, {"qubits": [17], "gate": "u3", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0006383926471098424, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_17"}, {"qubits": [18], "gate": "id", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0008753524640779551, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_18"}, {"qubits": [18], "gate": "u1", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_18"}, {"qubits": [18], "gate": "u2", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0008753524640779551, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_18"}, {"qubits": [18], "gate": "u3", "parameters": [{"date": "2019-10-08T11:23:51+00:00", "unit": "", "value": 0.0017507049281559103, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_18"}, {"qubits": [19], "gate": "id", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0009430042933686515, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_19"}, {"qubits": [19], "gate": "u1", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_19"}, {"qubits": [19], "gate": "u2", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.0009430042933686515, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_19"}, {"qubits": [19], "gate": "u3", "parameters": [{"date": "2019-10-08T11:04:47+00:00", "unit": "", "value": 0.001886008586737303, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_19"}, {"qubits": [0, 1], "gate": "cx", "parameters": [{"date": "2019-10-08T11:55:03+00:00", "unit": "", "value": 0.01195103254058677, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 220.44444444444443, "name": "gate_length"}], "name": "cx0_1"}, {"qubits": [1, 0], "gate": "cx", "parameters": [{"date": "2019-10-08T11:55:03+00:00", "unit": "", "value": 0.01195103254058677, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 220.44444444444443, "name": "gate_length"}], "name": "cx1_0"}, {"qubits": [1, 2], "gate": "cx", "parameters": [{"date": "2019-10-08T12:07:42+00:00", "unit": "", "value": 0.009286590487134094, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 298.66666666666663, "name": "gate_length"}], "name": "cx1_2"}, {"qubits": [1, 6], "gate": "cx", "parameters": [{"date": "2019-10-08T13:12:51+00:00", "unit": "", "value": 0.015097443081646927, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 426.66666666666663, "name": "gate_length"}], "name": "cx1_6"}, {"qubits": [2, 1], "gate": "cx", "parameters": [{"date": "2019-10-08T12:07:42+00:00", "unit": "", "value": 0.009286590487134094, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 298.66666666666663, "name": "gate_length"}], "name": "cx2_1"}, {"qubits": [2, 3], "gate": "cx", "parameters": [{"date": "2019-10-08T12:37:08+00:00", "unit": "", "value": 0.00928514074018788, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 241.77777777777777, "name": "gate_length"}], "name": "cx2_3"}, {"qubits": [3, 2], "gate": "cx", "parameters": [{"date": "2019-10-08T12:37:08+00:00", "unit": "", "value": 0.00928514074018788, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 241.77777777777777, "name": "gate_length"}], "name": "cx3_2"}, {"qubits": [3, 4], "gate": "cx", "parameters": [{"date": "2019-10-08T12:49:55+00:00", "unit": "", "value": 0.014128731546206974, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 732.4444444444445, "name": "gate_length"}], "name": "cx3_4"}, {"qubits": [3, 8], "gate": "cx", "parameters": [{"date": "2019-10-08T13:50:21+00:00", "unit": "", "value": 0.00895720102940653, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 327.1111111111111, "name": "gate_length"}], "name": "cx3_8"}, {"qubits": [4, 3], "gate": "cx", "parameters": [{"date": "2019-10-08T12:49:55+00:00", "unit": "", "value": 0.014128731546206974, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 732.4444444444445, "name": "gate_length"}], "name": "cx4_3"}, {"qubits": [5, 6], "gate": "cx", "parameters": [{"date": "2019-10-08T13:54:37+00:00", "unit": "", "value": 0.010308156281825964, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 241.77777777777777, "name": "gate_length"}], "name": "cx5_6"}, {"qubits": [5, 10], "gate": "cx", "parameters": [{"date": "2019-10-08T14:25:11+00:00", "unit": "", "value": 0.013522194386392322, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 270.22222222222223, "name": "gate_length"}], "name": "cx5_10"}, {"qubits": [6, 1], "gate": "cx", "parameters": [{"date": "2019-10-08T13:12:51+00:00", "unit": "", "value": 0.015097443081646927, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 426.66666666666663, "name": "gate_length"}], "name": "cx6_1"}, {"qubits": [6, 5], "gate": "cx", "parameters": [{"date": "2019-10-08T13:54:37+00:00", "unit": "", "value": 0.010308156281825964, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 241.77777777777777, "name": "gate_length"}], "name": "cx6_5"}, {"qubits": [6, 7], "gate": "cx", "parameters": [{"date": "2019-10-08T14:01:51+00:00", "unit": "", "value": 0.01002416268455436, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 256.0, "name": "gate_length"}], "name": "cx6_7"}, {"qubits": [7, 6], "gate": "cx", "parameters": [{"date": "2019-10-08T14:01:51+00:00", "unit": "", "value": 0.01002416268455436, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 256.0, "name": "gate_length"}], "name": "cx7_6"}, {"qubits": [7, 8], "gate": "cx", "parameters": [{"date": "2019-10-08T14:13:10+00:00", "unit": "", "value": 0.01145315064928265, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 412.4444444444444, "name": "gate_length"}], "name": "cx7_8"}, {"qubits": [7, 12], "gate": "cx", "parameters": [{"date": "2019-10-08T14:30:49+00:00", "unit": "", "value": 0.010994247837299465, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 270.22222222222223, "name": "gate_length"}], "name": "cx7_12"}, {"qubits": [8, 3], "gate": "cx", "parameters": [{"date": "2019-10-08T13:50:21+00:00", "unit": "", "value": 0.00895720102940653, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 327.1111111111111, "name": "gate_length"}], "name": "cx8_3"}, {"qubits": [8, 7], "gate": "cx", "parameters": [{"date": "2019-10-08T14:13:10+00:00", "unit": "", "value": 0.01145315064928265, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 412.4444444444444, "name": "gate_length"}], "name": "cx8_7"}, {"qubits": [8, 9], "gate": "cx", "parameters": [{"date": "2019-10-08T14:18:54+00:00", "unit": "", "value": 0.012436526525968605, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 348.4444444444444, "name": "gate_length"}], "name": "cx8_9"}, {"qubits": [9, 8], "gate": "cx", "parameters": [{"date": "2019-10-08T14:18:54+00:00", "unit": "", "value": 0.012436526525968605, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 348.4444444444444, "name": "gate_length"}], "name": "cx9_8"}, {"qubits": [9, 14], "gate": "cx", "parameters": [{"date": "2019-10-08T14:53:57+00:00", "unit": "", "value": 0.014850441649694512, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 512.0, "name": "gate_length"}], "name": "cx9_14"}, {"qubits": [10, 5], "gate": "cx", "parameters": [{"date": "2019-10-08T14:25:11+00:00", "unit": "", "value": 0.013522194386392322, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 270.22222222222223, "name": "gate_length"}], "name": "cx10_5"}, {"qubits": [10, 11], "gate": "cx", "parameters": [{"date": "2019-10-08T14:36:25+00:00", "unit": "", "value": 0.012364974892208141, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 632.8888888888888, "name": "gate_length"}], "name": "cx10_11"}, {"qubits": [11, 10], "gate": "cx", "parameters": [{"date": "2019-10-08T14:36:25+00:00", "unit": "", "value": 0.012364974892208141, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 632.8888888888888, "name": "gate_length"}], "name": "cx11_10"}, {"qubits": [11, 12], "gate": "cx", "parameters": [{"date": "2019-10-08T14:42:12+00:00", "unit": "", "value": 0.011796265985676652, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 327.1111111111111, "name": "gate_length"}], "name": "cx11_12"}, {"qubits": [11, 16], "gate": "cx", "parameters": [{"date": "2019-10-08T14:59:47+00:00", "unit": "", "value": 0.011738155139105588, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 448.0, "name": "gate_length"}], "name": "cx11_16"}, {"qubits": [12, 7], "gate": "cx", "parameters": [{"date": "2019-10-08T14:30:49+00:00", "unit": "", "value": 0.010994247837299465, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 270.22222222222223, "name": "gate_length"}], "name": "cx12_7"}, {"qubits": [12, 11], "gate": "cx", "parameters": [{"date": "2019-10-08T14:42:12+00:00", "unit": "", "value": 0.011796265985676652, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 327.1111111111111, "name": "gate_length"}], "name": "cx12_11"}, {"qubits": [12, 13], "gate": "cx", "parameters": [{"date": "2019-10-08T14:47:57+00:00", "unit": "", "value": 0.011515789543863064, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 384.0, "name": "gate_length"}], "name": "cx12_13"}, {"qubits": [13, 12], "gate": "cx", "parameters": [{"date": "2019-10-08T14:47:57+00:00", "unit": "", "value": 0.011515789543863064, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 384.0, "name": "gate_length"}], "name": "cx13_12"}, {"qubits": [13, 14], "gate": "cx", "parameters": [{"date": "2019-10-08T14:51:04+00:00", "unit": "", "value": 0.008283569960770426, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 334.22222222222223, "name": "gate_length"}], "name": "cx13_14"}, {"qubits": [13, 18], "gate": "cx", "parameters": [{"date": "2019-10-08T15:10:53+00:00", "unit": "", "value": 0.018717524573515665, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 312.88888888888886, "name": "gate_length"}], "name": "cx13_18"}, {"qubits": [14, 9], "gate": "cx", "parameters": [{"date": "2019-10-08T14:53:57+00:00", "unit": "", "value": 0.014850441649694512, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 512.0, "name": "gate_length"}], "name": "cx14_9"}, {"qubits": [14, 13], "gate": "cx", "parameters": [{"date": "2019-10-08T14:51:04+00:00", "unit": "", "value": 0.008283569960770426, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 334.22222222222223, "name": "gate_length"}], "name": "cx14_13"}, {"qubits": [15, 16], "gate": "cx", "parameters": [{"date": "2019-10-08T15:04:16+00:00", "unit": "", "value": 0.00905237381849966, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 448.0, "name": "gate_length"}], "name": "cx15_16"}, {"qubits": [16, 11], "gate": "cx", "parameters": [{"date": "2019-10-08T14:59:47+00:00", "unit": "", "value": 0.011738155139105588, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 448.0, "name": "gate_length"}], "name": "cx16_11"}, {"qubits": [16, 15], "gate": "cx", "parameters": [{"date": "2019-10-08T15:04:16+00:00", "unit": "", "value": 0.00905237381849966, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 448.0, "name": "gate_length"}], "name": "cx16_15"}, {"qubits": [16, 17], "gate": "cx", "parameters": [{"date": "2019-10-08T15:06:54+00:00", "unit": "", "value": 0.01168188511789725, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 341.3333333333333, "name": "gate_length"}], "name": "cx16_17"}, {"qubits": [17, 16], "gate": "cx", "parameters": [{"date": "2019-10-08T15:06:54+00:00", "unit": "", "value": 0.01168188511789725, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 341.3333333333333, "name": "gate_length"}], "name": "cx17_16"}, {"qubits": [17, 18], "gate": "cx", "parameters": [{"date": "2019-10-08T15:31:53+00:00", "unit": "", "value": 0.020976239712194034, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 746.6666666666666, "name": "gate_length"}], "name": "cx17_18"}, {"qubits": [18, 13], "gate": "cx", "parameters": [{"date": "2019-10-08T15:10:53+00:00", "unit": "", "value": 0.018717524573515665, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 312.88888888888886, "name": "gate_length"}], "name": "cx18_13"}, {"qubits": [18, 17], "gate": "cx", "parameters": [{"date": "2019-10-08T15:31:53+00:00", "unit": "", "value": 0.020976239712194034, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 746.6666666666666, "name": "gate_length"}], "name": "cx18_17"}, {"qubits": [18, 19], "gate": "cx", "parameters": [{"date": "2019-10-08T15:37:16+00:00", "unit": "", "value": 0.028018452831777257, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 440.88888888888886, "name": "gate_length"}], "name": "cx18_19"}, {"qubits": [19, 18], "gate": "cx", "parameters": [{"date": "2019-10-08T15:37:16+00:00", "unit": "", "value": 0.028018452831777257, "name": "gate_error"}, {"date": "2019-10-08T15:37:16+00:00", "unit": "ns", "value": 440.88888888888886, "name": "gate_length"}], "name": "cx19_18"}], "backend_version": "1.0.2", "backend_name": "ibmq_boeblingen", "last_update_date": "2019-10-08T15:37:16+00:00"}'
with open(props_boeblingen_target_filepath, 'w') as f:
  f.write(props_boeblingen_str)

props_johannesburg_target_filepath = base_dir + "props_johannesburg.json"
props_johannesburg_str = '{"general": [], "qubits": [[{"date": "2019-10-07T09:17:06+00:00", "unit": "\u00b5s", "value": 79.90862967923539, "name": "T1"}, {"date": "2019-10-08T08:55:52+00:00", "unit": "\u00b5s", "value": 51.70490169618038, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 4.664855849870114, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.027000000000000024, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.029000000000000026, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.025, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:52:47+00:00", "unit": "\u00b5s", "value": 80.07419199360729, "name": "T1"}, {"date": "2019-10-08T08:57:55+00:00", "unit": "\u00b5s", "value": 94.0391293234105, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 4.767706651154839, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.031500000000000083, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.04600000000000004, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.017, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:52:47+00:00", "unit": "\u00b5s", "value": 59.10105873110132, "name": "T1"}, {"date": "2019-10-08T08:55:52+00:00", "unit": "\u00b5s", "value": 73.98164351582824, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 4.6182469984349135, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.041000000000000036, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.050000000000000044, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.032, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:52:47+00:00", "unit": "\u00b5s", "value": 54.426318483055596, "name": "T1"}, {"date": "2019-10-08T08:57:55+00:00", "unit": "\u00b5s", "value": 71.13699364215981, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 5.022873277819942, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.031500000000000083, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.039000000000000035, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.024, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:52:47+00:00", "unit": "\u00b5s", "value": 70.68011971943773, "name": "T1"}, {"date": "2019-10-08T08:55:52+00:00", "unit": "\u00b5s", "value": 87.65496684766875, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 4.655469902368598, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.03849999999999998, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.04600000000000004, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.031, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:52:47+00:00", "unit": "\u00b5s", "value": 74.15173498463481, "name": "T1"}, {"date": "2019-10-08T08:57:55+00:00", "unit": "\u00b5s", "value": 50.34061712105218, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 4.745969359620675, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.025000000000000022, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.03500000000000003, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.015, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:52:47+00:00", "unit": "\u00b5s", "value": 62.03503971287683, "name": "T1"}, {"date": "2019-10-08T08:55:52+00:00", "unit": "\u00b5s", "value": 82.39741086620967, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 4.8313700685867, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.022499999999999964, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.031000000000000028, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.014, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:52:47+00:00", "unit": "\u00b5s", "value": 86.32730030065105, "name": "T1"}, {"date": "2019-10-08T08:57:55+00:00", "unit": "\u00b5s", "value": 76.81807041370062, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 4.706046370302528, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.041000000000000036, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.051000000000000045, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.031, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T09:17:06+00:00", "unit": "\u00b5s", "value": 50.64200781594366, "name": "T1"}, {"date": "2019-10-08T08:55:52+00:00", "unit": "\u00b5s", "value": 62.72647819383072, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 4.8986213412052075, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.02300000000000002, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.034, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.01200000000000001, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:52:47+00:00", "unit": "\u00b5s", "value": 72.96808780154093, "name": "T1"}, {"date": "2019-10-08T08:57:55+00:00", "unit": "\u00b5s", "value": 96.06295971500931, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 4.740779700794559, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.014499999999999957, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.020000000000000018, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.009, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:52:47+00:00", "unit": "\u00b5s", "value": 19.97842761685982, "name": "T1"}, {"date": "2019-10-08T08:55:52+00:00", "unit": "\u00b5s", "value": 10.757360634016502, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 4.841220957752294, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.033500000000000085, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.04400000000000004, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.023, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:52:47+00:00", "unit": "\u00b5s", "value": 36.52070655732671, "name": "T1"}, {"date": "2019-10-08T08:57:55+00:00", "unit": "\u00b5s", "value": 52.18517536156183, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 4.75712328672827, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.02400000000000002, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.033, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.015000000000000013, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:52:47+00:00", "unit": "\u00b5s", "value": 92.90147625789466, "name": "T1"}, {"date": "2019-10-08T08:55:52+00:00", "unit": "\u00b5s", "value": 98.44616479171279, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 4.619451718041902, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.031500000000000083, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.04200000000000004, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.021, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:52:47+00:00", "unit": "\u00b5s", "value": 71.54009896146913, "name": "T1"}, {"date": "2019-10-08T08:57:55+00:00", "unit": "\u00b5s", "value": 68.43248444903838, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 4.853960616271936, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.024499999999999966, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.026000000000000023, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.023, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:52:47+00:00", "unit": "\u00b5s", "value": 76.99825024203868, "name": "T1"}, {"date": "2019-10-08T08:55:52+00:00", "unit": "\u00b5s", "value": 42.821158460863714, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 4.401445053082915, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.11749999999999994, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.09299999999999997, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.142, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:52:47+00:00", "unit": "\u00b5s", "value": 71.41502634976783, "name": "T1"}, {"date": "2019-10-08T08:57:55+00:00", "unit": "\u00b5s", "value": 77.57362176321678, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 4.693973464532125, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.03700000000000003, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.044, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.030000000000000027, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:52:47+00:00", "unit": "\u00b5s", "value": 86.5175474350321, "name": "T1"}, {"date": "2019-10-08T08:55:52+00:00", "unit": "\u00b5s", "value": 81.82777837984511, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 4.51262335806064, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.03849999999999998, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.051000000000000045, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.026, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:52:47+00:00", "unit": "\u00b5s", "value": 56.5541155189968, "name": "T1"}, {"date": "2019-10-08T08:57:55+00:00", "unit": "\u00b5s", "value": 75.84360452986378, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 4.724594025206416, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.01750000000000007, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.030000000000000027, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.005, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:52:47+00:00", "unit": "\u00b5s", "value": 58.312264427248735, "name": "T1"}, {"date": "2019-10-08T08:55:52+00:00", "unit": "\u00b5s", "value": 80.43160739056886, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 4.335040231422692, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.07499999999999996, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.063, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.08699999999999997, "name": "prob_meas1_prep0"}], [{"date": "2019-10-08T08:52:47+00:00", "unit": "\u00b5s", "value": 106.93634291729703, "name": "T1"}, {"date": "2019-10-08T08:57:55+00:00", "unit": "\u00b5s", "value": 106.26709205776956, "name": "T2"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "GHz", "value": 4.586048166982023, "name": "frequency"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.045500000000000096, "name": "readout_error"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.05500000000000005, "name": "prob_meas0_prep1"}, {"date": "2019-10-08T08:51:42+00:00", "unit": "", "value": 0.036, "name": "prob_meas1_prep0"}]], "gates": [{"qubits": [0], "gate": "id", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0004284937617407187, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_0"}, {"qubits": [0], "gate": "u1", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_0"}, {"qubits": [0], "gate": "u2", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0004284937617407187, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_0"}, {"qubits": [0], "gate": "u3", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0008569875234814374, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_0"}, {"qubits": [1], "gate": "id", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.00028041973815140796, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_1"}, {"qubits": [1], "gate": "u1", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_1"}, {"qubits": [1], "gate": "u2", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.00028041973815140796, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_1"}, {"qubits": [1], "gate": "u3", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0005608394763028159, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_1"}, {"qubits": [2], "gate": "id", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0012383353326959215, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_2"}, {"qubits": [2], "gate": "u1", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_2"}, {"qubits": [2], "gate": "u2", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0012383353326959215, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_2"}, {"qubits": [2], "gate": "u3", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.002476670665391843, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_2"}, {"qubits": [3], "gate": "id", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0003188280537731939, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_3"}, {"qubits": [3], "gate": "u1", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_3"}, {"qubits": [3], "gate": "u2", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0003188280537731939, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_3"}, {"qubits": [3], "gate": "u3", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0006376561075463878, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_3"}, {"qubits": [4], "gate": "id", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.00029987401735934757, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_4"}, {"qubits": [4], "gate": "u1", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_4"}, {"qubits": [4], "gate": "u2", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.00029987401735934757, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_4"}, {"qubits": [4], "gate": "u3", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0005997480347186951, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_4"}, {"qubits": [5], "gate": "id", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0004401639231731916, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_5"}, {"qubits": [5], "gate": "u1", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_5"}, {"qubits": [5], "gate": "u2", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0004401639231731916, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_5"}, {"qubits": [5], "gate": "u3", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0008803278463463833, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_5"}, {"qubits": [6], "gate": "id", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.00044019770352380456, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_6"}, {"qubits": [6], "gate": "u1", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_6"}, {"qubits": [6], "gate": "u2", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.00044019770352380456, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_6"}, {"qubits": [6], "gate": "u3", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0008803954070476091, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_6"}, {"qubits": [7], "gate": "id", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0003086298364448771, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_7"}, {"qubits": [7], "gate": "u1", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_7"}, {"qubits": [7], "gate": "u2", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0003086298364448771, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_7"}, {"qubits": [7], "gate": "u3", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0006172596728897542, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_7"}, {"qubits": [8], "gate": "id", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0003371547300813942, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_8"}, {"qubits": [8], "gate": "u1", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_8"}, {"qubits": [8], "gate": "u2", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0003371547300813942, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_8"}, {"qubits": [8], "gate": "u3", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0006743094601627884, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_8"}, {"qubits": [9], "gate": "id", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0003045379857555012, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_9"}, {"qubits": [9], "gate": "u1", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_9"}, {"qubits": [9], "gate": "u2", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0003045379857555012, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_9"}, {"qubits": [9], "gate": "u3", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0006090759715110024, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_9"}, {"qubits": [10], "gate": "id", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0007866613177609661, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_10"}, {"qubits": [10], "gate": "u1", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_10"}, {"qubits": [10], "gate": "u2", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0007866613177609661, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_10"}, {"qubits": [10], "gate": "u3", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0015733226355219322, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_10"}, {"qubits": [11], "gate": "id", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.00040707618759562683, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_11"}, {"qubits": [11], "gate": "u1", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_11"}, {"qubits": [11], "gate": "u2", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.00040707618759562683, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_11"}, {"qubits": [11], "gate": "u3", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0008141523751912537, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_11"}, {"qubits": [12], "gate": "id", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.00032997519854414676, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_12"}, {"qubits": [12], "gate": "u1", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_12"}, {"qubits": [12], "gate": "u2", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.00032997519854414676, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_12"}, {"qubits": [12], "gate": "u3", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0006599503970882935, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_12"}, {"qubits": [13], "gate": "id", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0003553494636307493, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_13"}, {"qubits": [13], "gate": "u1", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_13"}, {"qubits": [13], "gate": "u2", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0003553494636307493, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_13"}, {"qubits": [13], "gate": "u3", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0007106989272614985, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_13"}, {"qubits": [14], "gate": "id", "parameters": [{"date": "2019-10-08T08:59:39+00:00", "unit": "", "value": 0.0002689157824180266, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_14"}, {"qubits": [14], "gate": "u1", "parameters": [{"date": "2019-10-08T08:59:39+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_14"}, {"qubits": [14], "gate": "u2", "parameters": [{"date": "2019-10-08T08:59:39+00:00", "unit": "", "value": 0.0002689157824180266, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_14"}, {"qubits": [14], "gate": "u3", "parameters": [{"date": "2019-10-08T08:59:39+00:00", "unit": "", "value": 0.0005378315648360532, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_14"}, {"qubits": [15], "gate": "id", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0002994661029364117, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_15"}, {"qubits": [15], "gate": "u1", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_15"}, {"qubits": [15], "gate": "u2", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0002994661029364117, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_15"}, {"qubits": [15], "gate": "u3", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0005989322058728234, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_15"}, {"qubits": [16], "gate": "id", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.00030437103794333446, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_16"}, {"qubits": [16], "gate": "u1", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_16"}, {"qubits": [16], "gate": "u2", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.00030437103794333446, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_16"}, {"qubits": [16], "gate": "u3", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0006087420758866689, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_16"}, {"qubits": [17], "gate": "id", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.00041648977197195677, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_17"}, {"qubits": [17], "gate": "u1", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_17"}, {"qubits": [17], "gate": "u2", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.00041648977197195677, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_17"}, {"qubits": [17], "gate": "u3", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0008329795439439135, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_17"}, {"qubits": [18], "gate": "id", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0002970457343260588, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_18"}, {"qubits": [18], "gate": "u1", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_18"}, {"qubits": [18], "gate": "u2", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0002970457343260588, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_18"}, {"qubits": [18], "gate": "u3", "parameters": [{"date": "2019-10-08T09:01:05+00:00", "unit": "", "value": 0.0005940914686521176, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_18"}, {"qubits": [19], "gate": "id", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.00023870868394927587, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_19"}, {"qubits": [19], "gate": "u1", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_19"}, {"qubits": [19], "gate": "u2", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.00023870868394927587, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_19"}, {"qubits": [19], "gate": "u3", "parameters": [{"date": "2019-10-08T09:06:34+00:00", "unit": "", "value": 0.00047741736789855174, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_19"}, {"qubits": [0, 1], "gate": "cx", "parameters": [{"date": "2019-10-08T09:10:35+00:00", "unit": "", "value": 0.0076000366115634155, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 270.22222222222223, "name": "gate_length"}], "name": "cx0_1"}, {"qubits": [0, 5], "gate": "cx", "parameters": [{"date": "2019-10-08T09:50:05+00:00", "unit": "", "value": 0.00991276818968237, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 334.22222222222223, "name": "gate_length"}], "name": "cx0_5"}, {"qubits": [1, 0], "gate": "cx", "parameters": [{"date": "2019-10-08T09:10:35+00:00", "unit": "", "value": 0.0076000366115634155, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 270.22222222222223, "name": "gate_length"}], "name": "cx1_0"}, {"qubits": [1, 2], "gate": "cx", "parameters": [{"date": "2019-10-08T09:18:54+00:00", "unit": "", "value": 0.026438656057174242, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 519.1111111111111, "name": "gate_length"}], "name": "cx1_2"}, {"qubits": [2, 1], "gate": "cx", "parameters": [{"date": "2019-10-08T09:18:54+00:00", "unit": "", "value": 0.026438656057174242, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 519.1111111111111, "name": "gate_length"}], "name": "cx2_1"}, {"qubits": [2, 3], "gate": "cx", "parameters": [{"date": "2019-10-08T09:29:29+00:00", "unit": "", "value": 0.029567546045012022, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 853.3333333333333, "name": "gate_length"}], "name": "cx2_3"}, {"qubits": [3, 2], "gate": "cx", "parameters": [{"date": "2019-10-08T09:29:29+00:00", "unit": "", "value": 0.029567546045012022, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 853.3333333333333, "name": "gate_length"}], "name": "cx3_2"}, {"qubits": [3, 4], "gate": "cx", "parameters": [{"date": "2019-10-08T09:38:58+00:00", "unit": "", "value": 0.048653489972689146, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 1329.7777777777776, "name": "gate_length"}], "name": "cx3_4"}, {"qubits": [4, 3], "gate": "cx", "parameters": [{"date": "2019-10-08T09:38:58+00:00", "unit": "", "value": 0.048653489972689146, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 1329.7777777777776, "name": "gate_length"}], "name": "cx4_3"}, {"qubits": [4, 9], "gate": "cx", "parameters": [{"date": "2019-10-08T10:37:04+00:00", "unit": "", "value": 0.007432563241399126, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 398.2222222222222, "name": "gate_length"}], "name": "cx4_9"}, {"qubits": [5, 0], "gate": "cx", "parameters": [{"date": "2019-10-08T09:50:05+00:00", "unit": "", "value": 0.00991276818968237, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 334.22222222222223, "name": "gate_length"}], "name": "cx5_0"}, {"qubits": [5, 6], "gate": "cx", "parameters": [{"date": "2019-10-08T09:58:20+00:00", "unit": "", "value": 0.008644481324433267, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 291.55555555555554, "name": "gate_length"}], "name": "cx5_6"}, {"qubits": [5, 10], "gate": "cx", "parameters": [{"date": "2019-10-08T10:56:53+00:00", "unit": "", "value": 0.020377777816212805, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 490.66666666666663, "name": "gate_length"}], "name": "cx5_10"}, {"qubits": [6, 5], "gate": "cx", "parameters": [{"date": "2019-10-08T09:58:20+00:00", "unit": "", "value": 0.008644481324433267, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 291.55555555555554, "name": "gate_length"}], "name": "cx6_5"}, {"qubits": [6, 7], "gate": "cx", "parameters": [{"date": "2019-10-08T10:08:09+00:00", "unit": "", "value": 0.01204500121546298, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 597.3333333333333, "name": "gate_length"}], "name": "cx6_7"}, {"qubits": [7, 6], "gate": "cx", "parameters": [{"date": "2019-10-08T10:08:09+00:00", "unit": "", "value": 0.01204500121546298, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 597.3333333333333, "name": "gate_length"}], "name": "cx7_6"}, {"qubits": [7, 8], "gate": "cx", "parameters": [{"date": "2019-10-08T10:25:10+00:00", "unit": "", "value": 0.011837962147037417, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 284.44444444444446, "name": "gate_length"}], "name": "cx7_8"}, {"qubits": [7, 12], "gate": "cx", "parameters": [{"date": "2019-10-08T10:15:43+00:00", "unit": "", "value": 0.010385642316039612, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 490.66666666666663, "name": "gate_length"}], "name": "cx7_12"}, {"qubits": [8, 7], "gate": "cx", "parameters": [{"date": "2019-10-08T10:25:10+00:00", "unit": "", "value": 0.011837962147037417, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 284.44444444444446, "name": "gate_length"}], "name": "cx8_7"}, {"qubits": [8, 9], "gate": "cx", "parameters": [{"date": "2019-10-08T10:45:21+00:00", "unit": "", "value": 0.00943932799739211, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 419.55555555555554, "name": "gate_length"}], "name": "cx8_9"}, {"qubits": [9, 4], "gate": "cx", "parameters": [{"date": "2019-10-08T10:37:04+00:00", "unit": "", "value": 0.007432563241399126, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 398.2222222222222, "name": "gate_length"}], "name": "cx9_4"}, {"qubits": [9, 8], "gate": "cx", "parameters": [{"date": "2019-10-08T10:45:21+00:00", "unit": "", "value": 0.00943932799739211, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 419.55555555555554, "name": "gate_length"}], "name": "cx9_8"}, {"qubits": [9, 14], "gate": "cx", "parameters": [{"date": "2019-10-08T11:46:14+00:00", "unit": "", "value": 0.020337534868602153, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 853.3333333333333, "name": "gate_length"}], "name": "cx9_14"}, {"qubits": [10, 5], "gate": "cx", "parameters": [{"date": "2019-10-08T10:56:53+00:00", "unit": "", "value": 0.020377777816212805, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 490.66666666666663, "name": "gate_length"}], "name": "cx10_5"}, {"qubits": [10, 11], "gate": "cx", "parameters": [{"date": "2019-10-08T11:07:05+00:00", "unit": "", "value": 0.013506274684092462, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 298.66666666666663, "name": "gate_length"}], "name": "cx10_11"}, {"qubits": [10, 15], "gate": "cx", "parameters": [{"date": "2019-10-08T11:58:18+00:00", "unit": "", "value": 0.01883765780500679, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 519.1111111111111, "name": "gate_length"}], "name": "cx10_15"}, {"qubits": [11, 10], "gate": "cx", "parameters": [{"date": "2019-10-08T11:07:05+00:00", "unit": "", "value": 0.013506274684092462, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 298.66666666666663, "name": "gate_length"}], "name": "cx11_10"}, {"qubits": [11, 12], "gate": "cx", "parameters": [{"date": "2019-10-08T11:14:37+00:00", "unit": "", "value": 0.014438587384607793, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 369.77777777777777, "name": "gate_length"}], "name": "cx11_12"}, {"qubits": [12, 7], "gate": "cx", "parameters": [{"date": "2019-10-08T10:15:43+00:00", "unit": "", "value": 0.010385642316039612, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 490.66666666666663, "name": "gate_length"}], "name": "cx12_7"}, {"qubits": [12, 11], "gate": "cx", "parameters": [{"date": "2019-10-08T11:14:37+00:00", "unit": "", "value": 0.014438587384607793, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 369.77777777777777, "name": "gate_length"}], "name": "cx12_11"}, {"qubits": [12, 13], "gate": "cx", "parameters": [{"date": "2019-10-08T11:27:15+00:00", "unit": "", "value": 0.008373214198481055, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 312.88888888888886, "name": "gate_length"}], "name": "cx12_13"}, {"qubits": [13, 12], "gate": "cx", "parameters": [{"date": "2019-10-08T11:27:15+00:00", "unit": "", "value": 0.008373214198481055, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 312.88888888888886, "name": "gate_length"}], "name": "cx13_12"}, {"qubits": [13, 14], "gate": "cx", "parameters": [{"date": "2019-10-08T11:37:46+00:00", "unit": "", "value": 0.03333632311422591, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 1920.0, "name": "gate_length"}], "name": "cx13_14"}, {"qubits": [14, 9], "gate": "cx", "parameters": [{"date": "2019-10-08T11:46:14+00:00", "unit": "", "value": 0.020337534868602153, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 853.3333333333333, "name": "gate_length"}], "name": "cx14_9"}, {"qubits": [14, 13], "gate": "cx", "parameters": [{"date": "2019-10-08T11:37:46+00:00", "unit": "", "value": 0.03333632311422591, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 1920.0, "name": "gate_length"}], "name": "cx14_13"}, {"qubits": [14, 19], "gate": "cx", "parameters": [{"date": "2019-10-08T12:35:33+00:00", "unit": "", "value": 0.009553898559408358, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 448.0, "name": "gate_length"}], "name": "cx14_19"}, {"qubits": [15, 10], "gate": "cx", "parameters": [{"date": "2019-10-08T11:58:18+00:00", "unit": "", "value": 0.01883765780500679, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 519.1111111111111, "name": "gate_length"}], "name": "cx15_10"}, {"qubits": [15, 16], "gate": "cx", "parameters": [{"date": "2019-10-08T12:04:48+00:00", "unit": "", "value": 0.009970557581103379, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 384.0, "name": "gate_length"}], "name": "cx15_16"}, {"qubits": [16, 15], "gate": "cx", "parameters": [{"date": "2019-10-08T12:04:48+00:00", "unit": "", "value": 0.009970557581103379, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 384.0, "name": "gate_length"}], "name": "cx16_15"}, {"qubits": [16, 17], "gate": "cx", "parameters": [{"date": "2019-10-08T12:11:54+00:00", "unit": "", "value": 0.01642147875866587, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 753.7777777777777, "name": "gate_length"}], "name": "cx16_17"}, {"qubits": [17, 16], "gate": "cx", "parameters": [{"date": "2019-10-08T12:11:54+00:00", "unit": "", "value": 0.01642147875866587, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 753.7777777777777, "name": "gate_length"}], "name": "cx17_16"}, {"qubits": [17, 18], "gate": "cx", "parameters": [{"date": "2019-10-08T12:22:55+00:00", "unit": "", "value": 0.02678926978908469, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 1216.0, "name": "gate_length"}], "name": "cx17_18"}, {"qubits": [18, 17], "gate": "cx", "parameters": [{"date": "2019-10-08T12:22:55+00:00", "unit": "", "value": 0.02678926978908469, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 1216.0, "name": "gate_length"}], "name": "cx18_17"}, {"qubits": [18, 19], "gate": "cx", "parameters": [{"date": "2019-10-08T12:46:37+00:00", "unit": "", "value": 0.010726627979331549, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 576.0, "name": "gate_length"}], "name": "cx18_19"}, {"qubits": [19, 14], "gate": "cx", "parameters": [{"date": "2019-10-08T12:35:33+00:00", "unit": "", "value": 0.009553898559408358, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 448.0, "name": "gate_length"}], "name": "cx19_14"}, {"qubits": [19, 18], "gate": "cx", "parameters": [{"date": "2019-10-08T12:46:37+00:00", "unit": "", "value": 0.010726627979331549, "name": "gate_error"}, {"date": "2019-10-08T16:18:03+00:00", "unit": "ns", "value": 576.0, "name": "gate_length"}], "name": "cx19_18"}], "backend_version": "1.0.11", "backend_name": "ibmq_johannesburg", "last_update_date": "2019-10-08T16:18:03+00:00"}'
with open(props_johannesburg_target_filepath, 'w') as f:
  f.write(props_johannesburg_str)

props_melbourne_target_filepath = base_dir + "props_melbourne.json"
props_melbourne_str = '{"backend_version": "1.0.0", "general": [], "last_update_date": "2019-08-30T08:40:39+00:00", "backend_name": "ibmq_16_melbourne", "qubits": [[{"date": "2019-08-30T06:25:05+00:00", "unit": "\u00b5s", "value": 50.77314177960378, "name": "T1"}, {"date": "2019-08-30T06:26:18+00:00", "unit": "\u00b5s", "value": 23.26716550447232, "name": "T2"}, {"date": "2019-08-30T08:40:39+00:00", "unit": "GHz", "value": 5.100143648658738, "name": "frequency"}, {"date": "2019-08-30T06:24:42+00:00", "unit": "", "value": 0.0816, "name": "readout_error"}], [{"date": "2019-08-30T06:25:05+00:00", "unit": "\u00b5s", "value": 75.03457165555137, "name": "T1"}, {"date": "2019-08-30T06:27:17+00:00", "unit": "\u00b5s", "value": 123.45446126755621, "name": "T2"}, {"date": "2019-08-30T08:40:39+00:00", "unit": "GHz", "value": 5.238515519242375, "name": "frequency"}, {"date": "2019-08-30T06:24:42+00:00", "unit": "", "value": 0.05669999999999997, "name": "readout_error"}], [{"date": "2019-08-30T06:25:05+00:00", "unit": "\u00b5s", "value": 50.275404762879674, "name": "T1"}, {"date": "2019-08-30T06:28:16+00:00", "unit": "\u00b5s", "value": 97.1824505756254, "name": "T2"}, {"date": "2019-08-30T08:40:39+00:00", "unit": "GHz", "value": 5.0325918748371725, "name": "frequency"}, {"date": "2019-08-30T06:24:42+00:00", "unit": "", "value": 0.03649999999999998, "name": "readout_error"}], [{"date": "2019-08-30T06:25:05+00:00", "unit": "\u00b5s", "value": 77.06523067691023, "name": "T1"}, {"date": "2019-08-30T06:29:16+00:00", "unit": "\u00b5s", "value": 65.88343921328119, "name": "T2"}, {"date": "2019-08-30T08:40:39+00:00", "unit": "GHz", "value": 4.89622995143569, "name": "frequency"}, {"date": "2019-08-30T06:24:42+00:00", "unit": "", "value": 0.033600000000000074, "name": "readout_error"}], [{"date": "2019-08-30T06:25:05+00:00", "unit": "\u00b5s", "value": 48.6121344326565, "name": "T1"}, {"date": "2019-08-30T06:26:18+00:00", "unit": "\u00b5s", "value": 33.43851454952923, "name": "T2"}, {"date": "2019-08-30T08:40:39+00:00", "unit": "GHz", "value": 5.030492276013032, "name": "frequency"}, {"date": "2019-08-30T06:24:42+00:00", "unit": "", "value": 0.06389999999999996, "name": "readout_error"}], [{"date": "2019-08-30T06:25:05+00:00", "unit": "\u00b5s", "value": 27.22545338386326, "name": "T1"}, {"date": "2019-08-30T06:27:17+00:00", "unit": "\u00b5s", "value": 55.30006621968561, "name": "T2"}, {"date": "2019-08-30T08:40:39+00:00", "unit": "GHz", "value": 5.067164115486811, "name": "frequency"}, {"date": "2019-08-30T06:24:42+00:00", "unit": "", "value": 0.05700000000000005, "name": "readout_error"}], [{"date": "2019-08-30T06:25:05+00:00", "unit": "\u00b5s", "value": 62.04463882018325, "name": "T1"}, {"date": "2019-08-30T06:28:16+00:00", "unit": "\u00b5s", "value": 61.95417571799565, "name": "T2"}, {"date": "2019-08-30T08:40:39+00:00", "unit": "GHz", "value": 4.923824384016654, "name": "frequency"}, {"date": "2019-08-30T06:24:42+00:00", "unit": "", "value": 0.07909999999999995, "name": "readout_error"}], [{"date": "2019-08-30T06:25:05+00:00", "unit": "\u00b5s", "value": 43.16853024912021, "name": "T1"}, {"date": "2019-08-30T06:29:16+00:00", "unit": "\u00b5s", "value": 58.427459338851314, "name": "T2"}, {"date": "2019-08-30T08:40:39+00:00", "unit": "GHz", "value": 4.974544202937344, "name": "frequency"}, {"date": "2019-08-30T06:24:42+00:00", "unit": "", "value": 0.06340000000000001, "name": "readout_error"}], [{"date": "2019-08-30T06:25:05+00:00", "unit": "\u00b5s", "value": 63.71797543850963, "name": "T1"}, {"date": "2019-08-30T06:26:18+00:00", "unit": "\u00b5s", "value": 88.67507787276234, "name": "T2"}, {"date": "2019-08-30T08:40:39+00:00", "unit": "GHz", "value": 4.739562250938468, "name": "frequency"}, {"date": "2019-08-30T06:24:42+00:00", "unit": "", "value": 0.034599999999999964, "name": "readout_error"}], [{"date": "2019-08-30T06:25:05+00:00", "unit": "\u00b5s", "value": 46.51934163391105, "name": "T1"}, {"date": "2019-08-30T06:28:16+00:00", "unit": "\u00b5s", "value": 81.94019993440179, "name": "T2"}, {"date": "2019-08-30T08:40:39+00:00", "unit": "GHz", "value": 4.963380996175847, "name": "frequency"}, {"date": "2019-08-30T06:24:42+00:00", "unit": "", "value": 0.034399999999999986, "name": "readout_error"}], [{"date": "2019-08-30T06:25:05+00:00", "unit": "\u00b5s", "value": 41.30403837241326, "name": "T1"}, {"date": "2019-08-30T06:27:17+00:00", "unit": "\u00b5s", "value": 51.44366097786817, "name": "T2"}, {"date": "2019-08-30T08:40:39+00:00", "unit": "GHz", "value": 4.945057612825996, "name": "frequency"}, {"date": "2019-08-30T06:24:42+00:00", "unit": "", "value": 0.062000000000000055, "name": "readout_error"}], [{"date": "2019-08-30T06:25:05+00:00", "unit": "\u00b5s", "value": 68.19724093725696, "name": "T1"}, {"date": "2019-08-30T06:28:16+00:00", "unit": "\u00b5s", "value": 116.05875432702656, "name": "T2"}, {"date": "2019-08-30T08:40:39+00:00", "unit": "GHz", "value": 5.005006866957911, "name": "frequency"}, {"date": "2019-08-30T06:24:42+00:00", "unit": "", "value": 0.03180000000000005, "name": "readout_error"}], [{"date": "2019-08-30T06:25:05+00:00", "unit": "\u00b5s", "value": 42.18872718994089, "name": "T1"}, {"date": "2019-08-30T06:27:17+00:00", "unit": "\u00b5s", "value": 72.39113698104636, "name": "T2"}, {"date": "2019-08-30T08:40:39+00:00", "unit": "GHz", "value": 4.759964985519018, "name": "frequency"}, {"date": "2019-08-30T06:24:42+00:00", "unit": "", "value": 0.05730000000000002, "name": "readout_error"}], [{"date": "2019-08-30T06:25:05+00:00", "unit": "\u00b5s", "value": 23.388913061674028, "name": "T1"}, {"date": "2019-08-30T06:26:18+00:00", "unit": "\u00b5s", "value": 49.568617451351685, "name": "T2"}, {"date": "2019-08-30T08:40:39+00:00", "unit": "GHz", "value": 4.968519920718844, "name": "frequency"}, {"date": "2019-08-30T06:24:42+00:00", "unit": "", "value": 0.05249999999999999, "name": "readout_error"}]], "gates": [{"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0, "name": "gate_error"}], "qubits": [0], "gate": "u1"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0022890476549879413, "name": "gate_error"}], "qubits": [0], "gate": "u2"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0045780953099758825, "name": "gate_error"}], "qubits": [0], "gate": "u3"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0, "name": "gate_error"}], "qubits": [1], "gate": "u1"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.011161011211293481, "name": "gate_error"}], "qubits": [1], "gate": "u2"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.022322022422586962, "name": "gate_error"}], "qubits": [1], "gate": "u3"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0, "name": "gate_error"}], "qubits": [2], "gate": "u1"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.004338853227556672, "name": "gate_error"}], "qubits": [2], "gate": "u2"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.008677706455113343, "name": "gate_error"}], "qubits": [2], "gate": "u3"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0, "name": "gate_error"}], "qubits": [3], "gate": "u1"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0012604117926789393, "name": "gate_error"}], "qubits": [3], "gate": "u2"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0025208235853578786, "name": "gate_error"}], "qubits": [3], "gate": "u3"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0, "name": "gate_error"}], "qubits": [4], "gate": "u1"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.004304021029278404, "name": "gate_error"}], "qubits": [4], "gate": "u2"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.008608042058556808, "name": "gate_error"}], "qubits": [4], "gate": "u3"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0, "name": "gate_error"}], "qubits": [5], "gate": "u1"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0022045855046065044, "name": "gate_error"}], "qubits": [5], "gate": "u2"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.004409171009213009, "name": "gate_error"}], "qubits": [5], "gate": "u3"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0, "name": "gate_error"}], "qubits": [6], "gate": "u1"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.003454676084759689, "name": "gate_error"}], "qubits": [6], "gate": "u2"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.006909352169519378, "name": "gate_error"}], "qubits": [6], "gate": "u3"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0, "name": "gate_error"}], "qubits": [7], "gate": "u1"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0016963998440643335, "name": "gate_error"}], "qubits": [7], "gate": "u2"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.003392799688128667, "name": "gate_error"}], "qubits": [7], "gate": "u3"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0, "name": "gate_error"}], "qubits": [8], "gate": "u1"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0016534919884557686, "name": "gate_error"}], "qubits": [8], "gate": "u2"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0033069839769115372, "name": "gate_error"}], "qubits": [8], "gate": "u3"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0, "name": "gate_error"}], "qubits": [9], "gate": "u1"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.002924197752053881, "name": "gate_error"}], "qubits": [9], "gate": "u2"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.005848395504107762, "name": "gate_error"}], "qubits": [9], "gate": "u3"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0, "name": "gate_error"}], "qubits": [10], "gate": "u1"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0019467418954691729, "name": "gate_error"}], "qubits": [10], "gate": "u2"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0038934837909383457, "name": "gate_error"}], "qubits": [10], "gate": "u3"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0, "name": "gate_error"}], "qubits": [11], "gate": "u1"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0015616057095417801, "name": "gate_error"}], "qubits": [11], "gate": "u2"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0031232114190835603, "name": "gate_error"}], "qubits": [11], "gate": "u3"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0, "name": "gate_error"}], "qubits": [12], "gate": "u1"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0038211298054479648, "name": "gate_error"}], "qubits": [12], "gate": "u2"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0076422596108959295, "name": "gate_error"}], "qubits": [12], "gate": "u3"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.0, "name": "gate_error"}], "qubits": [13], "gate": "u1"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.007137874193502647, "name": "gate_error"}], "qubits": [13], "gate": "u2"}, {"parameters": [{"date": "2019-08-30T06:30:09+00:00", "unit": "", "value": 0.014275748387005294, "name": "gate_error"}], "qubits": [13], "gate": "u3"}, {"parameters": [{"date": "2019-08-30T07:40:28+00:00", "unit": "", "value": 0.06766941423407083, "name": "gate_error"}], "qubits": [1, 0], "gate": "cx", "name": "CX1_0"}, {"parameters": [{"date": "2019-08-29T08:20:16+00:00", "unit": "", "value": 0.03801037092728929, "name": "gate_error"}], "qubits": [1, 2], "gate": "cx", "name": "CX1_2"}, {"parameters": [{"date": "2019-08-30T07:43:40+00:00", "unit": "", "value": 0.04035927412341517, "name": "gate_error"}], "qubits": [2, 3], "gate": "cx", "name": "CX2_3"}, {"parameters": [{"date": "2019-08-30T07:46:54+00:00", "unit": "", "value": 0.06554808443374471, "name": "gate_error"}], "qubits": [4, 3], "gate": "cx", "name": "CX4_3"}, {"parameters": [{"date": "2019-08-30T07:50:12+00:00", "unit": "", "value": 0.11000377135049015, "name": "gate_error"}], "qubits": [4, 10], "gate": "cx", "name": "CX4_10"}, {"parameters": [{"date": "2019-08-30T07:53:32+00:00", "unit": "", "value": 0.082271205303343, "name": "gate_error"}], "qubits": [5, 4], "gate": "cx", "name": "CX5_4"}, {"parameters": [{"date": "2019-08-30T07:56:51+00:00", "unit": "", "value": 0.08000264893528869, "name": "gate_error"}], "qubits": [5, 6], "gate": "cx", "name": "CX5_6"}, {"parameters": [{"date": "2019-08-30T08:00:47+00:00", "unit": "", "value": 0.053982705107386536, "name": "gate_error"}], "qubits": [5, 9], "gate": "cx", "name": "CX5_9"}, {"parameters": [{"date": "2019-08-30T08:04:12+00:00", "unit": "", "value": 0.044040614765997216, "name": "gate_error"}], "qubits": [6, 8], "gate": "cx", "name": "CX6_8"}, {"parameters": [{"date": "2019-08-30T08:07:39+00:00", "unit": "", "value": 0.03259217494162084, "name": "gate_error"}], "qubits": [7, 8], "gate": "cx", "name": "CX7_8"}, {"parameters": [{"date": "2019-08-30T08:10:56+00:00", "unit": "", "value": 0.05439659729448815, "name": "gate_error"}], "qubits": [9, 8], "gate": "cx", "name": "CX9_8"}, {"parameters": [{"date": "2019-08-30T08:15:16+00:00", "unit": "", "value": 0.05590172348391173, "name": "gate_error"}], "qubits": [9, 10], "gate": "cx", "name": "CX9_10"}, {"parameters": [{"date": "2019-08-30T08:25:02+00:00", "unit": "", "value": 0.03230502354557954, "name": "gate_error"}], "qubits": [11, 3], "gate": "cx", "name": "CX11_3"}, {"parameters": [{"date": "2019-08-30T08:18:40+00:00", "unit": "", "value": 0.04992714061173503, "name": "gate_error"}], "qubits": [11, 10], "gate": "cx", "name": "CX11_10"}, {"parameters": [{"date": "2019-08-30T08:21:50+00:00", "unit": "", "value": 0.0577565027812707, "name": "gate_error"}], "qubits": [11, 12], "gate": "cx", "name": "CX11_12"}, {"parameters": [{"date": "2019-08-30T08:28:18+00:00", "unit": "", "value": 0.07996541354812434, "name": "gate_error"}], "qubits": [12, 2], "gate": "cx", "name": "CX12_2"}, {"parameters": [{"date": "2019-08-30T08:32:48+00:00", "unit": "", "value": 0.20353725512052456, "name": "gate_error"}], "qubits": [13, 1], "gate": "cx", "name": "CX13_1"}, {"parameters": [{"date": "2019-08-30T08:36:49+00:00", "unit": "", "value": 0.06325184775759388, "name": "gate_error"}], "qubits": [13, 12], "gate": "cx", "name": "CX13_12"}]}'
with open(props_melbourne_target_filepath, 'w') as f:
  f.write(props_melbourne_str)

props_ourense_target_filepath = base_dir + "props_ourense.json"
props_ourense_str = '{"backend_version": "1.0.0", "general": [], "last_update_date": "2019-08-30T14:22:47+00:00", "backend_name": "ibmq_ourense", "qubits": [[{"date": "2019-08-30T12:55:17+00:00", "unit": "\u00b5s", "value": 100.37114723680725, "name": "T1"}, {"date": "2019-08-30T13:06:02+00:00", "unit": "\u00b5s", "value": 53.65920099859683, "name": "T2"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "GHz", "value": 4.819483735955884, "name": "frequency"}, {"date": "2019-08-30T12:45:05+00:00", "unit": "", "value": 0.02200000000000002, "name": "readout_error"}, {"date": "2019-08-30T12:45:05+00:00", "unit": "", "value": 0.028000000000000025, "name": "prob_meas0_prep1"}, {"date": "2019-08-30T12:45:05+00:00", "unit": "", "value": 0.016, "name": "prob_meas1_prep0"}], [{"date": "2019-08-30T12:55:17+00:00", "unit": "\u00b5s", "value": 78.12795783193603, "name": "T1"}, {"date": "2019-08-30T13:06:38+00:00", "unit": "\u00b5s", "value": 33.54159997805283, "name": "T2"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "GHz", "value": 4.891102721325479, "name": "frequency"}, {"date": "2019-08-30T12:45:05+00:00", "unit": "", "value": 0.031000000000000028, "name": "readout_error"}, {"date": "2019-08-30T12:45:05+00:00", "unit": "", "value": 0.04600000000000004, "name": "prob_meas0_prep1"}, {"date": "2019-08-30T12:45:05+00:00", "unit": "", "value": 0.016, "name": "prob_meas1_prep0"}], [{"date": "2019-08-30T12:55:17+00:00", "unit": "\u00b5s", "value": 72.9244465510776, "name": "T1"}, {"date": "2019-08-30T13:06:02+00:00", "unit": "\u00b5s", "value": 71.35873287702873, "name": "T2"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "GHz", "value": 4.71687548290116, "name": "frequency"}, {"date": "2019-08-30T12:45:05+00:00", "unit": "", "value": 0.018000000000000016, "name": "readout_error"}, {"date": "2019-08-30T12:45:05+00:00", "unit": "", "value": 0.026000000000000023, "name": "prob_meas0_prep1"}, {"date": "2019-08-30T12:45:05+00:00", "unit": "", "value": 0.01, "name": "prob_meas1_prep0"}], [{"date": "2019-08-30T12:55:17+00:00", "unit": "\u00b5s", "value": 26.190022390860502, "name": "T1"}, {"date": "2019-08-30T13:06:02+00:00", "unit": "\u00b5s", "value": 32.240945005936354, "name": "T2"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "GHz", "value": 4.78902749798773, "name": "frequency"}, {"date": "2019-08-30T12:45:05+00:00", "unit": "", "value": 0.050999999999999934, "name": "readout_error"}, {"date": "2019-08-30T12:45:05+00:00", "unit": "", "value": 0.076, "name": "prob_meas0_prep1"}, {"date": "2019-08-30T12:45:05+00:00", "unit": "", "value": 0.026000000000000023, "name": "prob_meas1_prep0"}], [{"date": "2019-08-30T12:55:17+00:00", "unit": "\u00b5s", "value": 103.04094444113744, "name": "T1"}, {"date": "2019-08-30T13:06:38+00:00", "unit": "\u00b5s", "value": 45.25375042867465, "name": "T2"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "GHz", "value": 5.023691537288777, "name": "frequency"}, {"date": "2019-08-30T12:45:05+00:00", "unit": "", "value": 0.04300000000000004, "name": "readout_error"}, {"date": "2019-08-30T12:45:05+00:00", "unit": "", "value": 0.046, "name": "prob_meas0_prep1"}, {"date": "2019-08-30T12:45:05+00:00", "unit": "", "value": 0.040000000000000036, "name": "prob_meas1_prep0"}]], "gates": [{"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.00031177333311129094, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "qubits": [0], "gate": "id", "name": "id_0"}, {"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [0], "gate": "u1", "name": "u1_0"}, {"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.00031177333311129094, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "qubits": [0], "gate": "u2", "name": "u2_0"}, {"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.0006235466662225819, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "qubits": [0], "gate": "u3", "name": "u3_0"}, {"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.000464071792789536, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "qubits": [1], "gate": "id", "name": "id_1"}, {"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [1], "gate": "u1", "name": "u1_1"}, {"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.000464071792789536, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "qubits": [1], "gate": "u2", "name": "u2_1"}, {"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.000928143585579072, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "qubits": [1], "gate": "u3", "name": "u3_1"}, {"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.0002517609248549556, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "qubits": [2], "gate": "id", "name": "id_2"}, {"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [2], "gate": "u1", "name": "u1_2"}, {"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.0002517609248549556, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "qubits": [2], "gate": "u2", "name": "u2_2"}, {"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.0005035218497099112, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "qubits": [2], "gate": "u3", "name": "u3_2"}, {"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.0012093335635556912, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "qubits": [3], "gate": "id", "name": "id_3"}, {"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [3], "gate": "u1", "name": "u1_3"}, {"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.0012093335635556912, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "qubits": [3], "gate": "u2", "name": "u2_3"}, {"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.0024186671271113824, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "qubits": [3], "gate": "u3", "name": "u3_3"}, {"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.0005593027000324215, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "qubits": [4], "gate": "id", "name": "id_4"}, {"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [4], "gate": "u1", "name": "u1_4"}, {"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.0005593027000324215, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "qubits": [4], "gate": "u2", "name": "u2_4"}, {"parameters": [{"date": "2019-08-30T13:07:50+00:00", "unit": "", "value": 0.001118605400064843, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "qubits": [4], "gate": "u3", "name": "u3_4"}, {"parameters": [{"date": "2019-08-30T13:11:13+00:00", "unit": "", "value": 0.009131957296410004, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 234.66666666666666, "name": "gate_length"}], "qubits": [0, 1], "gate": "cx", "name": "cx0_1"}, {"parameters": [{"date": "2019-08-30T13:11:13+00:00", "unit": "", "value": 0.009131957296410004, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 234.66666666666666, "name": "gate_length"}], "qubits": [1, 0], "gate": "cx", "name": "cx1_0"}, {"parameters": [{"date": "2019-08-30T13:44:41+00:00", "unit": "", "value": 0.010095254518550478, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 391.1111111111111, "name": "gate_length"}], "qubits": [1, 2], "gate": "cx", "name": "cx1_2"}, {"parameters": [{"date": "2019-08-30T14:00:02+00:00", "unit": "", "value": 0.0219516066297471, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 576.0, "name": "gate_length"}], "qubits": [1, 3], "gate": "cx", "name": "cx1_3"}, {"parameters": [{"date": "2019-08-30T13:44:41+00:00", "unit": "", "value": 0.010095254518550478, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 391.1111111111111, "name": "gate_length"}], "qubits": [2, 1], "gate": "cx", "name": "cx2_1"}, {"parameters": [{"date": "2019-08-30T14:00:02+00:00", "unit": "", "value": 0.0219516066297471, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 576.0, "name": "gate_length"}], "qubits": [3, 1], "gate": "cx", "name": "cx3_1"}, {"parameters": [{"date": "2019-08-30T14:22:47+00:00", "unit": "", "value": 0.01179141510185705, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 270.22222222222223, "name": "gate_length"}], "qubits": [3, 4], "gate": "cx", "name": "cx3_4"}, {"parameters": [{"date": "2019-08-30T14:22:47+00:00", "unit": "", "value": 0.01179141510185705, "name": "gate_error"}, {"date": "2019-08-30T14:22:47+00:00", "unit": "ns", "value": 270.22222222222223, "name": "gate_length"}], "qubits": [4, 3], "gate": "cx", "name": "cx4_3"}]}'
with open(props_ourense_target_filepath, 'w') as f:
  f.write(props_ourense_str)

props_poughkeepsie_target_filepath = base_dir + "props_poughkeepsie.json"
props_poughkeepsie_str = '{"general": [], "qubits": [[{"date": "2019-10-06T20:47:47+00:00", "unit": "\u00b5s", "value": 74.83952330243338, "name": "T1"}, {"date": "2019-10-07T20:40:40+00:00", "unit": "\u00b5s", "value": 71.4202454959009, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 4.919892119600908, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.025000000000000022, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.040000000000000036, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.01, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T20:39:30+00:00", "unit": "\u00b5s", "value": 54.68626103330947, "name": "T1"}, {"date": "2019-10-07T20:45:51+00:00", "unit": "\u00b5s", "value": 60.70195921779713, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 4.831960664452536, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.029000000000000026, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.04600000000000004, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.012, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T20:39:30+00:00", "unit": "\u00b5s", "value": 54.71491358065838, "name": "T1"}, {"date": "2019-10-07T20:40:40+00:00", "unit": "\u00b5s", "value": 78.10680047360002, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 4.940456353916524, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.03400000000000003, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.06000000000000005, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.008, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T20:39:30+00:00", "unit": "\u00b5s", "value": 91.79892479694225, "name": "T1"}, {"date": "2019-10-07T20:45:51+00:00", "unit": "\u00b5s", "value": 91.03108645956549, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 4.5147345268577315, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.02400000000000002, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.030000000000000027, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.018, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T20:39:30+00:00", "unit": "\u00b5s", "value": 93.76815773128386, "name": "T1"}, {"date": "2019-10-07T20:40:40+00:00", "unit": "\u00b5s", "value": 125.53730248433925, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 4.662919458970246, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.04899999999999993, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.06999999999999995, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.028, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T20:39:30+00:00", "unit": "\u00b5s", "value": 78.81320050319941, "name": "T1"}, {"date": "2019-10-07T20:45:51+00:00", "unit": "\u00b5s", "value": 73.31878305794932, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 4.957353894573826, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.041000000000000036, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.050000000000000044, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.032, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T20:39:30+00:00", "unit": "\u00b5s", "value": 74.55443606215356, "name": "T1"}, {"date": "2019-10-07T20:40:40+00:00", "unit": "\u00b5s", "value": 83.24642666041788, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 4.9955667289306005, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.031000000000000028, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.038000000000000034, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.024, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T20:39:30+00:00", "unit": "\u00b5s", "value": 81.01111563369945, "name": "T1"}, {"date": "2019-10-05T20:55:27+00:00", "unit": "\u00b5s", "value": 22.42756570850294, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 4.811619192641292, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.041000000000000036, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.05600000000000005, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.026, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T20:39:30+00:00", "unit": "\u00b5s", "value": 71.123068497754, "name": "T1"}, {"date": "2019-10-07T20:40:40+00:00", "unit": "\u00b5s", "value": 91.88163087188487, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 5.013719103432012, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.05899999999999994, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.10799999999999998, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.01, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T20:39:30+00:00", "unit": "\u00b5s", "value": 51.68509818846387, "name": "T1"}, {"date": "2019-10-07T20:45:51+00:00", "unit": "\u00b5s", "value": 97.68146160856635, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 5.056299100211388, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.030000000000000027, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.04400000000000004, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.016, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T20:39:30+00:00", "unit": "\u00b5s", "value": 89.44217749808405, "name": "T1"}, {"date": "2019-10-07T20:40:40+00:00", "unit": "\u00b5s", "value": 17.87784188741667, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 4.718486297983972, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.05600000000000005, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.07399999999999995, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.038, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T20:39:30+00:00", "unit": "\u00b5s", "value": 101.87602856574809, "name": "T1"}, {"date": "2019-10-07T20:45:51+00:00", "unit": "\u00b5s", "value": 134.97336000877485, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 4.899889567567074, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.038000000000000034, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.05600000000000005, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.02, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T20:39:30+00:00", "unit": "\u00b5s", "value": 90.66859352347241, "name": "T1"}, {"date": "2019-10-07T20:40:40+00:00", "unit": "\u00b5s", "value": 18.361798056038648, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 4.772827330542938, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.03500000000000003, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.052000000000000046, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.018, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T20:39:30+00:00", "unit": "\u00b5s", "value": 56.630371989177256, "name": "T1"}, {"date": "2019-10-03T21:45:49+00:00", "unit": "\u00b5s", "value": 30.937840378708813, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 5.110855311409945, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.052000000000000046, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.09199999999999997, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.012, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T20:39:30+00:00", "unit": "\u00b5s", "value": 66.88722168094999, "name": "T1"}, {"date": "2019-10-07T20:40:40+00:00", "unit": "\u00b5s", "value": 65.44944881338212, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 4.990057161447801, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.02200000000000002, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.038000000000000034, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.006, "name": "prob_meas1_prep0"}], [{"date": "2019-10-06T20:47:47+00:00", "unit": "\u00b5s", "value": 89.0915614312045, "name": "T1"}, {"date": "2019-10-07T20:45:51+00:00", "unit": "\u00b5s", "value": 21.874335440429398, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 4.806125096701479, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.03600000000000003, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.06000000000000005, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.012, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T20:39:30+00:00", "unit": "\u00b5s", "value": 30.37708981471975, "name": "T1"}, {"date": "2019-10-07T20:40:40+00:00", "unit": "\u00b5s", "value": 41.149159357563526, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 4.955671316070989, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.15599999999999992, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.19999999999999996, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.112, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T20:39:30+00:00", "unit": "\u00b5s", "value": 66.4095213876651, "name": "T1"}, {"date": "2019-10-07T20:45:51+00:00", "unit": "\u00b5s", "value": 21.292757340112722, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 4.599299215561779, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.052000000000000046, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.07799999999999996, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.026, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T20:39:30+00:00", "unit": "\u00b5s", "value": 79.64375893667459, "name": "T1"}, {"date": "2019-10-07T20:40:40+00:00", "unit": "\u00b5s", "value": 100.80611581913071, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 4.82797989029422, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.025000000000000022, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.04200000000000004, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.008, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T20:39:30+00:00", "unit": "\u00b5s", "value": 78.3231913687434, "name": "T1"}, {"date": "2019-10-07T20:45:51+00:00", "unit": "\u00b5s", "value": 90.73162313154104, "name": "T2"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "GHz", "value": 4.938658589204626, "name": "frequency"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.041000000000000036, "name": "readout_error"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.04200000000000004, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T20:37:36+00:00", "unit": "", "value": 0.04, "name": "prob_meas1_prep0"}]], "gates": [{"qubits": [0], "gate": "id", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0010076467591284945, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_0"}, {"qubits": [0], "gate": "u1", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_0"}, {"qubits": [0], "gate": "u2", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0010076467591284945, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_0"}, {"qubits": [0], "gate": "u3", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.002015293518256989, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_0"}, {"qubits": [1], "gate": "id", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0017562067933842497, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_1"}, {"qubits": [1], "gate": "u1", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_1"}, {"qubits": [1], "gate": "u2", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0017562067933842497, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_1"}, {"qubits": [1], "gate": "u3", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0035124135867684995, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_1"}, {"qubits": [2], "gate": "id", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.001051351273010438, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_2"}, {"qubits": [2], "gate": "u1", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_2"}, {"qubits": [2], "gate": "u2", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.001051351273010438, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_2"}, {"qubits": [2], "gate": "u3", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.002102702546020876, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_2"}, {"qubits": [3], "gate": "id", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0007850689758185343, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_3"}, {"qubits": [3], "gate": "u1", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_3"}, {"qubits": [3], "gate": "u2", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0007850689758185343, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_3"}, {"qubits": [3], "gate": "u3", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0015701379516370685, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_3"}, {"qubits": [4], "gate": "id", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0007303980169498843, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_4"}, {"qubits": [4], "gate": "u1", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_4"}, {"qubits": [4], "gate": "u2", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0007303980169498843, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_4"}, {"qubits": [4], "gate": "u3", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0014607960338997685, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_4"}, {"qubits": [5], "gate": "id", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0012497194248676556, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_5"}, {"qubits": [5], "gate": "u1", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_5"}, {"qubits": [5], "gate": "u2", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0012497194248676556, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_5"}, {"qubits": [5], "gate": "u3", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0024994388497353113, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_5"}, {"qubits": [6], "gate": "id", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.001174875227741368, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_6"}, {"qubits": [6], "gate": "u1", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_6"}, {"qubits": [6], "gate": "u2", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.001174875227741368, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_6"}, {"qubits": [6], "gate": "u3", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.002349750455482736, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_6"}, {"qubits": [7], "gate": "id", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0036571994143867314, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_7"}, {"qubits": [7], "gate": "u1", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_7"}, {"qubits": [7], "gate": "u2", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0036571994143867314, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_7"}, {"qubits": [7], "gate": "u3", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.007314398828773463, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_7"}, {"qubits": [8], "gate": "id", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.002213415356051083, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_8"}, {"qubits": [8], "gate": "u1", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_8"}, {"qubits": [8], "gate": "u2", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.002213415356051083, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_8"}, {"qubits": [8], "gate": "u3", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.004426830712102166, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_8"}, {"qubits": [9], "gate": "id", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0011483347542554168, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_9"}, {"qubits": [9], "gate": "u1", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_9"}, {"qubits": [9], "gate": "u2", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0011483347542554168, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_9"}, {"qubits": [9], "gate": "u3", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0022966695085108336, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_9"}, {"qubits": [10], "gate": "id", "parameters": [{"date": "2019-10-07T20:53:23+00:00", "unit": "", "value": 0.001270662900791756, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_10"}, {"qubits": [10], "gate": "u1", "parameters": [{"date": "2019-10-07T20:53:23+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_10"}, {"qubits": [10], "gate": "u2", "parameters": [{"date": "2019-10-07T20:53:23+00:00", "unit": "", "value": 0.001270662900791756, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_10"}, {"qubits": [10], "gate": "u3", "parameters": [{"date": "2019-10-07T20:53:23+00:00", "unit": "", "value": 0.002541325801583512, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_10"}, {"qubits": [11], "gate": "id", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0006950916134566943, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_11"}, {"qubits": [11], "gate": "u1", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_11"}, {"qubits": [11], "gate": "u2", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0006950916134566943, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_11"}, {"qubits": [11], "gate": "u3", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0013901832269133886, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_11"}, {"qubits": [12], "gate": "id", "parameters": [{"date": "2019-10-07T20:53:23+00:00", "unit": "", "value": 0.0013923308431948703, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_12"}, {"qubits": [12], "gate": "u1", "parameters": [{"date": "2019-10-07T20:53:23+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_12"}, {"qubits": [12], "gate": "u2", "parameters": [{"date": "2019-10-07T20:53:23+00:00", "unit": "", "value": 0.0013923308431948703, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_12"}, {"qubits": [12], "gate": "u3", "parameters": [{"date": "2019-10-07T20:53:23+00:00", "unit": "", "value": 0.0027846616863897405, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_12"}, {"qubits": [13], "gate": "id", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0035138403022632056, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_13"}, {"qubits": [13], "gate": "u1", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_13"}, {"qubits": [13], "gate": "u2", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0035138403022632056, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_13"}, {"qubits": [13], "gate": "u3", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.007027680604526411, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_13"}, {"qubits": [14], "gate": "id", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0019024682295245429, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_14"}, {"qubits": [14], "gate": "u1", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_14"}, {"qubits": [14], "gate": "u2", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0019024682295245429, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_14"}, {"qubits": [14], "gate": "u3", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0038049364590490857, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_14"}, {"qubits": [15], "gate": "id", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.002596675651085207, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_15"}, {"qubits": [15], "gate": "u1", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_15"}, {"qubits": [15], "gate": "u2", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.002596675651085207, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_15"}, {"qubits": [15], "gate": "u3", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.005193351302170414, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_15"}, {"qubits": [16], "gate": "id", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.011569140749443901, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_16"}, {"qubits": [16], "gate": "u1", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_16"}, {"qubits": [16], "gate": "u2", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.011569140749443901, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_16"}, {"qubits": [16], "gate": "u3", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.023138281498887803, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_16"}, {"qubits": [17], "gate": "id", "parameters": [{"date": "2019-10-07T20:53:23+00:00", "unit": "", "value": 0.0011352113768054416, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_17"}, {"qubits": [17], "gate": "u1", "parameters": [{"date": "2019-10-07T20:53:23+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_17"}, {"qubits": [17], "gate": "u2", "parameters": [{"date": "2019-10-07T20:53:23+00:00", "unit": "", "value": 0.0011352113768054416, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_17"}, {"qubits": [17], "gate": "u3", "parameters": [{"date": "2019-10-07T20:53:23+00:00", "unit": "", "value": 0.002270422753610883, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_17"}, {"qubits": [18], "gate": "id", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0007077057192830814, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_18"}, {"qubits": [18], "gate": "u1", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_18"}, {"qubits": [18], "gate": "u2", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0007077057192830814, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_18"}, {"qubits": [18], "gate": "u3", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0014154114385661629, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_18"}, {"qubits": [19], "gate": "id", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0009223892992605438, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 113.77777777777777, "name": "gate_length"}], "name": "id_19"}, {"qubits": [19], "gate": "u1", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_19"}, {"qubits": [19], "gate": "u2", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0009223892992605438, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "name": "u2_19"}, {"qubits": [19], "gate": "u3", "parameters": [{"date": "2019-10-07T20:51:06+00:00", "unit": "", "value": 0.0018447785985210876, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "name": "u3_19"}, {"qubits": [0, 1], "gate": "cx", "parameters": [{"date": "2019-10-07T21:18:16+00:00", "unit": "", "value": 0.018348234720821227, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 455.1111111111111, "name": "gate_length"}], "name": "cx0_1"}, {"qubits": [0, 5], "gate": "cx", "parameters": [{"date": "2019-10-07T21:26:31+00:00", "unit": "", "value": 0.01950952631756478, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 433.77777777777777, "name": "gate_length"}], "name": "cx0_5"}, {"qubits": [1, 0], "gate": "cx", "parameters": [{"date": "2019-10-07T21:18:16+00:00", "unit": "", "value": 0.018348234720821227, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 455.1111111111111, "name": "gate_length"}], "name": "cx1_0"}, {"qubits": [1, 2], "gate": "cx", "parameters": [{"date": "2019-10-07T21:11:57+00:00", "unit": "", "value": 0.020385377535338506, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 419.55555555555554, "name": "gate_length"}], "name": "cx1_2"}, {"qubits": [2, 1], "gate": "cx", "parameters": [{"date": "2019-10-07T21:11:57+00:00", "unit": "", "value": 0.020385377535338506, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 419.55555555555554, "name": "gate_length"}], "name": "cx2_1"}, {"qubits": [2, 3], "gate": "cx", "parameters": [{"date": "2019-10-07T21:05:41+00:00", "unit": "", "value": 0.023464178979961364, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 839.1111111111111, "name": "gate_length"}], "name": "cx2_3"}, {"qubits": [3, 2], "gate": "cx", "parameters": [{"date": "2019-10-07T21:05:41+00:00", "unit": "", "value": 0.023464178979961364, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 839.1111111111111, "name": "gate_length"}], "name": "cx3_2"}, {"qubits": [3, 4], "gate": "cx", "parameters": [{"date": "2019-10-07T20:58:49+00:00", "unit": "", "value": 0.015947074681332857, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 1137.7777777777778, "name": "gate_length"}], "name": "cx3_4"}, {"qubits": [4, 3], "gate": "cx", "parameters": [{"date": "2019-10-07T20:58:49+00:00", "unit": "", "value": 0.015947074681332857, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 1137.7777777777778, "name": "gate_length"}], "name": "cx4_3"}, {"qubits": [4, 9], "gate": "cx", "parameters": [{"date": "2019-10-07T21:33:51+00:00", "unit": "", "value": 0.05166261475184261, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 888.8888888888888, "name": "gate_length"}], "name": "cx4_9"}, {"qubits": [5, 0], "gate": "cx", "parameters": [{"date": "2019-10-07T21:26:31+00:00", "unit": "", "value": 0.01950952631756478, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 433.77777777777777, "name": "gate_length"}], "name": "cx5_0"}, {"qubits": [5, 6], "gate": "cx", "parameters": [{"date": "2019-10-07T22:13:03+00:00", "unit": "", "value": 0.016225623627552893, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 455.1111111111111, "name": "gate_length"}], "name": "cx5_6"}, {"qubits": [5, 10], "gate": "cx", "parameters": [{"date": "2019-10-07T22:19:01+00:00", "unit": "", "value": 0.020354930739063998, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 462.2222222222222, "name": "gate_length"}], "name": "cx5_10"}, {"qubits": [6, 5], "gate": "cx", "parameters": [{"date": "2019-10-07T22:13:03+00:00", "unit": "", "value": 0.016225623627552893, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 455.1111111111111, "name": "gate_length"}], "name": "cx6_5"}, {"qubits": [6, 7], "gate": "cx", "parameters": [{"date": "2019-10-07T22:07:13+00:00", "unit": "", "value": 0.02789881487841339, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 618.6666666666666, "name": "gate_length"}], "name": "cx6_7"}, {"qubits": [7, 6], "gate": "cx", "parameters": [{"date": "2019-10-07T22:07:13+00:00", "unit": "", "value": 0.02789881487841339, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 618.6666666666666, "name": "gate_length"}], "name": "cx7_6"}, {"qubits": [7, 8], "gate": "cx", "parameters": [{"date": "2019-10-07T21:53:45+00:00", "unit": "", "value": 0.035117643483730226, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 398.2222222222222, "name": "gate_length"}], "name": "cx7_8"}, {"qubits": [7, 12], "gate": "cx", "parameters": [{"date": "2019-10-07T22:00:01+00:00", "unit": "", "value": 0.01705159434530748, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 554.6666666666666, "name": "gate_length"}], "name": "cx7_12"}, {"qubits": [8, 7], "gate": "cx", "parameters": [{"date": "2019-10-07T21:53:45+00:00", "unit": "", "value": 0.035117643483730226, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 398.2222222222222, "name": "gate_length"}], "name": "cx8_7"}, {"qubits": [8, 9], "gate": "cx", "parameters": [{"date": "2019-10-07T21:40:34+00:00", "unit": "", "value": 0.01719170274508794, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 426.66666666666663, "name": "gate_length"}], "name": "cx8_9"}, {"qubits": [9, 4], "gate": "cx", "parameters": [{"date": "2019-10-07T21:33:51+00:00", "unit": "", "value": 0.05166261475184261, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 888.8888888888888, "name": "gate_length"}], "name": "cx9_4"}, {"qubits": [9, 8], "gate": "cx", "parameters": [{"date": "2019-10-07T21:40:34+00:00", "unit": "", "value": 0.01719170274508794, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 426.66666666666663, "name": "gate_length"}], "name": "cx9_8"}, {"qubits": [9, 14], "gate": "cx", "parameters": [{"date": "2019-10-07T21:46:51+00:00", "unit": "", "value": 0.02168406202870754, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 568.8888888888889, "name": "gate_length"}], "name": "cx9_14"}, {"qubits": [10, 5], "gate": "cx", "parameters": [{"date": "2019-10-07T22:19:01+00:00", "unit": "", "value": 0.020354930739063998, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 462.2222222222222, "name": "gate_length"}], "name": "cx10_5"}, {"qubits": [10, 11], "gate": "cx", "parameters": [{"date": "2019-10-07T22:50:04+00:00", "unit": "", "value": 0.02126993330289842, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 661.3333333333333, "name": "gate_length"}], "name": "cx10_11"}, {"qubits": [10, 15], "gate": "cx", "parameters": [{"date": "2019-10-07T23:23:26+00:00", "unit": "", "value": 0.015545910453450856, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 476.4444444444444, "name": "gate_length"}], "name": "cx10_15"}, {"qubits": [11, 10], "gate": "cx", "parameters": [{"date": "2019-10-07T22:50:04+00:00", "unit": "", "value": 0.02126993330289842, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 661.3333333333333, "name": "gate_length"}], "name": "cx11_10"}, {"qubits": [11, 12], "gate": "cx", "parameters": [{"date": "2019-10-07T22:44:35+00:00", "unit": "", "value": 0.01687289466942163, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 448.0, "name": "gate_length"}], "name": "cx11_12"}, {"qubits": [12, 7], "gate": "cx", "parameters": [{"date": "2019-10-07T22:00:01+00:00", "unit": "", "value": 0.01705159434530748, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 554.6666666666666, "name": "gate_length"}], "name": "cx12_7"}, {"qubits": [12, 11], "gate": "cx", "parameters": [{"date": "2019-10-07T22:44:35+00:00", "unit": "", "value": 0.01687289466942163, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 448.0, "name": "gate_length"}], "name": "cx12_11"}, {"qubits": [12, 13], "gate": "cx", "parameters": [{"date": "2019-10-07T22:38:01+00:00", "unit": "", "value": 0.038297168369467116, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 618.6666666666666, "name": "gate_length"}], "name": "cx12_13"}, {"qubits": [13, 12], "gate": "cx", "parameters": [{"date": "2019-10-07T22:38:01+00:00", "unit": "", "value": 0.038297168369467116, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 618.6666666666666, "name": "gate_length"}], "name": "cx13_12"}, {"qubits": [13, 14], "gate": "cx", "parameters": [{"date": "2019-10-07T22:31:53+00:00", "unit": "", "value": 0.023990962483357997, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 398.2222222222222, "name": "gate_length"}], "name": "cx13_14"}, {"qubits": [14, 9], "gate": "cx", "parameters": [{"date": "2019-10-07T21:46:51+00:00", "unit": "", "value": 0.02168406202870754, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 568.8888888888889, "name": "gate_length"}], "name": "cx14_9"}, {"qubits": [14, 13], "gate": "cx", "parameters": [{"date": "2019-10-07T22:31:53+00:00", "unit": "", "value": 0.023990962483357997, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 398.2222222222222, "name": "gate_length"}], "name": "cx14_13"}, {"qubits": [14, 19], "gate": "cx", "parameters": [{"date": "2019-10-07T22:25:38+00:00", "unit": "", "value": 0.021277452082323373, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 469.3333333333333, "name": "gate_length"}], "name": "cx14_19"}, {"qubits": [15, 10], "gate": "cx", "parameters": [{"date": "2019-10-07T23:23:26+00:00", "unit": "", "value": 0.015545910453450856, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 476.4444444444444, "name": "gate_length"}], "name": "cx15_10"}, {"qubits": [15, 16], "gate": "cx", "parameters": [{"date": "2019-10-07T23:16:40+00:00", "unit": "", "value": 0.034187232822651986, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 789.3333333333333, "name": "gate_length"}], "name": "cx15_16"}, {"qubits": [16, 15], "gate": "cx", "parameters": [{"date": "2019-10-07T23:16:40+00:00", "unit": "", "value": 0.034187232822651986, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 789.3333333333333, "name": "gate_length"}], "name": "cx16_15"}, {"qubits": [16, 17], "gate": "cx", "parameters": [{"date": "2019-10-07T23:10:32+00:00", "unit": "", "value": 0.08244103637340802, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 704.0, "name": "gate_length"}], "name": "cx16_17"}, {"qubits": [17, 16], "gate": "cx", "parameters": [{"date": "2019-10-07T23:10:32+00:00", "unit": "", "value": 0.08244103637340802, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 704.0, "name": "gate_length"}], "name": "cx17_16"}, {"qubits": [17, 18], "gate": "cx", "parameters": [{"date": "2019-10-07T23:01:38+00:00", "unit": "", "value": 0.018285774093612706, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 668.4444444444445, "name": "gate_length"}], "name": "cx17_18"}, {"qubits": [18, 17], "gate": "cx", "parameters": [{"date": "2019-10-07T23:01:38+00:00", "unit": "", "value": 0.018285774093612706, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 668.4444444444445, "name": "gate_length"}], "name": "cx18_17"}, {"qubits": [18, 19], "gate": "cx", "parameters": [{"date": "2019-10-07T22:55:44+00:00", "unit": "", "value": 0.01285611913382742, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 419.55555555555554, "name": "gate_length"}], "name": "cx18_19"}, {"qubits": [19, 14], "gate": "cx", "parameters": [{"date": "2019-10-07T22:25:38+00:00", "unit": "", "value": 0.021277452082323373, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 469.3333333333333, "name": "gate_length"}], "name": "cx19_14"}, {"qubits": [19, 18], "gate": "cx", "parameters": [{"date": "2019-10-07T22:55:44+00:00", "unit": "", "value": 0.01285611913382742, "name": "gate_error"}, {"date": "2019-10-07T23:23:26+00:00", "unit": "ns", "value": 419.55555555555554, "name": "gate_length"}], "name": "cx19_18"}], "backend_version": "1.2.1", "backend_name": "ibmq_poughkeepsie", "last_update_date": "2019-10-07T23:23:26+00:00"}'
with open(props_poughkeepsie_target_filepath, 'w') as f:
  f.write(props_poughkeepsie_str)

# FakeRueschlikon does not need properties file and it is not in the github directory

props_singapore_target_filepath = base_dir + "props_singapore.json"
props_singapore_str = '{"general": [], "qubits": [[{"date": "2019-10-07T10:10:07+00:00", "unit": "\u00b5s", "value": 98.50001846063365, "name": "T1"}, {"date": "2019-10-07T10:11:53+00:00", "unit": "\u00b5s", "value": 142.79433528474766, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.676715786802928, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.028000000000000025, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.04, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.016000000000000014, "name": "prob_meas1_prep0"}], [{"date": "2019-10-05T10:14:29+00:00", "unit": "\u00b5s", "value": 71.8423910064075, "name": "T1"}, {"date": "2019-10-07T10:13:37+00:00", "unit": "\u00b5s", "value": 86.1000420353437, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.782680699300517, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.03500000000000003, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.05800000000000005, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.012, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T10:10:07+00:00", "unit": "\u00b5s", "value": 54.586493833620445, "name": "T1"}, {"date": "2019-10-07T10:11:53+00:00", "unit": "\u00b5s", "value": 64.28987220978689, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.685409810699721, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.02750000000000008, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.040000000000000036, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.015, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T10:10:07+00:00", "unit": "\u00b5s", "value": 71.88906145160064, "name": "T1"}, {"date": "2019-10-07T10:13:37+00:00", "unit": "\u00b5s", "value": 100.29926276217378, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.716928039422963, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.031000000000000028, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.049000000000000044, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.013, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T10:10:07+00:00", "unit": "\u00b5s", "value": 52.770110881257324, "name": "T1"}, {"date": "2019-10-07T10:11:53+00:00", "unit": "\u00b5s", "value": 61.064644709448174, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.564662441799778, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.024499999999999966, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.04400000000000004, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.005, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T10:10:07+00:00", "unit": "\u00b5s", "value": 106.63080350740401, "name": "T1"}, {"date": "2019-10-07T10:11:53+00:00", "unit": "\u00b5s", "value": 63.847221711183266, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.3093990890670115, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.05049999999999999, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.05700000000000005, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.044, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T10:10:07+00:00", "unit": "\u00b5s", "value": 53.25798980049998, "name": "T1"}, {"date": "2019-10-07T10:25:34+00:00", "unit": "\u00b5s", "value": 59.50768130704096, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.662502738898581, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.03500000000000003, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.05300000000000005, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.017, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T10:10:07+00:00", "unit": "\u00b5s", "value": 70.7178193327572, "name": "T1"}, {"date": "2019-09-30T09:12:16+00:00", "unit": "\u00b5s", "value": 91.5454015797137, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.83100170618901, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.034499999999999975, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.04400000000000004, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.025, "name": "prob_meas1_prep0"}], [{"date": "2019-10-05T10:14:29+00:00", "unit": "\u00b5s", "value": 81.4823552465602, "name": "T1"}, {"date": "2019-10-07T10:25:34+00:00", "unit": "\u00b5s", "value": 80.29259328068997, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.623118095677056, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.04200000000000004, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.05700000000000005, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.027, "name": "prob_meas1_prep0"}], [{"date": "2019-10-06T09:52:34+00:00", "unit": "\u00b5s", "value": 75.22358280239378, "name": "T1"}, {"date": "2019-10-04T10:03:13+00:00", "unit": "\u00b5s", "value": 33.6275779166682, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.697684600011119, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.040000000000000036, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.05600000000000005, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.024, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T10:10:07+00:00", "unit": "\u00b5s", "value": 70.98747296573573, "name": "T1"}, {"date": "2019-10-07T10:13:37+00:00", "unit": "\u00b5s", "value": 113.20586880438636, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.424172526087491, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.02849999999999997, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.04500000000000004, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.012, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T10:10:07+00:00", "unit": "\u00b5s", "value": 49.81041273082784, "name": "T1"}, {"date": "2019-10-07T10:11:53+00:00", "unit": "\u00b5s", "value": 72.19589834748233, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.521513114099939, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.03400000000000003, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.061, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.007000000000000006, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T10:10:07+00:00", "unit": "\u00b5s", "value": 82.5827071619044, "name": "T1"}, {"date": "2019-10-07T10:13:37+00:00", "unit": "\u00b5s", "value": 141.83075197141548, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.603498735627486, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.034499999999999975, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.05500000000000005, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.014, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T10:10:07+00:00", "unit": "\u00b5s", "value": 95.85019354756015, "name": "T1"}, {"date": "2019-10-07T10:11:53+00:00", "unit": "\u00b5s", "value": 131.6531371896787, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.624418341917924, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.027000000000000024, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.03500000000000003, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.019, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T10:10:07+00:00", "unit": "\u00b5s", "value": 35.88916991811218, "name": "T1"}, {"date": "2019-10-07T10:13:37+00:00", "unit": "\u00b5s", "value": 62.03490787220797, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.757476384494219, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.024499999999999966, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.03700000000000003, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.012, "name": "prob_meas1_prep0"}], [{"date": "2019-10-06T09:52:34+00:00", "unit": "\u00b5s", "value": 104.68198729947855, "name": "T1"}, {"date": "2019-10-07T10:11:53+00:00", "unit": "\u00b5s", "value": 57.97422234961757, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.488504164049106, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.033500000000000085, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.05600000000000005, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.011, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T10:10:07+00:00", "unit": "\u00b5s", "value": 92.36366680998361, "name": "T1"}, {"date": "2019-10-07T10:13:37+00:00", "unit": "\u00b5s", "value": 94.59861170877572, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.738439313844244, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.027000000000000024, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.039000000000000035, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.015, "name": "prob_meas1_prep0"}], [{"date": "2019-10-07T10:10:07+00:00", "unit": "\u00b5s", "value": 107.49950920916586, "name": "T1"}, {"date": "2019-10-07T10:11:53+00:00", "unit": "\u00b5s", "value": 133.11941489971574, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.8824966313847975, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.03049999999999997, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.04700000000000004, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.014, "name": "prob_meas1_prep0"}], [{"date": "2019-10-03T09:48:38+00:00", "unit": "\u00b5s", "value": 105.19080597037585, "name": "T1"}, {"date": "2019-10-07T10:13:37+00:00", "unit": "\u00b5s", "value": 111.7410815464684, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.5284500512834285, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.08450000000000002, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.101, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.06799999999999995, "name": "prob_meas1_prep0"}], [{"date": "2019-10-04T10:01:26+00:00", "unit": "\u00b5s", "value": 129.411447716364, "name": "T1"}, {"date": "2019-10-04T10:03:13+00:00", "unit": "\u00b5s", "value": 94.47076929782332, "name": "T2"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "GHz", "value": 4.6976528478044095, "name": "frequency"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.04200000000000004, "name": "readout_error"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.06499999999999995, "name": "prob_meas0_prep1"}, {"date": "2019-10-07T09:58:52+00:00", "unit": "", "value": 0.019, "name": "prob_meas1_prep0"}]], "gates": [{"qubits": [0], "gate": "id", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0005688851193218149, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_0"}, {"qubits": [0], "gate": "u1", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_0"}, {"qubits": [0], "gate": "u2", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0005688851193218149, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_0"}, {"qubits": [0], "gate": "u3", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0011377702386436298, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_0"}, {"qubits": [1], "gate": "id", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0005995105803450207, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_1"}, {"qubits": [1], "gate": "u1", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_1"}, {"qubits": [1], "gate": "u2", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0005995105803450207, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_1"}, {"qubits": [1], "gate": "u3", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0011990211606900413, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_1"}, {"qubits": [2], "gate": "id", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0022127694703092497, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_2"}, {"qubits": [2], "gate": "u1", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_2"}, {"qubits": [2], "gate": "u2", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0022127694703092497, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_2"}, {"qubits": [2], "gate": "u3", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0044255389406184995, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_2"}, {"qubits": [3], "gate": "id", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0009010448455703253, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_3"}, {"qubits": [3], "gate": "u1", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_3"}, {"qubits": [3], "gate": "u2", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0009010448455703253, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_3"}, {"qubits": [3], "gate": "u3", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0018020896911406506, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_3"}, {"qubits": [4], "gate": "id", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.00022510791737658382, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_4"}, {"qubits": [4], "gate": "u1", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_4"}, {"qubits": [4], "gate": "u2", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.00022510791737658382, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_4"}, {"qubits": [4], "gate": "u3", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.00045021583475316764, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_4"}, {"qubits": [5], "gate": "id", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.00037422191608514393, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_5"}, {"qubits": [5], "gate": "u1", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_5"}, {"qubits": [5], "gate": "u2", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.00037422191608514393, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_5"}, {"qubits": [5], "gate": "u3", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0007484438321702879, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_5"}, {"qubits": [6], "gate": "id", "parameters": [{"date": "2019-10-07T10:41:33+00:00", "unit": "", "value": 0.000378362284572259, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_6"}, {"qubits": [6], "gate": "u1", "parameters": [{"date": "2019-10-07T10:41:33+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_6"}, {"qubits": [6], "gate": "u2", "parameters": [{"date": "2019-10-07T10:41:33+00:00", "unit": "", "value": 0.000378362284572259, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_6"}, {"qubits": [6], "gate": "u3", "parameters": [{"date": "2019-10-07T10:41:33+00:00", "unit": "", "value": 0.000756724569144518, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_6"}, {"qubits": [7], "gate": "id", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0004125770256412488, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_7"}, {"qubits": [7], "gate": "u1", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_7"}, {"qubits": [7], "gate": "u2", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0004125770256412488, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_7"}, {"qubits": [7], "gate": "u3", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0008251540512824976, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_7"}, {"qubits": [8], "gate": "id", "parameters": [{"date": "2019-10-07T10:41:33+00:00", "unit": "", "value": 0.0005417535111698166, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_8"}, {"qubits": [8], "gate": "u1", "parameters": [{"date": "2019-10-07T10:41:33+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_8"}, {"qubits": [8], "gate": "u2", "parameters": [{"date": "2019-10-07T10:41:33+00:00", "unit": "", "value": 0.0005417535111698166, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_8"}, {"qubits": [8], "gate": "u3", "parameters": [{"date": "2019-10-07T10:41:33+00:00", "unit": "", "value": 0.0010835070223396332, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_8"}, {"qubits": [9], "gate": "id", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0003155501728177163, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_9"}, {"qubits": [9], "gate": "u1", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_9"}, {"qubits": [9], "gate": "u2", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0003155501728177163, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_9"}, {"qubits": [9], "gate": "u3", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0006311003456354326, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_9"}, {"qubits": [10], "gate": "id", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0005368264006968577, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_10"}, {"qubits": [10], "gate": "u1", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_10"}, {"qubits": [10], "gate": "u2", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0005368264006968577, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_10"}, {"qubits": [10], "gate": "u3", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0010736528013937153, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_10"}, {"qubits": [11], "gate": "id", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0004105472539004482, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_11"}, {"qubits": [11], "gate": "u1", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_11"}, {"qubits": [11], "gate": "u2", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0004105472539004482, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_11"}, {"qubits": [11], "gate": "u3", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0008210945078008964, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_11"}, {"qubits": [12], "gate": "id", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0027345208771739214, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_12"}, {"qubits": [12], "gate": "u1", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_12"}, {"qubits": [12], "gate": "u2", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0027345208771739214, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_12"}, {"qubits": [12], "gate": "u3", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.005469041754347843, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_12"}, {"qubits": [13], "gate": "id", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0034136411938485564, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_13"}, {"qubits": [13], "gate": "u1", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_13"}, {"qubits": [13], "gate": "u2", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0034136411938485564, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_13"}, {"qubits": [13], "gate": "u3", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.006827282387697113, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_13"}, {"qubits": [14], "gate": "id", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0003519344587204795, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_14"}, {"qubits": [14], "gate": "u1", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_14"}, {"qubits": [14], "gate": "u2", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0003519344587204795, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_14"}, {"qubits": [14], "gate": "u3", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.000703868917440959, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_14"}, {"qubits": [15], "gate": "id", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0016090282661921586, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_15"}, {"qubits": [15], "gate": "u1", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_15"}, {"qubits": [15], "gate": "u2", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0016090282661921586, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_15"}, {"qubits": [15], "gate": "u3", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.003218056532384317, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_15"}, {"qubits": [16], "gate": "id", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.00036655471720775684, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_16"}, {"qubits": [16], "gate": "u1", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_16"}, {"qubits": [16], "gate": "u2", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.00036655471720775684, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_16"}, {"qubits": [16], "gate": "u3", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0007331094344155137, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_16"}, {"qubits": [17], "gate": "id", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.00028697003074466967, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_17"}, {"qubits": [17], "gate": "u1", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_17"}, {"qubits": [17], "gate": "u2", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.00028697003074466967, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_17"}, {"qubits": [17], "gate": "u3", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0005739400614893393, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_17"}, {"qubits": [18], "gate": "id", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0003816864785326346, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_18"}, {"qubits": [18], "gate": "u1", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_18"}, {"qubits": [18], "gate": "u2", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0003816864785326346, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_18"}, {"qubits": [18], "gate": "u3", "parameters": [{"date": "2019-10-07T10:29:19+00:00", "unit": "", "value": 0.0007633729570652692, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_18"}, {"qubits": [19], "gate": "id", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0003179681265439094, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "id_19"}, {"qubits": [19], "gate": "u1", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "name": "u1_19"}, {"qubits": [19], "gate": "u2", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0003179681265439094, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 35.55555555555556, "name": "gate_length"}], "name": "u2_19"}, {"qubits": [19], "gate": "u3", "parameters": [{"date": "2019-10-07T10:27:19+00:00", "unit": "", "value": 0.0006359362530878187, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 71.11111111111111, "name": "gate_length"}], "name": "u3_19"}, {"qubits": [0, 1], "gate": "cx", "parameters": [{"date": "2019-10-07T10:56:40+00:00", "unit": "", "value": 0.016951800122571692, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 504.88888888888886, "name": "gate_length"}], "name": "cx0_1"}, {"qubits": [1, 0], "gate": "cx", "parameters": [{"date": "2019-10-07T10:56:40+00:00", "unit": "", "value": 0.016951800122571692, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 504.88888888888886, "name": "gate_length"}], "name": "cx1_0"}, {"qubits": [1, 2], "gate": "cx", "parameters": [{"date": "2019-10-07T11:13:55+00:00", "unit": "", "value": 0.03515653355748438, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 440.88888888888886, "name": "gate_length"}], "name": "cx1_2"}, {"qubits": [1, 6], "gate": "cx", "parameters": [{"date": "2019-10-07T11:20:02+00:00", "unit": "", "value": 0.015497896558171687, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 398.2222222222222, "name": "gate_length"}], "name": "cx1_6"}, {"qubits": [2, 1], "gate": "cx", "parameters": [{"date": "2019-10-07T11:13:55+00:00", "unit": "", "value": 0.03515653355748438, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 440.88888888888886, "name": "gate_length"}], "name": "cx2_1"}, {"qubits": [2, 3], "gate": "cx", "parameters": [{"date": "2019-10-07T11:37:38+00:00", "unit": "", "value": 0.0160990621644867, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 405.3333333333333, "name": "gate_length"}], "name": "cx2_3"}, {"qubits": [3, 2], "gate": "cx", "parameters": [{"date": "2019-10-07T11:37:38+00:00", "unit": "", "value": 0.0160990621644867, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 405.3333333333333, "name": "gate_length"}], "name": "cx3_2"}, {"qubits": [3, 4], "gate": "cx", "parameters": [{"date": "2019-10-07T11:55:00+00:00", "unit": "", "value": 0.011826337096007389, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 568.8888888888889, "name": "gate_length"}], "name": "cx3_4"}, {"qubits": [3, 8], "gate": "cx", "parameters": [{"date": "2019-10-07T12:12:23+00:00", "unit": "", "value": 0.02454490012996005, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 419.55555555555554, "name": "gate_length"}], "name": "cx3_8"}, {"qubits": [4, 3], "gate": "cx", "parameters": [{"date": "2019-10-07T11:55:00+00:00", "unit": "", "value": 0.011826337096007389, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 568.8888888888889, "name": "gate_length"}], "name": "cx4_3"}, {"qubits": [5, 6], "gate": "cx", "parameters": [{"date": "2019-10-07T12:29:49+00:00", "unit": "", "value": 0.02021153777322543, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 590.2222222222222, "name": "gate_length"}], "name": "cx5_6"}, {"qubits": [5, 10], "gate": "cx", "parameters": [{"date": "2019-10-07T14:13:49+00:00", "unit": "", "value": 0.015922830657213644, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 711.1111111111111, "name": "gate_length"}], "name": "cx5_10"}, {"qubits": [6, 1], "gate": "cx", "parameters": [{"date": "2019-10-07T11:20:02+00:00", "unit": "", "value": 0.015497896558171687, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 398.2222222222222, "name": "gate_length"}], "name": "cx6_1"}, {"qubits": [6, 5], "gate": "cx", "parameters": [{"date": "2019-10-07T12:29:49+00:00", "unit": "", "value": 0.02021153777322543, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 590.2222222222222, "name": "gate_length"}], "name": "cx6_5"}, {"qubits": [6, 7], "gate": "cx", "parameters": [{"date": "2019-10-07T12:47:04+00:00", "unit": "", "value": 0.013273348749692243, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 540.4444444444445, "name": "gate_length"}], "name": "cx6_7"}, {"qubits": [7, 6], "gate": "cx", "parameters": [{"date": "2019-10-07T12:47:04+00:00", "unit": "", "value": 0.013273348749692243, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 540.4444444444445, "name": "gate_length"}], "name": "cx7_6"}, {"qubits": [7, 8], "gate": "cx", "parameters": [{"date": "2019-10-07T13:04:16+00:00", "unit": "", "value": 0.016090620555644725, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 462.2222222222222, "name": "gate_length"}], "name": "cx7_8"}, {"qubits": [7, 12], "gate": "cx", "parameters": [{"date": "2019-10-07T13:21:38+00:00", "unit": "", "value": 0.011384053501933722, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 490.66666666666663, "name": "gate_length"}], "name": "cx7_12"}, {"qubits": [8, 3], "gate": "cx", "parameters": [{"date": "2019-10-07T12:12:23+00:00", "unit": "", "value": 0.02454490012996005, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 419.55555555555554, "name": "gate_length"}], "name": "cx8_3"}, {"qubits": [8, 7], "gate": "cx", "parameters": [{"date": "2019-10-07T13:04:16+00:00", "unit": "", "value": 0.016090620555644725, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 462.2222222222222, "name": "gate_length"}], "name": "cx8_7"}, {"qubits": [8, 9], "gate": "cx", "parameters": [{"date": "2019-10-07T13:39:02+00:00", "unit": "", "value": 0.014462795657984778, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 248.88888888888889, "name": "gate_length"}], "name": "cx8_9"}, {"qubits": [9, 8], "gate": "cx", "parameters": [{"date": "2019-10-07T13:39:02+00:00", "unit": "", "value": 0.014462795657984778, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 248.88888888888889, "name": "gate_length"}], "name": "cx9_8"}, {"qubits": [9, 14], "gate": "cx", "parameters": [{"date": "2019-10-07T13:56:21+00:00", "unit": "", "value": 0.010910395045313853, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 348.4444444444444, "name": "gate_length"}], "name": "cx9_14"}, {"qubits": [10, 5], "gate": "cx", "parameters": [{"date": "2019-10-07T14:13:49+00:00", "unit": "", "value": 0.015922830657213644, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 711.1111111111111, "name": "gate_length"}], "name": "cx10_5"}, {"qubits": [10, 11], "gate": "cx", "parameters": [{"date": "2019-10-07T14:35:42+00:00", "unit": "", "value": 0.010285529834766077, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 426.66666666666663, "name": "gate_length"}], "name": "cx10_11"}, {"qubits": [11, 10], "gate": "cx", "parameters": [{"date": "2019-10-07T14:35:42+00:00", "unit": "", "value": 0.010285529834766077, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 426.66666666666663, "name": "gate_length"}], "name": "cx11_10"}, {"qubits": [11, 12], "gate": "cx", "parameters": [{"date": "2019-10-07T14:57:14+00:00", "unit": "", "value": 0.02382591530175221, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 440.88888888888886, "name": "gate_length"}], "name": "cx11_12"}, {"qubits": [11, 16], "gate": "cx", "parameters": [{"date": "2019-10-07T16:09:43+00:00", "unit": "", "value": 0.01296503598444329, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 533.3333333333333, "name": "gate_length"}], "name": "cx11_16"}, {"qubits": [12, 7], "gate": "cx", "parameters": [{"date": "2019-10-07T13:21:38+00:00", "unit": "", "value": 0.011384053501933722, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 490.66666666666663, "name": "gate_length"}], "name": "cx12_7"}, {"qubits": [12, 11], "gate": "cx", "parameters": [{"date": "2019-10-07T14:57:14+00:00", "unit": "", "value": 0.02382591530175221, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 440.88888888888886, "name": "gate_length"}], "name": "cx12_11"}, {"qubits": [12, 13], "gate": "cx", "parameters": [{"date": "2019-10-07T15:14:44+00:00", "unit": "", "value": 0.0634606351390915, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 1045.3333333333333, "name": "gate_length"}], "name": "cx12_13"}, {"qubits": [13, 12], "gate": "cx", "parameters": [{"date": "2019-10-07T15:14:44+00:00", "unit": "", "value": 0.0634606351390915, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 1045.3333333333333, "name": "gate_length"}], "name": "cx13_12"}, {"qubits": [13, 14], "gate": "cx", "parameters": [{"date": "2019-10-07T15:50:34+00:00", "unit": "", "value": 0.02584357040051538, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 369.77777777777777, "name": "gate_length"}], "name": "cx13_14"}, {"qubits": [13, 18], "gate": "cx", "parameters": [{"date": "2019-10-07T15:33:08+00:00", "unit": "", "value": 0.02862492489048793, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 568.8888888888889, "name": "gate_length"}], "name": "cx13_18"}, {"qubits": [14, 9], "gate": "cx", "parameters": [{"date": "2019-10-07T13:56:21+00:00", "unit": "", "value": 0.010910395045313853, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 348.4444444444444, "name": "gate_length"}], "name": "cx14_9"}, {"qubits": [14, 13], "gate": "cx", "parameters": [{"date": "2019-10-07T15:50:34+00:00", "unit": "", "value": 0.02584357040051538, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 369.77777777777777, "name": "gate_length"}], "name": "cx14_13"}, {"qubits": [15, 16], "gate": "cx", "parameters": [{"date": "2019-10-07T16:32:14+00:00", "unit": "", "value": 0.013742969331295318, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 391.1111111111111, "name": "gate_length"}], "name": "cx15_16"}, {"qubits": [16, 11], "gate": "cx", "parameters": [{"date": "2019-10-07T16:09:43+00:00", "unit": "", "value": 0.01296503598444329, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 533.3333333333333, "name": "gate_length"}], "name": "cx16_11"}, {"qubits": [16, 15], "gate": "cx", "parameters": [{"date": "2019-10-07T16:32:14+00:00", "unit": "", "value": 0.013742969331295318, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 391.1111111111111, "name": "gate_length"}], "name": "cx16_15"}, {"qubits": [16, 17], "gate": "cx", "parameters": [{"date": "2019-10-07T16:49:53+00:00", "unit": "", "value": 0.012724327435765598, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 554.6666666666666, "name": "gate_length"}], "name": "cx16_17"}, {"qubits": [17, 16], "gate": "cx", "parameters": [{"date": "2019-10-07T16:49:53+00:00", "unit": "", "value": 0.012724327435765598, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 554.6666666666666, "name": "gate_length"}], "name": "cx17_16"}, {"qubits": [17, 18], "gate": "cx", "parameters": [{"date": "2019-10-07T17:07:15+00:00", "unit": "", "value": 0.0116296454858979, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 760.8888888888888, "name": "gate_length"}], "name": "cx17_18"}, {"qubits": [18, 13], "gate": "cx", "parameters": [{"date": "2019-10-07T15:33:08+00:00", "unit": "", "value": 0.02862492489048793, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 568.8888888888889, "name": "gate_length"}], "name": "cx18_13"}, {"qubits": [18, 17], "gate": "cx", "parameters": [{"date": "2019-10-07T17:07:15+00:00", "unit": "", "value": 0.0116296454858979, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 760.8888888888888, "name": "gate_length"}], "name": "cx18_17"}, {"qubits": [18, 19], "gate": "cx", "parameters": [{"date": "2019-10-07T17:24:07+00:00", "unit": "", "value": 0.009729516587356962, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 533.3333333333333, "name": "gate_length"}], "name": "cx18_19"}, {"qubits": [19, 18], "gate": "cx", "parameters": [{"date": "2019-10-07T17:24:07+00:00", "unit": "", "value": 0.009729516587356962, "name": "gate_error"}, {"date": "2019-10-07T17:24:07+00:00", "unit": "ns", "value": 533.3333333333333, "name": "gate_length"}], "name": "cx19_18"}], "backend_version": "1.0.10", "backend_name": "ibmq_singapore", "last_update_date": "2019-10-07T17:24:07+00:00"}'
with open(props_singapore_target_filepath, 'w') as f:
  f.write(props_singapore_str)

props_tenerife_target_filepath = base_dir + "props_tenerife.json"
props_tenerife_str = '{"backend_version": "1.0.0", "general": [], "last_update_date": "2019-08-23T09:50:32+00:00", "backend_name": "ibmqx4", "qubits": [[{"date": "2019-08-23T09:13:06+00:00", "unit": "\u00b5s", "value": 47.22966869058785, "name": "T1"}, {"date": "2019-08-23T09:13:51+00:00", "unit": "\u00b5s", "value": 14.618250240674037, "name": "T2"}, {"date": "2019-08-23T09:50:32+00:00", "unit": "GHz", "value": 5.246418075702811, "name": "frequency"}, {"date": "2019-08-23T09:12:48+00:00", "unit": "", "value": 0.07899999999999996, "name": "readout_error"}], [{"date": "2019-08-23T09:13:06+00:00", "unit": "\u00b5s", "value": 38.84873584352848, "name": "T1"}, {"date": "2019-08-23T09:14:33+00:00", "unit": "\u00b5s", "value": 8.99317856766105, "name": "T2"}, {"date": "2019-08-23T09:50:32+00:00", "unit": "GHz", "value": 5.298247616069848, "name": "frequency"}, {"date": "2019-08-23T09:12:48+00:00", "unit": "", "value": 0.09024999999999994, "name": "readout_error"}], [{"date": "2019-08-23T09:13:06+00:00", "unit": "\u00b5s", "value": 39.385341810017174, "name": "T1"}, {"date": "2019-08-23T09:15:15+00:00", "unit": "\u00b5s", "value": 42.748099150102384, "name": "T2"}, {"date": "2019-08-23T09:50:32+00:00", "unit": "GHz", "value": 5.338329702290311, "name": "frequency"}, {"date": "2019-08-23T09:12:48+00:00", "unit": "", "value": 0.20625000000000004, "name": "readout_error"}], [{"date": "2019-08-23T09:13:06+00:00", "unit": "\u00b5s", "value": 41.576153633262635, "name": "T1"}, {"date": "2019-08-23T09:14:33+00:00", "unit": "\u00b5s", "value": 22.72989208553766, "name": "T2"}, {"date": "2019-08-23T09:50:32+00:00", "unit": "GHz", "value": 5.42609984699337, "name": "frequency"}, {"date": "2019-08-23T09:12:48+00:00", "unit": "", "value": 0.34575, "name": "readout_error"}], [{"date": "2019-08-23T09:13:06+00:00", "unit": "\u00b5s", "value": 53.076017008114555, "name": "T1"}, {"date": "2019-08-23T09:13:51+00:00", "unit": "\u00b5s", "value": 5.681819536311601, "name": "T2"}, {"date": "2019-08-23T09:50:32+00:00", "unit": "GHz", "value": 5.1745665215145165, "name": "frequency"}, {"date": "2019-08-23T09:12:48+00:00", "unit": "", "value": 0.12050000000000005, "name": "readout_error"}]], "gates": [{"parameters": [{"date": "2019-08-23T09:53:36+00:00", "unit": "", "value": 0.0, "name": "gate_error"}], "qubits": [0], "gate": "u1"}, {"parameters": [{"date": "2019-08-23T09:53:36+00:00", "unit": "", "value": 0.0007726307293453583, "name": "gate_error"}], "qubits": [0], "gate": "u2"}, {"parameters": [{"date": "2019-08-23T09:53:36+00:00", "unit": "", "value": 0.0015452614586907165, "name": "gate_error"}], "qubits": [0], "gate": "u3"}, {"parameters": [{"date": "2019-08-23T09:53:36+00:00", "unit": "", "value": 0.0, "name": "gate_error"}], "qubits": [1], "gate": "u1"}, {"parameters": [{"date": "2019-08-23T09:53:36+00:00", "unit": "", "value": 0.0025762039467128184, "name": "gate_error"}], "qubits": [1], "gate": "u2"}, {"parameters": [{"date": "2019-08-23T09:53:36+00:00", "unit": "", "value": 0.005152407893425637, "name": "gate_error"}], "qubits": [1], "gate": "u3"}, {"parameters": [{"date": "2019-08-23T09:53:36+00:00", "unit": "", "value": 0.0, "name": "gate_error"}], "qubits": [2], "gate": "u1"}, {"parameters": [{"date": "2019-08-23T09:53:36+00:00", "unit": "", "value": 0.0012019552727863259, "name": "gate_error"}], "qubits": [2], "gate": "u2"}, {"parameters": [{"date": "2019-08-23T09:53:36+00:00", "unit": "", "value": 0.0024039105455726517, "name": "gate_error"}], "qubits": [2], "gate": "u3"}, {"parameters": [{"date": "2019-08-23T09:53:36+00:00", "unit": "", "value": 0.0, "name": "gate_error"}], "qubits": [3], "gate": "u1"}, {"parameters": [{"date": "2019-08-23T09:53:36+00:00", "unit": "", "value": 0.0013737021608475342, "name": "gate_error"}], "qubits": [3], "gate": "u2"}, {"parameters": [{"date": "2019-08-23T09:53:36+00:00", "unit": "", "value": 0.0027474043216950683, "name": "gate_error"}], "qubits": [3], "gate": "u3"}, {"parameters": [{"date": "2019-08-23T09:53:36+00:00", "unit": "", "value": 0.0, "name": "gate_error"}], "qubits": [4], "gate": "u1"}, {"parameters": [{"date": "2019-08-23T09:53:36+00:00", "unit": "", "value": 0.0014595792651160289, "name": "gate_error"}], "qubits": [4], "gate": "u2"}, {"parameters": [{"date": "2019-08-23T09:53:36+00:00", "unit": "", "value": 0.0029191585302320577, "name": "gate_error"}], "qubits": [4], "gate": "u3"}, {"parameters": [{"date": "2019-08-23T09:22:55+00:00", "unit": "", "value": 0.024825251753263056, "name": "gate_error"}], "qubits": [1, 0], "gate": "cx", "name": "CX1_0"}, {"parameters": [{"date": "2019-08-23T09:28:27+00:00", "unit": "", "value": 0.023835280244129398, "name": "gate_error"}], "qubits": [2, 0], "gate": "cx", "name": "CX2_0"}, {"parameters": [{"date": "2019-08-23T09:33:40+00:00", "unit": "", "value": 0.04972816061235677, "name": "gate_error"}], "qubits": [2, 1], "gate": "cx", "name": "CX2_1"}, {"parameters": [{"date": "2019-08-23T09:39:16+00:00", "unit": "", "value": 0.05219640255678959, "name": "gate_error"}], "qubits": [3, 2], "gate": "cx", "name": "CX3_2"}, {"parameters": [{"date": "2019-08-23T09:44:49+00:00", "unit": "", "value": 0.03898766681442359, "name": "gate_error"}], "qubits": [3, 4], "gate": "cx", "name": "CX3_4"}, {"parameters": [{"date": "2019-08-23T09:50:32+00:00", "unit": "", "value": 0.0528380205562545, "name": "gate_error"}], "qubits": [4, 2], "gate": "cx", "name": "CX4_2"}]}'
with open(props_tenerife_target_filepath, 'w') as f:
  f.write(props_tenerife_str)

props_tokyo_target_filepath = base_dir + "props_tokyo.json"
props_tokyo_str = '{"backend_version": "1.3.1", "general": [], "last_update_date": "2019-08-29T03:57:44+00:00", "backend_name": "ibmq_20_tokyo", "qubits": [[{"date": "2019-08-29T00:34:19+00:00", "unit": "\u00b5s", "value": 123.01755891092975, "name": "T1"}, {"date": "2019-08-29T00:35:40+00:00", "unit": "\u00b5s", "value": 82.60817509366677, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 5.020067674045979, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.06000000000000005, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.062000000000000055, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.058, "name": "prob_meas1_prep0"}], [{"date": "2019-08-28T02:41:06+00:00", "unit": "\u00b5s", "value": 69.54336965861351, "name": "T1"}, {"date": "2019-08-29T00:36:57+00:00", "unit": "\u00b5s", "value": 10.66094335224705, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 4.903314872531791, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.04400000000000004, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.08, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.008000000000000007, "name": "prob_meas1_prep0"}], [{"date": "2019-08-28T02:41:06+00:00", "unit": "\u00b5s", "value": 92.53354847628202, "name": "T1"}, {"date": "2019-08-02T22:14:22+00:00", "unit": "\u00b5s", "value": 29.117009285621773, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 4.777857954678067, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.11199999999999999, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.128, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.096, "name": "prob_meas1_prep0"}], [{"date": "2019-08-29T00:34:19+00:00", "unit": "\u00b5s", "value": 86.16109995755677, "name": "T1"}, {"date": "2019-08-29T00:36:57+00:00", "unit": "\u00b5s", "value": 56.407135347540546, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 5.094295974247867, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.04400000000000004, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.06000000000000005, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.028, "name": "prob_meas1_prep0"}], [{"date": "2019-08-29T00:34:19+00:00", "unit": "\u00b5s", "value": 107.23440278326854, "name": "T1"}, {"date": "2019-08-09T21:15:15+00:00", "unit": "\u00b5s", "value": 21.563344493292664, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 5.106775652097574, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.03500000000000003, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.062000000000000055, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.008, "name": "prob_meas1_prep0"}], [{"date": "2019-08-29T00:34:19+00:00", "unit": "\u00b5s", "value": 70.37233200765444, "name": "T1"}, {"date": "2019-08-29T00:36:57+00:00", "unit": "\u00b5s", "value": 55.37965778313132, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 5.1432691535874655, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.04300000000000004, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.052000000000000046, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.034, "name": "prob_meas1_prep0"}], [{"date": "2019-08-29T00:34:19+00:00", "unit": "\u00b5s", "value": 77.12694992773726, "name": "T1"}, {"date": "2019-08-29T00:38:16+00:00", "unit": "\u00b5s", "value": 62.78899664154189, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 5.033487835883658, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.031000000000000028, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.052000000000000046, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.01, "name": "prob_meas1_prep0"}], [{"date": "2019-08-29T00:34:19+00:00", "unit": "\u00b5s", "value": 115.35301088836374, "name": "T1"}, {"date": "2019-08-29T00:39:31+00:00", "unit": "\u00b5s", "value": 61.19512147948497, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 4.457372203597565, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.18900000000000006, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.29400000000000004, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.084, "name": "prob_meas1_prep0"}], [{"date": "2019-08-29T00:34:19+00:00", "unit": "\u00b5s", "value": 118.99757262592244, "name": "T1"}, {"date": "2019-08-29T00:38:16+00:00", "unit": "\u00b5s", "value": 79.91583970211185, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 5.018955025336272, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.025000000000000022, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.03400000000000003, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.016, "name": "prob_meas1_prep0"}], [{"date": "2019-08-28T02:41:06+00:00", "unit": "\u00b5s", "value": 52.51567460764532, "name": "T1"}, {"date": "2019-08-29T00:39:31+00:00", "unit": "\u00b5s", "value": 23.815219798533672, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 5.065231576267004, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.04899999999999993, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.06999999999999995, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.028, "name": "prob_meas1_prep0"}], [{"date": "2019-08-29T00:34:19+00:00", "unit": "\u00b5s", "value": 112.25671261600131, "name": "T1"}, {"date": "2019-08-29T00:35:40+00:00", "unit": "\u00b5s", "value": 63.709028734246615, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 4.959517920843555, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.10200000000000009, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.17800000000000005, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.026, "name": "prob_meas1_prep0"}], [{"date": "2019-08-29T00:34:19+00:00", "unit": "\u00b5s", "value": 73.77755675124585, "name": "T1"}, {"date": "2019-08-29T00:39:31+00:00", "unit": "\u00b5s", "value": 62.210126772510556, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 5.2296673837182395, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.03600000000000003, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.040000000000000036, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.032, "name": "prob_meas1_prep0"}], [{"date": "2019-08-28T02:41:06+00:00", "unit": "\u00b5s", "value": 122.45582126199709, "name": "T1"}, {"date": "2019-08-29T00:35:40+00:00", "unit": "\u00b5s", "value": 19.074326721834968, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 4.663567955647382, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.052000000000000046, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.07999999999999996, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.024, "name": "prob_meas1_prep0"}], [{"date": "2019-08-28T02:41:06+00:00", "unit": "\u00b5s", "value": 102.9893797615937, "name": "T1"}, {"date": "2019-08-29T00:36:57+00:00", "unit": "\u00b5s", "value": 49.48400849332707, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 4.898196290912248, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.04300000000000004, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.038000000000000034, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.048, "name": "prob_meas1_prep0"}], [{"date": "2019-08-29T00:34:19+00:00", "unit": "\u00b5s", "value": 61.165980228952655, "name": "T1"}, {"date": "2019-08-29T00:35:40+00:00", "unit": "\u00b5s", "value": 68.49907774155106, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 5.233166125811304, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.030000000000000027, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.05400000000000005, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.006, "name": "prob_meas1_prep0"}], [{"date": "2019-08-29T00:34:19+00:00", "unit": "\u00b5s", "value": 94.26280075655507, "name": "T1"}, {"date": "2019-08-29T00:36:57+00:00", "unit": "\u00b5s", "value": 82.00923703532516, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 4.491105815614907, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.04500000000000004, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.05600000000000005, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.034, "name": "prob_meas1_prep0"}], [{"date": "2019-08-29T00:34:19+00:00", "unit": "\u00b5s", "value": 59.39389912718584, "name": "T1"}, {"date": "2019-08-29T00:38:16+00:00", "unit": "\u00b5s", "value": 51.33196201268886, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 5.074508083065581, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.10099999999999998, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.16000000000000003, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.042, "name": "prob_meas1_prep0"}], [{"date": "2019-08-29T00:34:19+00:00", "unit": "\u00b5s", "value": 75.04495633010156, "name": "T1"}, {"date": "2019-08-29T00:36:57+00:00", "unit": "\u00b5s", "value": 58.766295263810285, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 4.984811943532805, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.06099999999999994, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.08799999999999997, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.034, "name": "prob_meas1_prep0"}], [{"date": "2019-08-29T00:34:19+00:00", "unit": "\u00b5s", "value": 76.1765286007167, "name": "T1"}, {"date": "2019-08-29T00:38:16+00:00", "unit": "\u00b5s", "value": 48.1076231790977, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 5.109337618382666, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.04500000000000004, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.04800000000000004, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.042, "name": "prob_meas1_prep0"}], [{"date": "2019-08-29T00:34:19+00:00", "unit": "\u00b5s", "value": 62.098614773402744, "name": "T1"}, {"date": "2019-08-29T00:39:31+00:00", "unit": "\u00b5s", "value": 53.070036968130985, "name": "T2"}, {"date": "2019-08-29T03:57:44+00:00", "unit": "GHz", "value": 5.124501966708196, "name": "frequency"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.04300000000000004, "name": "readout_error"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.062000000000000055, "name": "prob_meas0_prep1"}, {"date": "2019-08-29T00:33:18+00:00", "unit": "", "value": 0.024, "name": "prob_meas1_prep0"}]], "gates": [{"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.001409218006998817, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.975629+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [0], "gate": "id", "name": "id_0"}, {"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.975665+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [0], "gate": "u1", "name": "u1_0"}, {"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.001409218006998817, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.975696+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [0], "gate": "u2", "name": "u2_0"}, {"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.002818436013997634, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.975728+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [0], "gate": "u3", "name": "u3_0"}, {"parameters": [{"date": "2019-08-29T00:46:47+00:00", "unit": "", "value": 0.004176579235865799, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.975792+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [1], "gate": "id", "name": "id_1"}, {"parameters": [{"date": "2019-08-29T00:46:47+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.975823+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [1], "gate": "u1", "name": "u1_1"}, {"parameters": [{"date": "2019-08-29T00:46:47+00:00", "unit": "", "value": 0.004176579235865799, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.975852+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [1], "gate": "u2", "name": "u2_1"}, {"parameters": [{"date": "2019-08-29T00:46:47+00:00", "unit": "", "value": 0.008353158471731598, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.975883+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [1], "gate": "u3", "name": "u3_1"}, {"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.005250250248741274, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.975946+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [2], "gate": "id", "name": "id_2"}, {"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.975974+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [2], "gate": "u1", "name": "u1_2"}, {"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.005250250248741274, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976003+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [2], "gate": "u2", "name": "u2_2"}, {"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.010500500497482548, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976032+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [2], "gate": "u3", "name": "u3_2"}, {"parameters": [{"date": "2019-08-29T00:46:47+00:00", "unit": "", "value": 0.004257282555626724, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976095+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [3], "gate": "id", "name": "id_3"}, {"parameters": [{"date": "2019-08-29T00:46:47+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976123+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [3], "gate": "u1", "name": "u1_3"}, {"parameters": [{"date": "2019-08-29T00:46:47+00:00", "unit": "", "value": 0.004257282555626724, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976151+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [3], "gate": "u2", "name": "u2_3"}, {"parameters": [{"date": "2019-08-29T00:46:47+00:00", "unit": "", "value": 0.008514565111253447, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976180+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [3], "gate": "u3", "name": "u3_3"}, {"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.0033246130014941587, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976242+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [4], "gate": "id", "name": "id_4"}, {"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976270+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [4], "gate": "u1", "name": "u1_4"}, {"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.0033246130014941587, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976299+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [4], "gate": "u2", "name": "u2_4"}, {"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.006649226002988317, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976328+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [4], "gate": "u3", "name": "u3_4"}, {"parameters": [{"date": "2019-08-29T00:46:47+00:00", "unit": "", "value": 0.0025614406086142474, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976390+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [5], "gate": "id", "name": "id_5"}, {"parameters": [{"date": "2019-08-29T00:46:47+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976419+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [5], "gate": "u1", "name": "u1_5"}, {"parameters": [{"date": "2019-08-29T00:46:47+00:00", "unit": "", "value": 0.0025614406086142474, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976447+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [5], "gate": "u2", "name": "u2_5"}, {"parameters": [{"date": "2019-08-29T00:46:47+00:00", "unit": "", "value": 0.005122881217228495, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976478+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [5], "gate": "u3", "name": "u3_5"}, {"parameters": [{"date": "2019-08-29T00:48:14+00:00", "unit": "", "value": 0.0008563008248754674, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976545+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [6], "gate": "id", "name": "id_6"}, {"parameters": [{"date": "2019-08-29T00:48:14+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976574+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [6], "gate": "u1", "name": "u1_6"}, {"parameters": [{"date": "2019-08-29T00:48:14+00:00", "unit": "", "value": 0.0008563008248754674, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976605+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [6], "gate": "u2", "name": "u2_6"}, {"parameters": [{"date": "2019-08-29T00:48:14+00:00", "unit": "", "value": 0.0017126016497509347, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976637+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [6], "gate": "u3", "name": "u3_6"}, {"parameters": [{"date": "2019-08-25T21:26:01+00:00", "unit": "", "value": 0.0015855234247992111, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976699+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [7], "gate": "id", "name": "id_7"}, {"parameters": [{"date": "2019-08-25T21:26:01+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976729+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [7], "gate": "u1", "name": "u1_7"}, {"parameters": [{"date": "2019-08-25T21:26:01+00:00", "unit": "", "value": 0.0015855234247992111, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976758+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [7], "gate": "u2", "name": "u2_7"}, {"parameters": [{"date": "2019-08-25T21:26:01+00:00", "unit": "", "value": 0.0031710468495984223, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976789+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [7], "gate": "u3", "name": "u3_7"}, {"parameters": [{"date": "2019-08-29T00:48:14+00:00", "unit": "", "value": 0.0009646205708277285, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976854+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [8], "gate": "id", "name": "id_8"}, {"parameters": [{"date": "2019-08-29T00:48:14+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976884+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [8], "gate": "u1", "name": "u1_8"}, {"parameters": [{"date": "2019-08-29T00:48:14+00:00", "unit": "", "value": 0.0009646205708277285, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976914+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [8], "gate": "u2", "name": "u2_8"}, {"parameters": [{"date": "2019-08-29T00:48:14+00:00", "unit": "", "value": 0.001929241141655457, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.976947+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [8], "gate": "u3", "name": "u3_8"}, {"parameters": [{"date": "2019-08-29T00:49:40+00:00", "unit": "", "value": 0.002891761068235738, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977009+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [9], "gate": "id", "name": "id_9"}, {"parameters": [{"date": "2019-08-29T00:49:40+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977038+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [9], "gate": "u1", "name": "u1_9"}, {"parameters": [{"date": "2019-08-29T00:49:40+00:00", "unit": "", "value": 0.002891761068235738, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977065+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [9], "gate": "u2", "name": "u2_9"}, {"parameters": [{"date": "2019-08-29T00:49:40+00:00", "unit": "", "value": 0.005783522136471476, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977095+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [9], "gate": "u3", "name": "u3_9"}, {"parameters": [{"date": "2019-08-29T00:40:49+00:00", "unit": "", "value": 0.0012541507786638245, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977156+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [10], "gate": "id", "name": "id_10"}, {"parameters": [{"date": "2019-08-29T00:40:49+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977186+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [10], "gate": "u1", "name": "u1_10"}, {"parameters": [{"date": "2019-08-29T00:40:49+00:00", "unit": "", "value": 0.0012541507786638245, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977215+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [10], "gate": "u2", "name": "u2_10"}, {"parameters": [{"date": "2019-08-29T00:40:49+00:00", "unit": "", "value": 0.002508301557327649, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977246+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [10], "gate": "u3", "name": "u3_10"}, {"parameters": [{"date": "2019-08-29T00:40:49+00:00", "unit": "", "value": 0.001237724705714148, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977307+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [11], "gate": "id", "name": "id_11"}, {"parameters": [{"date": "2019-08-29T00:40:49+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977337+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [11], "gate": "u1", "name": "u1_11"}, {"parameters": [{"date": "2019-08-29T00:40:49+00:00", "unit": "", "value": 0.001237724705714148, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977367+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [11], "gate": "u2", "name": "u2_11"}, {"parameters": [{"date": "2019-08-29T00:40:49+00:00", "unit": "", "value": 0.002475449411428296, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977399+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [11], "gate": "u3", "name": "u3_11"}, {"parameters": [{"date": "2019-08-29T00:42:27+00:00", "unit": "", "value": 0.0018054460538394104, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977461+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [12], "gate": "id", "name": "id_12"}, {"parameters": [{"date": "2019-08-29T00:42:27+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977491+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [12], "gate": "u1", "name": "u1_12"}, {"parameters": [{"date": "2019-08-29T00:42:27+00:00", "unit": "", "value": 0.0018054460538394104, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977524+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [12], "gate": "u2", "name": "u2_12"}, {"parameters": [{"date": "2019-08-29T00:42:27+00:00", "unit": "", "value": 0.003610892107678821, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977557+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [12], "gate": "u3", "name": "u3_12"}, {"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.00733054141095281, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977619+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [13], "gate": "id", "name": "id_13"}, {"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977648+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [13], "gate": "u1", "name": "u1_13"}, {"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.00733054141095281, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977678+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [13], "gate": "u2", "name": "u2_13"}, {"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.01466108282190562, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977710+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [13], "gate": "u3", "name": "u3_13"}, {"parameters": [{"date": "2019-08-29T00:46:47+00:00", "unit": "", "value": 0.0008270891973384051, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977772+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [14], "gate": "id", "name": "id_14"}, {"parameters": [{"date": "2019-08-29T00:46:47+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977801+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [14], "gate": "u1", "name": "u1_14"}, {"parameters": [{"date": "2019-08-29T00:46:47+00:00", "unit": "", "value": 0.0008270891973384051, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977830+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [14], "gate": "u2", "name": "u2_14"}, {"parameters": [{"date": "2019-08-29T00:46:47+00:00", "unit": "", "value": 0.0016541783946768102, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977861+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [14], "gate": "u3", "name": "u3_14"}, {"parameters": [{"date": "2019-08-29T00:40:49+00:00", "unit": "", "value": 0.0009003599769165373, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977923+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [15], "gate": "id", "name": "id_15"}, {"parameters": [{"date": "2019-08-29T00:40:49+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977952+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [15], "gate": "u1", "name": "u1_15"}, {"parameters": [{"date": "2019-08-29T00:40:49+00:00", "unit": "", "value": 0.0009003599769165373, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.977981+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [15], "gate": "u2", "name": "u2_15"}, {"parameters": [{"date": "2019-08-29T00:40:49+00:00", "unit": "", "value": 0.0018007199538330747, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978011+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [15], "gate": "u3", "name": "u3_15"}, {"parameters": [{"date": "2019-08-29T00:40:49+00:00", "unit": "", "value": 0.0011954870592093985, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978073+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [16], "gate": "id", "name": "id_16"}, {"parameters": [{"date": "2019-08-29T00:40:49+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978101+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [16], "gate": "u1", "name": "u1_16"}, {"parameters": [{"date": "2019-08-29T00:40:49+00:00", "unit": "", "value": 0.0011954870592093985, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978130+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [16], "gate": "u2", "name": "u2_16"}, {"parameters": [{"date": "2019-08-29T00:40:49+00:00", "unit": "", "value": 0.002390974118418797, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978161+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [16], "gate": "u3", "name": "u3_16"}, {"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.001004709877292779, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978223+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [17], "gate": "id", "name": "id_17"}, {"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978252+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [17], "gate": "u1", "name": "u1_17"}, {"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.001004709877292779, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978280+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [17], "gate": "u2", "name": "u2_17"}, {"parameters": [{"date": "2019-07-05T20:44:41+00:00", "unit": "", "value": 0.002009419754585558, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978312+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [17], "gate": "u3", "name": "u3_17"}, {"parameters": [{"date": "2019-08-29T00:48:14+00:00", "unit": "", "value": 0.003991742762731593, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978375+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [18], "gate": "id", "name": "id_18"}, {"parameters": [{"date": "2019-08-29T00:48:14+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978405+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [18], "gate": "u1", "name": "u1_18"}, {"parameters": [{"date": "2019-08-29T00:48:14+00:00", "unit": "", "value": 0.003991742762731593, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978434+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [18], "gate": "u2", "name": "u2_18"}, {"parameters": [{"date": "2019-08-29T00:48:14+00:00", "unit": "", "value": 0.007983485525463185, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978465+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [18], "gate": "u3", "name": "u3_18"}, {"parameters": [{"date": "2019-08-29T00:49:40+00:00", "unit": "", "value": 0.0024214733920636367, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978532+00:00", "unit": "ns", "value": 99.55555555555554, "name": "gate_length"}], "qubits": [19], "gate": "id", "name": "id_19"}, {"parameters": [{"date": "2019-08-29T00:49:40+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978561+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [19], "gate": "u1", "name": "u1_19"}, {"parameters": [{"date": "2019-08-29T00:49:40+00:00", "unit": "", "value": 0.0024214733920636367, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978590+00:00", "unit": "ns", "value": 103.1111111111111, "name": "gate_length"}], "qubits": [19], "gate": "u2", "name": "u2_19"}, {"parameters": [{"date": "2019-08-29T00:49:40+00:00", "unit": "", "value": 0.004842946784127273, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978619+00:00", "unit": "ns", "value": 206.2222222222222, "name": "gate_length"}], "qubits": [19], "gate": "u3", "name": "u3_19"}, {"parameters": [{"date": "2019-08-29T00:53:44+00:00", "unit": "", "value": 0.03521401370473984, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978916+00:00", "unit": "ns", "value": 1393.7777777777776, "name": "gate_length"}], "qubits": [0, 1], "gate": "cx", "name": "cx0_1"}, {"parameters": [{"date": "2019-08-29T01:00:30+00:00", "unit": "", "value": 0.04987411721502363, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978995+00:00", "unit": "ns", "value": 568.8888888888889, "name": "gate_length"}], "qubits": [0, 5], "gate": "cx", "name": "cx0_5"}, {"parameters": [{"date": "2019-08-29T00:53:44+00:00", "unit": "", "value": 0.03521401370473984, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978916+00:00", "unit": "ns", "value": 1393.7777777777776, "name": "gate_length"}], "qubits": [1, 0], "gate": "cx", "name": "cx1_0"}, {"parameters": [{"date": "2019-08-29T01:05:38+00:00", "unit": "", "value": 0.037493190769214924, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980416+00:00", "unit": "ns", "value": 689.7777777777777, "name": "gate_length"}], "qubits": [1, 2], "gate": "cx", "name": "cx1_2"}, {"parameters": [{"date": "2019-08-29T01:21:15+00:00", "unit": "", "value": 0.03427838652765164, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980640+00:00", "unit": "ns", "value": 1393.7777777777776, "name": "gate_length"}], "qubits": [1, 6], "gate": "cx", "name": "cx1_6"}, {"parameters": [{"date": "2019-08-29T01:42:48+00:00", "unit": "", "value": 0.06870677071255399, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980929+00:00", "unit": "ns", "value": 1393.7777777777776, "name": "gate_length"}], "qubits": [1, 7], "gate": "cx", "name": "cx1_7"}, {"parameters": [{"date": "2019-08-29T01:05:38+00:00", "unit": "", "value": 0.037493190769214924, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980416+00:00", "unit": "ns", "value": 689.7777777777777, "name": "gate_length"}], "qubits": [2, 1], "gate": "cx", "name": "cx2_1"}, {"parameters": [{"date": "2019-08-29T01:26:26+00:00", "unit": "", "value": 0.04025723379314397, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980711+00:00", "unit": "ns", "value": 583.1111111111111, "name": "gate_length"}], "qubits": [2, 6], "gate": "cx", "name": "cx2_6"}, {"parameters": [{"date": "2019-08-29T01:54:09+00:00", "unit": "", "value": 0.03999622207736486, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.981218+00:00", "unit": "ns", "value": 533.3333333333333, "name": "gate_length"}], "qubits": [3, 8], "gate": "cx", "name": "cx3_8"}, {"parameters": [{"date": "2019-08-29T01:59:20+00:00", "unit": "", "value": 0.027723825875041685, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.981290+00:00", "unit": "ns", "value": 832.0, "name": "gate_length"}], "qubits": [4, 8], "gate": "cx", "name": "cx4_8"}, {"parameters": [{"date": "2019-08-29T02:15:39+00:00", "unit": "", "value": 0.03384110535480819, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.981361+00:00", "unit": "ns", "value": 832.0, "name": "gate_length"}], "qubits": [4, 9], "gate": "cx", "name": "cx4_9"}, {"parameters": [{"date": "2019-08-29T01:00:30+00:00", "unit": "", "value": 0.04987411721502363, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.978995+00:00", "unit": "ns", "value": 568.8888888888889, "name": "gate_length"}], "qubits": [5, 0], "gate": "cx", "name": "cx5_0"}, {"parameters": [{"date": "2019-08-29T01:31:36+00:00", "unit": "", "value": 0.02293471169776723, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980783+00:00", "unit": "ns", "value": 568.8888888888889, "name": "gate_length"}], "qubits": [5, 6], "gate": "cx", "name": "cx5_6"}, {"parameters": [{"date": "2019-08-29T01:10:55+00:00", "unit": "", "value": 0.026427550277632827, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980488+00:00", "unit": "ns", "value": 1144.888888888889, "name": "gate_length"}], "qubits": [5, 10], "gate": "cx", "name": "cx5_10"}, {"parameters": [{"date": "2019-08-29T01:16:15+00:00", "unit": "", "value": 0.03645862801896202, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980566+00:00", "unit": "ns", "value": 533.3333333333333, "name": "gate_length"}], "qubits": [5, 11], "gate": "cx", "name": "cx5_11"}, {"parameters": [{"date": "2019-08-29T01:21:15+00:00", "unit": "", "value": 0.03427838652765164, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980640+00:00", "unit": "ns", "value": 1393.7777777777776, "name": "gate_length"}], "qubits": [6, 1], "gate": "cx", "name": "cx6_1"}, {"parameters": [{"date": "2019-08-29T01:26:26+00:00", "unit": "", "value": 0.04025723379314397, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980711+00:00", "unit": "ns", "value": 583.1111111111111, "name": "gate_length"}], "qubits": [6, 2], "gate": "cx", "name": "cx6_2"}, {"parameters": [{"date": "2019-08-29T01:31:36+00:00", "unit": "", "value": 0.02293471169776723, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980783+00:00", "unit": "ns", "value": 568.8888888888889, "name": "gate_length"}], "qubits": [6, 5], "gate": "cx", "name": "cx6_5"}, {"parameters": [{"date": "2019-08-29T01:36:47+00:00", "unit": "", "value": 0.03754135340392492, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980856+00:00", "unit": "ns", "value": 2375.111111111111, "name": "gate_length"}], "qubits": [6, 7], "gate": "cx", "name": "cx6_7"}, {"parameters": [{"date": "2019-08-29T02:26:07+00:00", "unit": "", "value": 0.018448907554389993, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979147+00:00", "unit": "ns", "value": 540.4444444444445, "name": "gate_length"}], "qubits": [6, 10], "gate": "cx", "name": "cx6_10"}, {"parameters": [{"date": "2019-08-29T02:36:49+00:00", "unit": "", "value": 0.016510173006301032, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979296+00:00", "unit": "ns", "value": 490.66666666666663, "name": "gate_length"}], "qubits": [6, 11], "gate": "cx", "name": "cx6_11"}, {"parameters": [{"date": "2019-08-29T01:42:48+00:00", "unit": "", "value": 0.06870677071255399, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980929+00:00", "unit": "ns", "value": 1393.7777777777776, "name": "gate_length"}], "qubits": [7, 1], "gate": "cx", "name": "cx7_1"}, {"parameters": [{"date": "2019-08-29T01:36:47+00:00", "unit": "", "value": 0.03754135340392492, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980856+00:00", "unit": "ns", "value": 2375.111111111111, "name": "gate_length"}], "qubits": [7, 6], "gate": "cx", "name": "cx7_6"}, {"parameters": [{"date": "2019-08-29T01:48:21+00:00", "unit": "", "value": 0.03690677071410642, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.981002+00:00", "unit": "ns", "value": 1848.8888888888887, "name": "gate_length"}], "qubits": [7, 8], "gate": "cx", "name": "cx7_8"}, {"parameters": [{"date": "2019-08-29T02:47:06+00:00", "unit": "", "value": 0.021131714462582912, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979602+00:00", "unit": "ns", "value": 576.0, "name": "gate_length"}], "qubits": [7, 12], "gate": "cx", "name": "cx7_12"}, {"parameters": [{"date": "2019-08-29T01:54:09+00:00", "unit": "", "value": 0.03999622207736486, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.981218+00:00", "unit": "ns", "value": 533.3333333333333, "name": "gate_length"}], "qubits": [8, 3], "gate": "cx", "name": "cx8_3"}, {"parameters": [{"date": "2019-08-29T01:59:20+00:00", "unit": "", "value": 0.027723825875041685, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.981290+00:00", "unit": "ns", "value": 832.0, "name": "gate_length"}], "qubits": [8, 4], "gate": "cx", "name": "cx8_4"}, {"parameters": [{"date": "2019-08-29T01:48:21+00:00", "unit": "", "value": 0.03690677071410642, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.981002+00:00", "unit": "ns", "value": 1848.8888888888887, "name": "gate_length"}], "qubits": [8, 7], "gate": "cx", "name": "cx8_7"}, {"parameters": [{"date": "2019-08-29T02:20:54+00:00", "unit": "", "value": 0.022334050057862515, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.981433+00:00", "unit": "ns", "value": 618.6666666666666, "name": "gate_length"}], "qubits": [8, 9], "gate": "cx", "name": "cx8_9"}, {"parameters": [{"date": "2019-08-29T02:04:47+00:00", "unit": "", "value": 0.050011031518455235, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.981074+00:00", "unit": "ns", "value": 960.0, "name": "gate_length"}], "qubits": [8, 12], "gate": "cx", "name": "cx8_12"}, {"parameters": [{"date": "2019-08-29T02:10:18+00:00", "unit": "", "value": 0.030275075827968456, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.981146+00:00", "unit": "ns", "value": 1016.8888888888888, "name": "gate_length"}], "qubits": [8, 13], "gate": "cx", "name": "cx8_13"}, {"parameters": [{"date": "2019-08-29T02:15:39+00:00", "unit": "", "value": 0.03384110535480819, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.981361+00:00", "unit": "ns", "value": 832.0, "name": "gate_length"}], "qubits": [9, 4], "gate": "cx", "name": "cx9_4"}, {"parameters": [{"date": "2019-08-29T02:20:54+00:00", "unit": "", "value": 0.022334050057862515, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.981433+00:00", "unit": "ns", "value": 618.6666666666666, "name": "gate_length"}], "qubits": [9, 8], "gate": "cx", "name": "cx9_8"}, {"parameters": [{"date": "2019-08-29T01:10:55+00:00", "unit": "", "value": 0.026427550277632827, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980488+00:00", "unit": "ns", "value": 1144.888888888889, "name": "gate_length"}], "qubits": [10, 5], "gate": "cx", "name": "cx10_5"}, {"parameters": [{"date": "2019-08-29T02:26:07+00:00", "unit": "", "value": 0.018448907554389993, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979147+00:00", "unit": "ns", "value": 540.4444444444445, "name": "gate_length"}], "qubits": [10, 6], "gate": "cx", "name": "cx10_6"}, {"parameters": [{"date": "2019-08-29T02:31:23+00:00", "unit": "", "value": 0.03385617838527716, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979072+00:00", "unit": "ns", "value": 746.6666666666666, "name": "gate_length"}], "qubits": [10, 11], "gate": "cx", "name": "cx10_11"}, {"parameters": [{"date": "2019-08-29T03:25:37+00:00", "unit": "", "value": 0.02773385250352678, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979902+00:00", "unit": "ns", "value": 1144.888888888889, "name": "gate_length"}], "qubits": [10, 15], "gate": "cx", "name": "cx10_15"}, {"parameters": [{"date": "2019-08-29T01:16:15+00:00", "unit": "", "value": 0.03645862801896202, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980566+00:00", "unit": "ns", "value": 533.3333333333333, "name": "gate_length"}], "qubits": [11, 5], "gate": "cx", "name": "cx11_5"}, {"parameters": [{"date": "2019-08-29T02:36:49+00:00", "unit": "", "value": 0.016510173006301032, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979296+00:00", "unit": "ns", "value": 490.66666666666663, "name": "gate_length"}], "qubits": [11, 6], "gate": "cx", "name": "cx11_6"}, {"parameters": [{"date": "2019-08-29T02:31:23+00:00", "unit": "", "value": 0.03385617838527716, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979072+00:00", "unit": "ns", "value": 746.6666666666666, "name": "gate_length"}], "qubits": [11, 10], "gate": "cx", "name": "cx11_10"}, {"parameters": [{"date": "2019-08-29T02:52:19+00:00", "unit": "", "value": 0.04782211274426948, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979371+00:00", "unit": "ns", "value": 1386.6666666666665, "name": "gate_length"}], "qubits": [11, 12], "gate": "cx", "name": "cx11_12"}, {"parameters": [{"date": "2019-08-29T03:36:55+00:00", "unit": "", "value": 0.017795444267703325, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980049+00:00", "unit": "ns", "value": 533.3333333333333, "name": "gate_length"}], "qubits": [11, 16], "gate": "cx", "name": "cx11_16"}, {"parameters": [{"date": "2019-08-29T02:41:55+00:00", "unit": "", "value": 0.017994806783665718, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979221+00:00", "unit": "ns", "value": 597.3333333333333, "name": "gate_length"}], "qubits": [11, 17], "gate": "cx", "name": "cx11_17"}, {"parameters": [{"date": "2019-08-29T02:47:06+00:00", "unit": "", "value": 0.021131714462582912, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979602+00:00", "unit": "ns", "value": 576.0, "name": "gate_length"}], "qubits": [12, 7], "gate": "cx", "name": "cx12_7"}, {"parameters": [{"date": "2019-08-29T02:04:47+00:00", "unit": "", "value": 0.050011031518455235, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.981074+00:00", "unit": "ns", "value": 960.0, "name": "gate_length"}], "qubits": [12, 8], "gate": "cx", "name": "cx12_8"}, {"parameters": [{"date": "2019-08-29T02:52:19+00:00", "unit": "", "value": 0.04782211274426948, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979371+00:00", "unit": "ns", "value": 1386.6666666666665, "name": "gate_length"}], "qubits": [12, 11], "gate": "cx", "name": "cx12_11"}, {"parameters": [{"date": "2019-08-29T02:57:56+00:00", "unit": "", "value": 0.030678722924223023, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979446+00:00", "unit": "ns", "value": 960.0, "name": "gate_length"}], "qubits": [12, 13], "gate": "cx", "name": "cx12_13"}, {"parameters": [{"date": "2019-08-29T03:03:20+00:00", "unit": "", "value": 0.028891929243521602, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979525+00:00", "unit": "ns", "value": 1080.888888888889, "name": "gate_length"}], "qubits": [12, 16], "gate": "cx", "name": "cx12_16"}, {"parameters": [{"date": "2019-08-29T02:10:18+00:00", "unit": "", "value": 0.030275075827968456, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.981146+00:00", "unit": "ns", "value": 1016.8888888888888, "name": "gate_length"}], "qubits": [13, 8], "gate": "cx", "name": "cx13_8"}, {"parameters": [{"date": "2019-08-29T02:57:56+00:00", "unit": "", "value": 0.030678722924223023, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979446+00:00", "unit": "ns", "value": 960.0, "name": "gate_length"}], "qubits": [13, 12], "gate": "cx", "name": "cx13_12"}, {"parameters": [{"date": "2019-08-29T03:09:47+00:00", "unit": "", "value": 0.03318763653768775, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979677+00:00", "unit": "ns", "value": 1457.7777777777776, "name": "gate_length"}], "qubits": [13, 14], "gate": "cx", "name": "cx13_14"}, {"parameters": [{"date": "2019-08-29T03:52:30+00:00", "unit": "", "value": 0.026191570091749217, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980271+00:00", "unit": "ns", "value": 689.7777777777777, "name": "gate_length"}], "qubits": [13, 18], "gate": "cx", "name": "cx13_18"}, {"parameters": [{"date": "2019-08-29T03:57:44+00:00", "unit": "", "value": 0.027389508318681283, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980345+00:00", "unit": "ns", "value": 483.55555555555554, "name": "gate_length"}], "qubits": [13, 19], "gate": "cx", "name": "cx13_19"}, {"parameters": [{"date": "2019-08-29T03:09:47+00:00", "unit": "", "value": 0.03318763653768775, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979677+00:00", "unit": "ns", "value": 1457.7777777777776, "name": "gate_length"}], "qubits": [14, 13], "gate": "cx", "name": "cx14_13"}, {"parameters": [{"date": "2019-08-29T03:15:19+00:00", "unit": "", "value": 0.017789939003427763, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979752+00:00", "unit": "ns", "value": 490.66666666666663, "name": "gate_length"}], "qubits": [14, 18], "gate": "cx", "name": "cx14_18"}, {"parameters": [{"date": "2019-08-29T03:20:29+00:00", "unit": "", "value": 0.02085482437462391, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979827+00:00", "unit": "ns", "value": 504.88888888888886, "name": "gate_length"}], "qubits": [14, 19], "gate": "cx", "name": "cx14_19"}, {"parameters": [{"date": "2019-08-29T03:25:37+00:00", "unit": "", "value": 0.02773385250352678, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979902+00:00", "unit": "ns", "value": 1144.888888888889, "name": "gate_length"}], "qubits": [15, 10], "gate": "cx", "name": "cx15_10"}, {"parameters": [{"date": "2019-08-29T03:31:24+00:00", "unit": "", "value": 0.034972737463091014, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979976+00:00", "unit": "ns", "value": 1336.888888888889, "name": "gate_length"}], "qubits": [15, 16], "gate": "cx", "name": "cx15_16"}, {"parameters": [{"date": "2019-08-29T03:36:55+00:00", "unit": "", "value": 0.017795444267703325, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980049+00:00", "unit": "ns", "value": 533.3333333333333, "name": "gate_length"}], "qubits": [16, 11], "gate": "cx", "name": "cx16_11"}, {"parameters": [{"date": "2019-08-29T03:03:20+00:00", "unit": "", "value": 0.028891929243521602, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979525+00:00", "unit": "ns", "value": 1080.888888888889, "name": "gate_length"}], "qubits": [16, 12], "gate": "cx", "name": "cx16_12"}, {"parameters": [{"date": "2019-08-29T03:31:24+00:00", "unit": "", "value": 0.034972737463091014, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979976+00:00", "unit": "ns", "value": 1336.888888888889, "name": "gate_length"}], "qubits": [16, 15], "gate": "cx", "name": "cx16_15"}, {"parameters": [{"date": "2019-08-29T03:42:04+00:00", "unit": "", "value": 0.021430122261072604, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980123+00:00", "unit": "ns", "value": 490.66666666666663, "name": "gate_length"}], "qubits": [16, 17], "gate": "cx", "name": "cx16_17"}, {"parameters": [{"date": "2019-08-29T02:41:55+00:00", "unit": "", "value": 0.017994806783665718, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979221+00:00", "unit": "ns", "value": 597.3333333333333, "name": "gate_length"}], "qubits": [17, 11], "gate": "cx", "name": "cx17_11"}, {"parameters": [{"date": "2019-08-29T03:42:04+00:00", "unit": "", "value": 0.021430122261072604, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980123+00:00", "unit": "ns", "value": 490.66666666666663, "name": "gate_length"}], "qubits": [17, 16], "gate": "cx", "name": "cx17_16"}, {"parameters": [{"date": "2019-08-29T03:47:23+00:00", "unit": "", "value": 0.022798773268862788, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980197+00:00", "unit": "ns", "value": 490.66666666666663, "name": "gate_length"}], "qubits": [17, 18], "gate": "cx", "name": "cx17_18"}, {"parameters": [{"date": "2019-08-29T03:52:30+00:00", "unit": "", "value": 0.026191570091749217, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980271+00:00", "unit": "ns", "value": 689.7777777777777, "name": "gate_length"}], "qubits": [18, 13], "gate": "cx", "name": "cx18_13"}, {"parameters": [{"date": "2019-08-29T03:15:19+00:00", "unit": "", "value": 0.017789939003427763, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979752+00:00", "unit": "ns", "value": 490.66666666666663, "name": "gate_length"}], "qubits": [18, 14], "gate": "cx", "name": "cx18_14"}, {"parameters": [{"date": "2019-08-29T03:47:23+00:00", "unit": "", "value": 0.022798773268862788, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980197+00:00", "unit": "ns", "value": 490.66666666666663, "name": "gate_length"}], "qubits": [18, 17], "gate": "cx", "name": "cx18_17"}, {"parameters": [{"date": "2019-08-29T03:57:44+00:00", "unit": "", "value": 0.027389508318681283, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.980345+00:00", "unit": "ns", "value": 483.55555555555554, "name": "gate_length"}], "qubits": [19, 13], "gate": "cx", "name": "cx19_13"}, {"parameters": [{"date": "2019-08-29T03:20:29+00:00", "unit": "", "value": 0.02085482437462391, "name": "gate_error"}, {"date": "2019-08-29T04:01:35.979827+00:00", "unit": "ns", "value": 504.88888888888886, "name": "gate_length"}], "qubits": [19, 14], "gate": "cx", "name": "cx19_14"}]}'
with open(props_tokyo_target_filepath, 'w') as f:
  f.write(props_tokyo_str)

props_vigo_target_filepath = base_dir + "props_vigo.json"
props_vigo_str = '{"gates": [{"name": "id_0", "gate": "id", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.00043827201384123726, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 35.55555555555556, "unit": "ns"}], "qubits": [0]}, {"name": "u1_0", "gate": "u1", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.0, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 0.0, "unit": "ns"}], "qubits": [0]}, {"name": "u2_0", "gate": "u2", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.00043827201384123726, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 35.55555555555556, "unit": "ns"}], "qubits": [0]}, {"name": "u3_0", "gate": "u3", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.0008765440276824745, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 71.11111111111111, "unit": "ns"}], "qubits": [0]}, {"name": "id_1", "gate": "id", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.0005034477534859994, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 35.55555555555556, "unit": "ns"}], "qubits": [1]}, {"name": "u1_1", "gate": "u1", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.0, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 0.0, "unit": "ns"}], "qubits": [1]}, {"name": "u2_1", "gate": "u2", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.0005034477534859994, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 35.55555555555556, "unit": "ns"}], "qubits": [1]}, {"name": "u3_1", "gate": "u3", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.0010068955069719987, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 71.11111111111111, "unit": "ns"}], "qubits": [1]}, {"name": "id_2", "gate": "id", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.00034326438431702425, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 35.55555555555556, "unit": "ns"}], "qubits": [2]}, {"name": "u1_2", "gate": "u1", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.0, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 0.0, "unit": "ns"}], "qubits": [2]}, {"name": "u2_2", "gate": "u2", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.00034326438431702425, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 35.55555555555556, "unit": "ns"}], "qubits": [2]}, {"name": "u3_2", "gate": "u3", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.0006865287686340485, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 71.11111111111111, "unit": "ns"}], "qubits": [2]}, {"name": "id_3", "gate": "id", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.0005198394690553291, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 35.55555555555556, "unit": "ns"}], "qubits": [3]}, {"name": "u1_3", "gate": "u1", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.0, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 0.0, "unit": "ns"}], "qubits": [3]}, {"name": "u2_3", "gate": "u2", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.0005198394690553291, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 35.55555555555556, "unit": "ns"}], "qubits": [3]}, {"name": "u3_3", "gate": "u3", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.0010396789381106582, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 71.11111111111111, "unit": "ns"}], "qubits": [3]}, {"name": "id_4", "gate": "id", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.0007731054928936085, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 35.55555555555556, "unit": "ns"}], "qubits": [4]}, {"name": "u1_4", "gate": "u1", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.0, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 0.0, "unit": "ns"}], "qubits": [4]}, {"name": "u2_4", "gate": "u2", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.0007731054928936085, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 35.55555555555556, "unit": "ns"}], "qubits": [4]}, {"name": "u3_4", "gate": "u3", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:08:27+00:00", "value": 0.001546210985787217, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 71.11111111111111, "unit": "ns"}], "qubits": [4]}, {"name": "cx0_1", "gate": "cx", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:10:23+00:00", "value": 0.009675280360838395, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 348.4444444444444, "unit": "ns"}], "qubits": [0, 1]}, {"name": "cx1_0", "gate": "cx", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:10:23+00:00", "value": 0.009675280360838395, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 348.4444444444444, "unit": "ns"}], "qubits": [1, 0]}, {"name": "cx1_2", "gate": "cx", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:14:23+00:00", "value": 0.006942447193059004, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 227.55555555555554, "unit": "ns"}], "qubits": [1, 2]}, {"name": "cx1_3", "gate": "cx", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:18:28+00:00", "value": 0.012781729903015071, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 462.2222222222222, "unit": "ns"}], "qubits": [1, 3]}, {"name": "cx2_1", "gate": "cx", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:14:23+00:00", "value": 0.006942447193059004, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 227.55555555555554, "unit": "ns"}], "qubits": [2, 1]}, {"name": "cx3_1", "gate": "cx", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:18:28+00:00", "value": 0.012781729903015071, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 462.2222222222222, "unit": "ns"}], "qubits": [3, 1]}, {"name": "cx3_4", "gate": "cx", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:22:30+00:00", "value": 0.0070733452899932325, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 270.22222222222223, "unit": "ns"}], "qubits": [3, 4]}, {"name": "cx4_3", "gate": "cx", "parameters": [{"name": "gate_error", "date": "2019-10-16T06:22:30+00:00", "value": 0.0070733452899932325, "unit": ""}, {"name": "gate_length", "date": "2019-10-16T06:22:30+00:00", "value": 270.22222222222223, "unit": "ns"}], "qubits": [4, 3]}], "backend_name": "ibmq_vigo", "backend_version": "1.0.1", "last_update_date": "2019-10-16T06:22:30+00:00", "general": [], "qubits": [[{"name": "T1", "date": "2019-10-16T06:06:08+00:00", "value": 70.68917259689157, "unit": "\u00b5s"}, {"name": "T2", "date": "2019-08-09T16:59:15+00:00", "value": 18.38638524492989, "unit": "\u00b5s"}, {"name": "frequency", "date": "2019-10-16T06:22:30+00:00", "value": 4.79596103128903, "unit": "GHz"}, {"name": "readout_error", "date": "2019-10-16T06:05:50+00:00", "value": 0.07899999999999996, "unit": ""}, {"name": "prob_meas0_prep1", "date": "2019-10-16T06:05:50+00:00", "value": 0.06399999999999995, "unit": ""}, {"name": "prob_meas1_prep0", "date": "2019-10-16T06:05:50+00:00", "value": 0.094, "unit": ""}], [{"name": "T1", "date": "2019-10-16T06:06:08+00:00", "value": 98.60468683197823, "unit": "\u00b5s"}, {"name": "T2", "date": "2019-10-16T06:07:38+00:00", "value": 134.22307255375856, "unit": "\u00b5s"}, {"name": "frequency", "date": "2019-10-16T06:22:30+00:00", "value": 4.940048118160465, "unit": "GHz"}, {"name": "readout_error", "date": "2019-10-16T06:05:50+00:00", "value": 0.013000000000000012, "unit": ""}, {"name": "prob_meas0_prep1", "date": "2019-10-16T06:05:50+00:00", "value": 0.022, "unit": ""}, {"name": "prob_meas1_prep0", "date": "2019-10-16T06:05:50+00:00", "value": 0.0040000000000000036, "unit": ""}], [{"name": "T1", "date": "2019-10-16T06:06:08+00:00", "value": 138.11865391954476, "unit": "\u00b5s"}, {"name": "T2", "date": "2019-10-15T06:08:31+00:00", "value": 108.00879361397641, "unit": "\u00b5s"}, {"name": "frequency", "date": "2019-10-16T06:22:30+00:00", "value": 4.833249502714023, "unit": "GHz"}, {"name": "readout_error", "date": "2019-10-16T06:05:50+00:00", "value": 0.017000000000000015, "unit": ""}, {"name": "prob_meas0_prep1", "date": "2019-10-16T06:05:50+00:00", "value": 0.02200000000000002, "unit": ""}, {"name": "prob_meas1_prep0", "date": "2019-10-16T06:05:50+00:00", "value": 0.012, "unit": ""}], [{"name": "T1", "date": "2019-10-16T06:06:08+00:00", "value": 65.73158590716808, "unit": "\u00b5s"}, {"name": "T2", "date": "2019-10-16T06:06:59+00:00", "value": 71.33222205581698, "unit": "\u00b5s"}, {"name": "frequency", "date": "2019-10-16T06:22:30+00:00", "value": 4.807990365982154, "unit": "GHz"}, {"name": "readout_error", "date": "2019-10-16T06:05:50+00:00", "value": 0.019000000000000017, "unit": ""}, {"name": "prob_meas0_prep1", "date": "2019-10-16T06:05:50+00:00", "value": 0.028000000000000025, "unit": ""}, {"name": "prob_meas1_prep0", "date": "2019-10-16T06:05:50+00:00", "value": 0.01, "unit": ""}], [{"name": "T1", "date": "2019-10-16T06:06:08+00:00", "value": 121.30447979603545, "unit": "\u00b5s"}, {"name": "T2", "date": "2019-10-15T06:09:21+00:00", "value": 48.84570520106248, "unit": "\u00b5s"}, {"name": "frequency", "date": "2019-10-16T06:22:30+00:00", "value": 4.749898910235787, "unit": "GHz"}, {"name": "readout_error", "date": "2019-10-16T06:05:50+00:00", "value": 0.020000000000000018, "unit": ""}, {"name": "prob_meas0_prep1", "date": "2019-10-16T06:05:50+00:00", "value": 0.03, "unit": ""}, {"name": "prob_meas1_prep0", "date": "2019-10-16T06:05:50+00:00", "value": 0.010000000000000009, "unit": ""}]]}'
with open(props_vigo_target_filepath, 'w') as f:
  f.write(props_vigo_str)

props_yorktown_target_filepath = base_dir + "props_yorktown.json"
props_yorktown_str = '{"backend_version": "1.3.0", "general": [], "last_update_date": "2019-08-27T12:08:17+00:00", "backend_name": "ibmqx2", "qubits": [[{"date": "2019-08-27T02:57:11+00:00", "unit": "\u00b5s", "value": 46.56742052080564, "name": "T1"}, {"date": "2019-08-27T02:58:08+00:00", "unit": "\u00b5s", "value": 50.82263137546248, "name": "T2"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "GHz", "value": 5.286307493964072, "name": "frequency"}, {"date": "2019-08-27T02:56:36+00:00", "unit": "", "value": 0.08645000000000003, "name": "readout_error"}, {"date": "2019-08-27T02:56:36+00:00", "unit": "", "value": 0.10880000000000001, "name": "prob_meas0_prep1"}, {"date": "2019-08-27T02:56:36+00:00", "unit": "", "value": 0.0641, "name": "prob_meas1_prep0"}], [{"date": "2019-08-27T02:57:11+00:00", "unit": "\u00b5s", "value": 55.96928837701522, "name": "T1"}, {"date": "2019-08-27T02:58:58+00:00", "unit": "\u00b5s", "value": 45.63271590635976, "name": "T2"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "GHz", "value": 5.2378526142870685, "name": "frequency"}, {"date": "2019-08-27T02:56:36+00:00", "unit": "", "value": 0.04149999999999998, "name": "readout_error"}, {"date": "2019-08-27T02:56:36+00:00", "unit": "", "value": 0.0507, "name": "prob_meas0_prep1"}, {"date": "2019-08-27T02:56:36+00:00", "unit": "", "value": 0.032299999999999995, "name": "prob_meas1_prep0"}], [{"date": "2019-08-27T02:57:11+00:00", "unit": "\u00b5s", "value": 59.099655054478355, "name": "T1"}, {"date": "2019-08-27T02:59:49+00:00", "unit": "\u00b5s", "value": 63.66658719729211, "name": "T2"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "GHz", "value": 5.030295621586882, "name": "frequency"}, {"date": "2019-08-27T02:56:36+00:00", "unit": "", "value": 0.026899999999999924, "name": "readout_error"}, {"date": "2019-08-27T02:56:36+00:00", "unit": "", "value": 0.048699999999999966, "name": "prob_meas0_prep1"}, {"date": "2019-08-27T02:56:36+00:00", "unit": "", "value": 0.0051, "name": "prob_meas1_prep0"}], [{"date": "2019-08-27T02:57:11+00:00", "unit": "\u00b5s", "value": 54.40840937009215, "name": "T1"}, {"date": "2019-08-27T02:58:08+00:00", "unit": "\u00b5s", "value": 27.92450139381814, "name": "T2"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "GHz", "value": 5.296073260464525, "name": "frequency"}, {"date": "2019-08-27T02:56:36+00:00", "unit": "", "value": 0.06600000000000006, "name": "readout_error"}, {"date": "2019-08-27T02:56:36+00:00", "unit": "", "value": 0.09289999999999998, "name": "prob_meas0_prep1"}, {"date": "2019-08-27T02:56:36+00:00", "unit": "", "value": 0.0391, "name": "prob_meas1_prep0"}], [{"date": "2019-08-27T02:57:11+00:00", "unit": "\u00b5s", "value": 42.28189300974562, "name": "T1"}, {"date": "2019-08-27T02:58:58+00:00", "unit": "\u00b5s", "value": 40.179343798188306, "name": "T2"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "GHz", "value": 5.084664672299471, "name": "frequency"}, {"date": "2019-08-27T02:56:36+00:00", "unit": "", "value": 0.03695000000000004, "name": "readout_error"}, {"date": "2019-08-27T02:56:36+00:00", "unit": "", "value": 0.0531, "name": "prob_meas0_prep1"}, {"date": "2019-08-27T02:56:36+00:00", "unit": "", "value": 0.02080000000000004, "name": "prob_meas1_prep0"}]], "gates": [{"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 1.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 158.33333333333334, "name": "gate_length"}], "qubits": [0], "gate": "id", "name": "id_0"}, {"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [0], "gate": "u1", "name": "u1_0"}, {"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 1.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 158.33333333333334, "name": "gate_length"}], "qubits": [0], "gate": "u2", "name": "u2_0"}, {"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 2.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 316.6666666666667, "name": "gate_length"}], "qubits": [0], "gate": "u3", "name": "u3_0"}, {"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 1.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 158.33333333333334, "name": "gate_length"}], "qubits": [1], "gate": "id", "name": "id_1"}, {"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [1], "gate": "u1", "name": "u1_1"}, {"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 1.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 158.33333333333334, "name": "gate_length"}], "qubits": [1], "gate": "u2", "name": "u2_1"}, {"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 2.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 316.6666666666667, "name": "gate_length"}], "qubits": [1], "gate": "u3", "name": "u3_1"}, {"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 1.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 158.33333333333334, "name": "gate_length"}], "qubits": [2], "gate": "id", "name": "id_2"}, {"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [2], "gate": "u1", "name": "u1_2"}, {"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 1.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 158.33333333333334, "name": "gate_length"}], "qubits": [2], "gate": "u2", "name": "u2_2"}, {"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 2.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 316.6666666666667, "name": "gate_length"}], "qubits": [2], "gate": "u3", "name": "u3_2"}, {"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 1.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 158.33333333333334, "name": "gate_length"}], "qubits": [3], "gate": "id", "name": "id_3"}, {"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [3], "gate": "u1", "name": "u1_3"}, {"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 1.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 158.33333333333334, "name": "gate_length"}], "qubits": [3], "gate": "u2", "name": "u2_3"}, {"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 2.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 316.6666666666667, "name": "gate_length"}], "qubits": [3], "gate": "u3", "name": "u3_3"}, {"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 1.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 158.33333333333334, "name": "gate_length"}], "qubits": [4], "gate": "id", "name": "id_4"}, {"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 0.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 0.0, "name": "gate_length"}], "qubits": [4], "gate": "u1", "name": "u1_4"}, {"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 1.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 158.33333333333334, "name": "gate_length"}], "qubits": [4], "gate": "u2", "name": "u2_4"}, {"parameters": [{"date": "2019-08-27T12:08:17+00:00", "unit": "", "value": 2.0, "name": "gate_error"}, {"date": "2019-08-27T12:08:17+00:00", "unit": "ns", "value": 316.6666666666667, "name": "gate_length"}], "qubits": [4], "gate": "u3", "name": "u3_4"}, {"parameters": [{"date": "2019-08-27T01:56:10+00:00", "unit": "", "value": 0.034110782443154825, "name": "gate_error"}], "qubits": [0, 1], "gate": "cx", "name": "cx0_1"}, {"parameters": [{"date": "2019-08-27T03:09:58+00:00", "unit": "", "value": 0.03999613656912579, "name": "gate_error"}], "qubits": [0, 2], "gate": "cx", "name": "cx0_2"}, {"parameters": [{"date": "2019-08-27T03:15:02+00:00", "unit": "", "value": 0.031773117339429696, "name": "gate_error"}], "qubits": [1, 2], "gate": "cx", "name": "cx1_2"}, {"parameters": [{"date": "2019-08-27T03:19:43+00:00", "unit": "", "value": 0.07365076080227159, "name": "gate_error"}], "qubits": [3, 2], "gate": "cx", "name": "cx3_2"}, {"parameters": [{"date": "2019-08-27T03:24:24+00:00", "unit": "", "value": 0.03732767086325206, "name": "gate_error"}], "qubits": [3, 4], "gate": "cx", "name": "cx3_4"}, {"parameters": [{"date": "2019-08-27T03:29:45+00:00", "unit": "", "value": 0.03699218435586163, "name": "gate_error"}], "qubits": [4, 2], "gate": "cx", "name": "cx4_2"}]}'
with open(props_yorktown_target_filepath, 'w') as f:
  f.write(props_yorktown_str)


In [0]:
#@title Random circuit generator
"""
Locally generates a DC JSON describing Quantum Circuits.
"""

import random
import numpy as np
import qiskit
from qiskit.test.mock import FakeAlmaden
from qiskit.test.mock import FakeBoeblingen
from qiskit.test.mock import FakeJohannesburg
from qiskit.test.mock import FakeMelbourne
from qiskit.test.mock import FakeOurense
from qiskit.test.mock import FakePoughkeepsie
from qiskit.test.mock import FakeRueschlikon
from qiskit.test.mock import FakeSingapore
from qiskit.test.mock import FakeTenerife
from qiskit.test.mock import FakeTokyo
from qiskit.test.mock import FakeVigo
from qiskit.test.mock import FakeYorktown

# this is the only qiskit submodule we import directly to prevent a bug
# which might be due to conflict with the import of the `random` library.
from qiskit.circuit.random.utils import random_circuit

def qc_to_statevectors_dict(
  qc: qiskit.QuantumCircuit,
  n_qubits: int,
) -> dict:
  """
  Simulates a 5-qubit Quantum Circuit to generate
  its statevectors from 1024 simulated ideal runs of the 5-qubit Quantum Circuit.
  :param qc: The 5-qubit Quantum Circuit to simulate
  :param n_qubits: The number of qubits for the generated Quantum Circuits.
  :return: The dictionary of statevectors generated from 1024 simulated ideal
  runs of the 5-qubit Quantum Circuit.
  """
  results = qiskit.execute(
    qc,
    qiskit.Aer.get_backend('qasm_simulator')
  ).result()
  counts = results.get_counts()

  keys = [bin(i)[2:].zfill(n_qubits) for i in range(2 ** n_qubits)]
  my_counts = {"statevector_" + key: 0 for key in keys}

  for key in counts:
    my_counts["statevector_" + key] = counts[key]
  return my_counts


def type_of_gate(
  gate: tuple,
) -> str:
  """
  Gets the type of gate.
  :param gate: <tuple> (gate, list, list)
    Example: (<qiskit.extensions.standard.u3.U3Gate>,[Qubit(QuantumRegister(5, 'q'), 0)],[])

  :return:
    The type of gate.
    Examples: "U3Gate", "CnotGate", or "Measure"
  """
  return str(type(gate[0])).split(".")[-1][:-2]


def gate_to_dict(
  gate: tuple,
) -> dict:
  """
  Given a gate, returns one row of the CSV file describing the gate.
  Example usage:
    print(gate_to_json(None))
    for n, gate in enumerate(qc.data):
      print(gate_to_json(gate))

  :param gate: (gate, list, list)
    Example: (<qiskit.extensions.standard.u3.U3Gate>,[Qubit(QuantumRegister(5, 'q'), 0)],[])

  :return:
    Example: {"Gate_Type": "U3Gate", "Control": 0, "Target": -1, "Angle_1": 0.0, "Angle_2": 0.0, "Angle_3": 0.5}
  """
  if gate is None:
    raise Exception("gate is None")

  COL_QUANTITY = 6
  gate_type = type_of_gate(gate)
  gate_control = gate[1][0].index
  try:
    gate_target = gate[1][1].index
  except IndexError:
    gate_target = -42
  gate_angles = gate[0].params
  u3_gate_str_list = [gate_type, gate_control, gate_target]
  # u1 has 1 angle, u2 has 2 angles, u3 has 3 angles
  if gate_angles and len(gate_angles) >= 1:
    adj_angle = lambda angle: float(angle / np.pi)
    u3_gate_str_list += list(map(adj_angle, gate_angles[0:3]))

  initial_length = len(u3_gate_str_list)
  for i in range(initial_length, COL_QUANTITY):
    u3_gate_str_list.append(-42)
  dict_json = {
    "Gate_Type": u3_gate_str_list[0],
    "Control": u3_gate_str_list[1],
    "Target": u3_gate_str_list[2],
    "Angle_1": u3_gate_str_list[3],
    "Angle_2": u3_gate_str_list[4],
    "Angle_3": u3_gate_str_list[5],
  }
  return dict_json


def qc_to_gates_dict(
  qc: qiskit.QuantumCircuit,
  max_gates: int,
) -> (dict, int):
  """
  Parses the gates of a Quantum Circuit
  into a common dictionary format
  that conforms to our JSON schema.
  :param qc: The Quantum Circuit to get the gates of.
  :param max_gates: The maximum number of gates in the generated circuits.
  :return: <tuple> (gates_dict, number_of_gates)
    gates_dict: <dict>
      The gates of the Quantum Circuit in a common dictionary format.
      For example: { "gate_00": {}, ..., "gate_40": {}}
    number_of_gates: <int>
      The number of gates in the Quantum Circuit.
  """
  n_gates = len(qc.data)

  NOT_EXIST_VALUE = -1
  # The dictionary that every blank gate uses
  BLANK_GATE_DICT = {
    "Gate_Number": NOT_EXIST_VALUE,
    "Gate_Type": "BLANK",
    "Control": NOT_EXIST_VALUE,
    "Target": NOT_EXIST_VALUE,
    "Angle_1": NOT_EXIST_VALUE,
    "Angle_2": NOT_EXIST_VALUE,
    "Angle_3": NOT_EXIST_VALUE,
  }
  # n_digit_gate_number = len(str(max_gates))
  n_digit_gate_number = 2  # must always be 2 to fit the json schema

  gates_dict = {}

  # if len(qc.data) > max_gates:
  #   # print("WARNING: More gates than capacity")
  #   # n_gates = 0  # fill up the whole circuit with blank gates
  #   raise ValueError("More gates than capacity")
  # else:
  for gate_number, gate in enumerate(qc.data):
    gate_dict = gate_to_dict(gate)
    gate_dict['Gate_Number'] = gate_number
    gate_dict_key = 'gate_' + str(gate_number).zfill(n_digit_gate_number)
    gates_dict[gate_dict_key] = gate_dict
  # for gate_number in range(n_gates, max_gates):
  #   gate_dict = BLANK_GATE_DICT
  #   gate_dict_key = 'gate_' + str(gate_number).zfill(n_digit_gate_number)
  #   gates_dict[gate_dict_key] = gate_dict

  number_of_gates = n_gates  # so we can easily remove empty circuits later
  return gates_dict, number_of_gates


def optimize(qc, backend, optimization_level):
  qc1 = qiskit.transpile(
    circuits=qc,
    backend=backend,
    optimization_level=optimization_level
  )
  # we accept the optimized circuit because it should be
  # just u1 u2 u3 cx id
  # pass_ = qiskit.transpiler.passes.Unroller(['u3', 'cx'])
  # pm = qiskit.transpiler.PassManager(pass_)
  # qc2 = pm.run(qc1)
  return qc1


def simulate(qc, backend):
  properties = backend.properties()
  noise_model = qiskit.providers.aer.noise.device.basic_device_noise_model(properties)
  simulator = qiskit.Aer.get_backend('qasm_simulator')
  coupling_map = backend.configuration().coupling_map
  # NB we include the noise model in the second run
  result_noisy = qiskit.execute(qc,
                                simulator,
                                noise_model=noise_model,
                                coupling_map=coupling_map
                                ).result().get_counts()
  return result_noisy


def qc_to_noisy_statevectors_dict(
  qc: qiskit.QuantumCircuit,
  n_qubits: int,
  backend,
) -> dict:
  """
  Simulates a 5-qubit Quantum Circuit to generate
  its statevectors from 1024 simulated noisy runs of the 5-qubit Quantum Circuit.
  :param qc: The 5-qubit Quantum Circuit to simulate
  :param n_qubits: The number of qubits for the generated Quantum Circuits.
  :return: The dictionary of statevectors generated from 1024 simulated noisy runs of the 5-qubit Quantum Circuit.
  """
  counts = simulate(qc, backend)

  keys = [bin(i)[2:].zfill(n_qubits) for i in range(2 ** n_qubits)]
  my_counts = {"statevector_" + key: 0 for key in keys}

  for key in counts:
    my_counts["statevector_" + key] = counts[key]
  return my_counts


def qc_to_qc_dict(
  qc: qiskit.QuantumCircuit,
  max_gates: int,
  optimization_levels,
  backend_prop,
) -> dict:
  """
  Converts the Quantum Circuit into a common dictionary format that conforms to our JSON schema.
  :param qc: The Quantum Circuit to convert into a common dictionary format.
  :param max_gates: The maximum number of gates in the generated circuits.
  :param n_qubits: The number of qubits for the generated Quantum Circuits.
  :return: The common dictionary format representation of the Quantum Circuit.
  """
  gates_dict, number_of_gates = qc_to_gates_dict(
    qc=qc,
    max_gates=max_gates,
  )
#   circuit_set_id = random.randint(0, 2 ** 62)
  qc_dict = {
    # "circuit_set_id": circuit_set_id,
    "circuit_level_0":
      {
        "number_of_gates": number_of_gates,
        "optimization_level": 0,
        "gates": gates_dict
      }
  }

  for optimization_level in optimization_levels:
    qc1 = optimize(qc, backend_prop["backend"], optimization_level=optimization_level)

    gates_dict_1, number_of_gates_1 = qc_to_gates_dict(
      qc=qc1,
      max_gates=max_gates,
    )
    qc_dict["circuit_level_" + str(optimization_level)] = {
    #   "circuit_set_id": circuit_set_id,
    #   "backend_prop": backend_prop["backend"].properties(),
    #   "coupling_map": backend_prop["backend"].configuration().coupling_map,
      "layout": dict((virtual, physical) for virtual, physical in enumerate(qc1._layout._p2v.keys())),
      "number_of_gates": number_of_gates_1,
      "optimization_level": optimization_level,
      "gates": gates_dict_1
    }

  return qc_dict

def generate_qc(
  n_qubits: int,
  min_gates: int,
  max_gates: int,
) -> qiskit.QuantumCircuit:
  """
  Generates a random Quantum Circuit consisting of U3, CNot and Measure gates
  that satisfies the specified number of qubits and gates.
  :param n_qubits: The number of qubits for the generated Quantum Circuits.
  :param min_gates: The minimum number of gates in the generated circuits.
  :param max_gates: The maximum number of gates in the generated circuits.
  :return: The generated Quantum Circuit.
  """
  depth = random.randint(min_gates, max_gates)
  qc = random_circuit(
    n_qubits=n_qubits,
    depth=depth,
    max_operands=2,
    measure=True
  )
  pass_ = qiskit.transpiler.passes.Unroller(['u1', 'u2', 'u3', 'cx', 'id'])
  pm = qiskit.transpiler.PassManager(pass_)
  #
  # # if the quantum circuit has more gates than allowed
  # if len(qc.data) > max_gates:
  #   # Quantum Circuit had too many gates, try again
  #   return generate_qc(
  #     n_qubits=n_qubits,
  #     min_gates=min_gates,
  #     max_gates=max_gates,
  #   )

  # if the quantum circuit has more gates than allowed
  try:
    # check if the quantum circuit has no id gates
    qc2 = pm.run(qc)
  except qiskit.QiskitError:
    # Quantum Circuit had id gate, try again
    return generate_qc(
      n_qubits=n_qubits,
      min_gates=min_gates,
      max_gates=max_gates,
    )
  #
  # # if the quantum circuit has more gates than allowed
  # if len(qc2.data) > max_gates:
  #   # Quantum Circuit had too many gates, try again
  #   return generate_qc(
  #     n_qubits=n_qubits,
  #     min_gates=min_gates,
  #     max_gates=max_gates,
  #   )

  # Quantum Circuit had allowed number and type of gates, therefore successful.
  return qc2


def generate_qc_dict(
  min_gates: int,
  max_gates: int,
  optimization_levels,
  backend_prop,
) -> dict:
  """
  Generates a dictionary describing a random Quantum Circuit.
  :param n_qubits: The number of qubits for the generated Quantum Circuits.
  :param min_gates: The minimum number of gates in the generated circuits.
  :param max_gates: The maximum number of gates in the generated circuits.
  :return: The generated dictionary describing a random Quantum Circuit.
  """
  try:
    qc = generate_qc(
      n_qubits=backend_prop["n_qubits"],
      min_gates=min_gates,
      max_gates=max_gates,
    )
    qc_dict = qc_to_qc_dict(
      qc=qc,
      max_gates=max_gates,
      optimization_levels=optimization_levels,
      backend_prop=backend_prop,
    )
  except (
    qiskit.transpiler.exceptions.TranspilerError,
    ValueError
  ):
    return generate_qc_dict(
      min_gates=min_gates,
      max_gates=max_gates,
      optimization_levels=optimization_levels,
      backend_prop=backend_prop,
    )
  return qc_dict


def generate_a_qc_dict(
  min_gates,
  max_gates,
  optimization_levels,
):
  backend_props = [
    # {
    #   "backend_name": "fake_almaden",
    #   "backend": FakeAlmaden(),
    #   "n_qubits": 20,
    #   "coupling_map": [[0, 1], [1, 0], [1, 2], [1, 6], [2, 1], [2, 3], [3, 2], [3, 4], [3, 8], [4, 3], [5, 6], [5, 10],
    #                    [6, 1], [6, 5], [6, 7], [7, 6], [7, 8], [7, 12], [8, 3], [8, 7], [8, 9], [9, 8], [9, 14],
    #                    [10, 5],
    #                    [10, 11], [11, 10], [11, 12], [11, 16], [12, 7], [12, 11], [12, 13], [13, 12], [13, 14],
    #                    [13, 18],
    #                    [14, 9], [14, 13], [15, 16], [16, 11], [16, 15], [16, 17], [17, 16], [17, 18], [18, 13],
    #                    [18, 17],
    #                    [18, 19], [19, 18]]
    # },
    # {
    #   "backend_name": "fake_boeblingen",
    #   "backend": FakeBoeblingen(),
    #   "n_qubits": 20,
    #   "coupling_map": [[0, 1], [1, 0], [1, 2], [1, 6], [2, 1], [2, 3], [3, 2],
    #                    [3, 4], [3, 8], [4, 3], [5, 6], [5, 10], [6, 1], [6, 5],
    #                    [6, 7], [7, 6], [7, 8], [7, 12], [8, 3], [8, 7], [8, 9],
    #                    [9, 8], [9, 14], [10, 5], [10, 11], [11, 10], [11, 12],
    #                    [11, 16], [12, 7], [12, 11], [12, 13], [13, 12], [13, 14],
    #                    [13, 18], [14, 9], [14, 13], [15, 16], [16, 11], [16, 15],
    #                    [16, 17], [17, 16], [17, 18], [18, 13], [18, 17], [18, 19],
    #                    [19, 18]]
    # },
    # {
    #   "backend_name": "fake_johannesburg",
    #   "backend": FakeJohannesburg(),
    #   "n_qubits": 20,
    #   "coupling_map": [[0, 1], [0, 5], [1, 0], [1, 2], [2, 1], [2, 3], [3, 2], [3, 4], [4, 3], [4, 9],
    #                    [5, 0], [5, 6], [5, 10], [6, 5], [6, 7], [7, 6], [7, 8], [7, 12], [8, 7], [8, 9],
    #                    [9, 4], [9, 8], [9, 14], [10, 5], [10, 11], [10, 15], [11, 10], [11, 12], [12, 7],
    #                    [12, 11], [12, 13], [13, 12], [13, 14], [14, 9], [14, 13], [14, 19], [15, 10],
    #                    [15, 16], [16, 15], [16, 17], [17, 16], [17, 18], [18, 17], [18, 19], [19, 14],
    #                    [19, 18]]
    # },
    # {
    #   "backend_name": "fake_melbourne",
    #   "backend": FakeMelbourne(),
    #   "n_qubits": 14,
    #   "coupling_map": [[1, 0], [1, 2], [2, 3], [4, 3], [4, 10], [5, 4],
    #                    [5, 6], [5, 9], [6, 8], [7, 8], [9, 8], [9, 10],
    #                    [11, 3], [11, 10], [11, 12], [12, 2], [13, 1], [13, 12]]
    # },
    {
      "backend_name": "fake_ourense",
      "backend": FakeOurense(),
      "n_qubits": 5,
      "coupling_map": [[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 4], [4, 3]]
    },
    # {
    #   "backend_name": "fake_poughkeepsie",
    #   "backend": FakePoughkeepsie(),
    #   "n_qubits": 20,
    #   "coupling_map": [[0, 1], [0, 5], [1, 0], [1, 2], [2, 1], [2, 3], [3, 2], [3, 4], [4, 3], [4, 9],
    #                    [5, 0], [5, 6], [5, 10], [6, 5], [6, 7], [7, 6], [7, 8], [7, 12], [8, 7], [8, 9],
    #                    [9, 4], [9, 8], [9, 14], [10, 5], [10, 11], [10, 15], [11, 10], [11, 12], [12, 7],
    #                    [12, 11], [12, 13], [13, 12], [13, 14], [14, 9], [14, 13], [14, 19], [15, 10],
    #                    [15, 16], [16, 15], [16, 17], [17, 16], [17, 18], [18, 17], [18, 19], [19, 14],
    #                    [19, 18]]
    # },
    # {
    #   "backend_name": "fake_rueschlikon",
    #   "backend": FakeRueschlikon(),
    #   "n_qubits": 16,
    #   "coupling_map": [[1, 0], [1, 2], [2, 3], [3, 4], [3, 14], [5, 4], [6, 5],
    #                    [6, 7], [6, 11], [7, 10], [8, 7], [9, 8], [9, 10],
    #                    [11, 10], [12, 5], [12, 11], [12, 13], [13, 4],
    #                    [13, 14], [15, 0], [15, 2], [15, 14]]
    # },
    # {
    #   "backend_name": "fake_singapore",
    #   "backend": FakeSingapore(),
    #   "n_qubits": 20,
    #   "coupling_map": [[0, 1], [1, 0], [1, 2], [1, 6], [2, 1], [2, 3], [3, 2],
    #                    [3, 4], [3, 8], [4, 3], [5, 6], [5, 10], [6, 1], [6, 5],
    #                    [6, 7], [7, 6], [7, 8], [7, 12], [8, 3], [8, 7], [8, 9],
    #                    [9, 8], [9, 14], [10, 5], [10, 11], [11, 10], [11, 12],
    #                    [11, 16], [12, 7], [12, 11], [12, 13], [13, 12], [13, 14],
    #                    [13, 18], [14, 9], [14, 13], [15, 16], [16, 11], [16, 15],
    #                    [16, 17], [17, 16], [17, 18], [18, 13], [18, 17], [18, 19],
    #                    [19, 18]]
    # },
    # {
    #   "backend_name": "fake_tenerife",
    #   "backend": FakeTenerife(),
    #   "n_qubits": 5,
    #   "coupling_map": [[1, 0], [2, 0], [2, 1], [3, 2], [3, 4], [4, 2]]
    # },
    # {
    #   "backend_name": "fake_tokyo",
    #   "backend": FakeTokyo(),
    #   "n_qubits": 20,
    #   "coupling_map": [[0, 1], [0, 5], [1, 0], [1, 2], [1, 6], [2, 1],
    #                    [2, 3], [2, 6], [3, 2], [3, 8], [3, 9], [4, 8], [4, 9],
    #                    [5, 0], [5, 6], [5, 10], [5, 11], [6, 1], [6, 2], [6, 5],
    #                    [6, 7], [6, 10], [6, 11], [7, 1], [7, 6], [7, 8], [7, 12],
    #                    [7, 13], [8, 3], [8, 4], [8, 7], [8, 9], [8, 12], [8, 13],
    #                    [9, 3], [9, 4], [9, 8], [10, 5], [10, 6], [10, 11], [10, 15],
    #                    [11, 5], [11, 6], [11, 10], [11, 12], [11, 16], [11, 17],
    #                    [12, 7], [12, 8], [12, 11], [12, 13], [12, 16], [13, 7],
    #                    [13, 8], [13, 12], [13, 14], [13, 18], [13, 19], [14, 13],
    #                    [14, 18], [14, 19], [15, 10], [15, 16], [16, 11], [16, 12],
    #                    [16, 15], [16, 17], [17, 11], [17, 16], [18, 13], [18, 14],
    #                    [19, 13], [19, 14]]
    # },
    # {
    #   "backend_name": "fake_vigo",
    #   "backend": FakeVigo(),
    #   "n_qubits": 5,
    #   "coupling_map": [[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 4], [4, 3]]
    # },
    # {
    #   "backend_name": "fake_yorktown",
    #   "backend": FakeYorktown(),
    #   "n_qubits": 5,
    #   "coupling_map": [[0, 1], [0, 2], [1, 2], [3, 2], [3, 4], [4, 2]]
    # },
  ]

  backend_prop = random.choice(backend_props)

  return generate_qc_dict(
    min_gates=min_gates,
    max_gates=max_gates,
    optimization_levels=optimization_levels,
    backend_prop=backend_prop,
  )

**Generate circuit data**
- Using only `ourense` quantum computer data

In [0]:
import math
import json
import os
import multiprocessing as mp
from tqdm import tqdm_notebook as tqdm

In [0]:
def circuit_generator(n_circuit=1, min_gate=1, max_gate=5, optimization_level=3, parallel_computing=False):
    total_circuit = []
    if (parallel_computing == True):
        print("PARALLEL COMPUTING MODE | Number of processors is : ", mp.cpu_count())
        pool = mp.Pool(mp.cpu_count())
        total_circuit = [pool.apply(generate_a_qc_dict, args=(min_gate, max_gate, [optimization_level])) for i in tqdm(range(n_circuit))]
    else:
        for i in tqdm(range(n_circuit)):
            total_circuit.append(generate_a_qc_dict(min_gates=1, max_gates=5, optimization_levels=[3]))
    return (total_circuit)

**Dataset size of circuit number 30000 is approximately 5GB**

If you are not using local computer or enough size of cloud computer recommend to generate the circuit below 10000

In [0]:
# TODO - choose the circuit size you want to generate
n_circuit = 30000
circuits = circuit_generator(n_circuit=n_circuit,
                             min_gate=1,
                             max_gate=5,
                             optimization_level=3,
                             parallel_computing=True)

**save data**

In [0]:
import pickle

# TODO - choose the dataset name you want
dataset_name = "5_qubit_dataset.pkl"
with open(os.path.join(dataset_path, dataset_name), 'wb') as f:
    pickle.dump(circuits, f)

**load data**

When you want to load the saved dataset use this!

In [0]:
import pickle

# TODO - choose the dataset name you want
dataset_name = "dataset_noise_5.pkl"
with open(os.path.join(dataset_path, dataset_name), 'rb') as f:
    circuits = pickle.load(f)

**check data**

- circuit 0 is original circuit unrolled with base gates
- circuit 3 is optimized circuit using transpiler optimization level 3
    - layout information is included
- one gate should include following information
    - name
    - type
    - control
    - target
    - angle1
    - angle2
    - angle3

**!!!WARNING!!! quantum hardware information will not be inside this dataset**

In [0]:
import json

# TODO - choose the circuit and check is it valid
circuit_index = 0
print(json.dumps(circuits[circuit_index], indent=2))

### Hardware information
**Quantum computer topologie and noise information**

We are using only ourense computer so the hardware data will be generated here. But when we want to use various type of quantum computer or various noise information then we should include this informations listed below to upper `random circuit generator` function.

    - Coupling map
    - Noise information
        1. qubit readerror
        2. gate error

## Encode dataset for Deep learning
- Gate information
- Hardware Topologie
- Noise information
- layout information

### Gate information

    One circuit includes several gates
    Total 29 float numbers are used to encode one gate



**Details about one gate**

    - Gate (8 types of gate) - One hot encoding
        - U1
        - U2
        - U3
        - CX (Cnot)
        - ID
        - Barrier
        - X (None)
    - Qubit
        - Control (6 size used) - One hot encoding
            - 5 qubit
            - X (None) 
        - Target (6 size used) - One hot encoding
            - 5 qubit
            - X (None)
    - Classic (6 size used) - One hot encoding
        - 5 classical bit
        - X (None)
    - Angle (3 size used) - 3 float numbers
        - pie
        - theta
        - gamma

### Hardware Topologie
**Coupling map**
- Expressed by adjacency matrix
- [0, 1]
    - cmap[0][1] = 1
- same as cx error encoding. But making in different matrix because cx error has a different value inside matrix so there must be another matrix that can told the neural network model clearly about the connection between qubits

In [0]:
coupling_map = [[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 4], [4, 3]]
cmap = [[0 for _ in range(5)] for _ in range(5)]

In [0]:
for cx in coupling_map:
    cmap[cx[0]][cx[1]] = 1

In [0]:
print(cmap)
coupling_map = cmap

[[0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 0, 0, 1, 0]]


### Noise information
**Error encoding**
1. qubit readerror
2. gate error
    - u1, u2, u3 gate error will be expressed just in a line
    - make a adjacency matrix and express the error this will be replacing the coupling map
    - row is control and column is target
        - cx [1, 0] -> cx_error[1][0]

In [0]:
# Qubit readerror
prop = FakeOurense().properties()
readout_error = [0 for _ in range(5)]
for idx, qubit in enumerate(prop.qubits):
    readout_error[idx] = qubit[2].value

In [0]:
print(readout_error)

[4.819483735955884, 4.891102721325479, 4.71687548290116, 4.78902749798773, 5.023691537288777]


In [0]:
# Gate error
u1_error = [0 for _ in range(5)]
u2_error = [0 for _ in range(5)]
u3_error = [0 for _ in range(5)]
cx_error = [[0 for _ in range(5)] for _ in range(5)]

In [0]:
for gate in prop.gates:
    if (gate.gate == 'u1'):
        u1_error[gate.qubits[0]] = gate.parameters[0].value
    elif (gate.gate == 'u2'):
        u2_error[gate.qubits[0]] = gate.parameters[0].value
    elif (gate.gate == 'u3'):
        u3_error[gate.qubits[0]] = gate.parameters[0].value
    elif (gate.gate == 'cx'):
        cx_error[gate.qubits[0]][gate.qubits[1]] = gate.parameters[0].value

In [0]:
print(u1_error)
print(u2_error)
print(u3_error)
print(cx_error)

[0.0, 0.0, 0.0, 0.0, 0.0]
[0.00031177333311129094, 0.000464071792789536, 0.0002517609248549556, 0.0012093335635556912, 0.0005593027000324215]
[0.0006235466662225819, 0.000928143585579072, 0.0005035218497099112, 0.0024186671271113824, 0.001118605400064843]
[[0, 0.009131957296410004, 0, 0, 0], [0.009131957296410004, 0, 0.010095254518550478, 0.0219516066297471, 0], [0, 0.010095254518550478, 0, 0, 0], [0, 0.0219516066297471, 0, 0, 0.01179141510185705], [0, 0, 0, 0.01179141510185705, 0]]


### layout information

**Choosing the logic qubit to physical qubit**

    - {logic qubit : physical qubit}

    {
      "0": 1,
      "1": 4,
      "2": 3,
      "3": 0,
      "4": 2
    }
physical qubit will be encoded the number between 0 - 1

    0 - 0
    1 - 0.25
    2 - 0.5
    3 - 0.75
    4 - 1

So the upper example will be encoded 

`[0.25, 1, 0.75, 0, 0.25]`

### Process Encoding
- Encode the data prepared
- Save the model

**Dictionary include index of gate, layout**

In [0]:
# gate type encode
gate_2_id = {'U1Gate': 0, 'U2Gate':1, 'U3Gate': 2, 'CnotGate': 3, 'Measure': 4, 'IdGate': 5, 'Barrier': 6, 'X': 7}
id_2_gate = dict((v, k) for k, v in gate_2_id.items())
# layout encode
# physical qubit to number
p2n = [0., 0.25, 0.5, 0.75, 1.]


**Check the max length of original and optimized circuit to choose the step_size of RNN**

In [0]:
print(len(circuits))

30000


In [0]:
# circuits = encode_input
max_n_level0 = -float('inf')
max_n_level3 = -float('inf')
for i in range(len(circuits)):
    cur_0_n = circuits[i]['circuit_level_0']['number_of_gates']
    cur_3_n = circuits[i]['circuit_level_3']['number_of_gates']
    if (max_n_level0 < cur_0_n):
        max_n_level0 = cur_0_n
    if (max_n_level3 < cur_3_n):
        max_n_level3 = cur_3_n
print("max gate number of level 0 : {}".format(max_n_level0))
print("max gate number of level 3 : {}".format(max_n_level3))

max gate number of level 0 : 65
max gate number of level 3 : 130


In [0]:
#@title function to encode one circuit
import math

def encode_circuit(gates,
                   pad_num,
                   start_token=False,
                   concatenated_style=True):
    if (start_token and not concatenated_style):
        raise "start_token and seperated style cannot coexisted"

    # Sort the dataset
    sorted_gates = []
    for gate in sorted(gates.items()):
        sorted_gates.append(gate)

    # Concatenated
    pre_circuit = []

    # Seperated
    pre_gate = []
    pre_qubit_control = []
    pre_qubit_target = []
    pre_classic = []
    pre_angle = []

    # Type Concatenated
    pre_type = []

    # Add start token to decode input
    if (start_token == True):
        encoded_data = [1. for _ in range(29)]
        pre_circuit.append(encoded_data)

    # Encode input
    for i, gate in enumerate(sorted_gates):
        if (gate[0].startswith('gate') == False):
            continue
        encoded_gate = [0. for _ in range(8)]
        encoded_qubit_control = [0. for _ in range(6)]
        encoded_qubit_target = [0. for _ in range(6)]
        encoded_classic = [0. for _ in range(6)]
        encoded_angle = [0. for _ in range(3)]

        # encode gate
        # last gate is indicating there is no gate
        encoded_gate[gate_2_id[gate[1]['Gate_Type']]] = 1

        # encode qubit
        # qubit control or qubit measure
        # qubit target
        # classic measure
        if (encoded_gate[0] == 1): # u1
            encoded_qubit_control[gate[1]['Control'] + 1] = 1
            encoded_qubit_target[0] = 1
            encoded_classic[0] = 1
        elif (encoded_gate[1] == 1): # u2
            encoded_qubit_control[gate[1]['Control'] + 1] = 1
            encoded_qubit_target[0] = 1
            encoded_classic[0] = 1
        elif (encoded_gate[2] == 1): # u3
            encoded_qubit_control[gate[1]['Control'] + 1] = 1
            encoded_qubit_target[0] = 1
            encoded_classic[0] = 1
        elif (encoded_gate[3] == 1): # cx
            encoded_qubit_control[gate[1]['Control'] + 1] = 1
            encoded_qubit_target[gate[1]['Target'] + 1] = 1
            encoded_classic[0] = 1
        elif (encoded_gate[4] == 1): # measure
            encoded_qubit_control[gate[1]['Control'] + 1] = 1
            encoded_qubit_target[0] = 1
            encoded_classic[gate[1]['Control'] + 1] = 1
        elif (encoded_gate[5] == 1): # Id
            encoded_qubit_control[gate[1]['Control'] + 1] = 1
            encoded_qubit_target[0] = 1
            encoded_classic[0] = 1
        elif (encoded_gate[6] == 1): # Barrier
            encoded_qubit_control[0] = 1
            encoded_qubit_target[0] = 1
            encoded_classic[0] = 1

        # encode angle
        if (encoded_gate[0] == 1): # u1
            encoded_angle[2] = gate[1]['Angle_1']
        elif (encoded_gate[1] == 1): # u2
            encoded_angle[0] = math.pi / 2
            encoded_angle[1] = gate[1]['Angle_1']
            encoded_angle[2] = gate[1]['Angle_2']
        elif (encoded_gate[2] == 1): # u3
            encoded_angle[0] = gate[1]['Angle_1']
            encoded_angle[1] = gate[1]['Angle_2']
            encoded_angle[2] = gate[1]['Angle_3']
        
        if concatenated_style == True:
            # Concatenate style
            pre_circuit.append(encoded_gate + \
                            encoded_qubit_control + \
                            encoded_qubit_target + \
                            encoded_classic + \
                            encoded_angle)
        else:
            # Seperated style
            pre_gate.append(encoded_gate)
            pre_qubit_control.append(encoded_qubit_control)
            pre_qubit_target.append(encoded_qubit_target)
            pre_classic.append(encoded_classic)
            pre_angle.append(encoded_angle)

            # type concatenated
            pre_type.append(encoded_gate + \
                            encoded_qubit_control + \
                            encoded_qubit_target + \
                            encoded_classic)
    
    # Pad the empty gates
    # Make the total sequence length to the max length
    for _ in range(pad_num):
        encoded_gate = [0. for _ in range(8)]
        encoded_qubit_control = [0. for _ in range(6)]
        encoded_qubit_target = [0. for _ in range(6)]
        encoded_classic = [0. for _ in range(6)]
        encoded_angle = [0. for _ in range(3)]
        
        if concatenated_style == True:
            # Concatenated style
            encoded_gate[-1] = 1
            encoded_qubit_control[0] = 1
            encoded_qubit_target[0] = 1
            encoded_classic[0] = 1
            pre_circuit.append(encoded_gate + \
                            encoded_qubit_control + \
                            encoded_qubit_target + \
                            encoded_classic + \
                            encoded_angle)
        else:
            # Seperated style
            pre_gate.append(encoded_gate)
            pre_qubit_control.append(encoded_qubit_control)
            pre_qubit_target.append(encoded_qubit_target)
            pre_classic.append(encoded_classic)
            pre_angle.append(encoded_angle)

            # type concatenated
            pre_type.append(encoded_gate + \
                            encoded_qubit_control + \
                            encoded_qubit_target + \
                            encoded_classic)

    if concatenated_style == True:
        return pre_circuit
    else:
        return (pre_type,
                pre_gate,
                pre_qubit_control,
                pre_qubit_target,
                pre_classic,
                pre_angle)

In [0]:
#@title Generate dataset
# Encode has no concatentated_output because it is same as input
encode_circuit_0_input = []

encode_circuit_0_output_type = []

encode_circuit_0_output_gate = []
encode_circuit_0_output_qubit_control = []
encode_circuit_0_output_qubit_target = []
encode_circuit_0_output_classic = []
encode_circuit_0_output_angle = []

decode_circuit_3_input = []

decode_circuit_3_concatenated_output = []
decode_circuit_3_output_type = []

decode_circuit_3_output_gate = []
decode_circuit_3_output_qubit_control = []
decode_circuit_3_output_qubit_target = []
decode_circuit_3_output_classic = []
decode_circuit_3_output_angle = []

encode_layout = []

for i, circuit in enumerate(circuits):
    # Circuit original & optimized with level 3 using passmanager
    circuit_0 = circuit['circuit_level_0']
    circuit_3 = circuit['circuit_level_3']

    # The length of circuit
    num_gates_0 = circuit_0['number_of_gates']
    num_gates_3 = circuit_3['number_of_gates']

    # preprocess layout
    pre_layout = [p2n[p] for p in circuit_3['layout'].values()]
    encode_layout.append(pre_layout)

    # encode input - preprocess gate, qubit, angle
    encode_circuit_0_input.append(encode_circuit(circuit_0['gates'],
                                                 max_n_level0 - num_gates_0))
    
    # encode output - preprocess gate, qubit, angle
    total_type, gate, control, target, classic, angle = encode_circuit(circuit_0['gates'],
                                                           max_n_level0 - num_gates_0,
                                                           concatenated_style=False)
    encode_circuit_0_output_gate.append(gate)
    encode_circuit_0_output_qubit_control.append(control)
    encode_circuit_0_output_qubit_target.append(target)
    encode_circuit_0_output_classic.append(classic)
    encode_circuit_0_output_angle.append(angle)

    # encode type
    encode_circuit_0_output_type.append(total_type)

    # decoding input has a starting token [1,...,1,1]
    # decoding input has additional start token so it will be same as decode output length(timesteps)
    decode_circuit_3_input.append(encode_circuit(circuit_3['gates'], max_n_level3 - num_gates_3, start_token=True))

    # decode concatenated style
    decode_circuit_3_concatenated_output.append(encode_circuit(circuit_3['gates'],
                                                 max_n_level3 - num_gates_3 + 1))

    # decoding output must have additional empty gates to tell the circuit it is ended
    # So we add 1 to guarantee that the longested gate is also has one empty circuit
    total_type, gate, control, target, classic, angle = encode_circuit(circuit_3['gates'],
                                                           max_n_level3 - num_gates_3 + 1,
                                                           concatenated_style=False)
    decode_circuit_3_output_gate.append(gate)
    decode_circuit_3_output_qubit_control.append(control)
    decode_circuit_3_output_qubit_target.append(target)
    decode_circuit_3_output_classic.append(classic)
    decode_circuit_3_output_angle.append(angle)

    # encode type
    decode_circuit_3_output_type.append(total_type)

**If we are not using only ourense computer we have to fix `generate dataset` function to modify the environment information too**

In [0]:
import numpy as np

# Encode
enc_in = np.array(encode_circuit_0_input)
enc_out_type = np.array(encode_circuit_0_output_type)
enc_out_gate = np.array(encode_circuit_0_output_gate)
enc_out_control = np.array(encode_circuit_0_output_qubit_control)
enc_out_target = np.array(encode_circuit_0_output_qubit_target)
enc_out_classic = np.array(encode_circuit_0_output_classic)
enc_out_angle = np.array(encode_circuit_0_output_angle)

# Decode
dec_in = np.array(decode_circuit_3_input)
dec_out_concatenated = np.array(decode_circuit_3_concatenated_output)
dec_out_type = np.array(decode_circuit_3_output_type)
dec_out_gate = np.array(decode_circuit_3_output_gate)
dec_out_control = np.array(decode_circuit_3_output_qubit_control)
dec_out_target = np.array(decode_circuit_3_output_qubit_target)
dec_out_classic = np.array(decode_circuit_3_output_classic)
dec_out_angle = np.array(decode_circuit_3_output_angle)

# Layout
layout_out = np.array(encode_layout)

# Print out the shape to check it is valid
# Encode
print(enc_in.shape)
print(enc_out_type.shape)
print(enc_out_gate.shape)
print(enc_out_control.shape)
print(enc_out_target.shape)
print(enc_out_classic.shape)
print(enc_out_angle.shape)

# Decode
print(dec_in.shape)
print(dec_out_concatenated.shape)
print(dec_out_type.shape)
print(dec_out_gate.shape)
print(dec_out_control.shape)
print(dec_out_target.shape)
print(dec_out_classic.shape)
print(dec_out_angle.shape)

# Layout
print(layout_out.shape)

(30000, 65, 29)
(30000, 65, 26)
(30000, 65, 8)
(30000, 65, 6)
(30000, 65, 6)
(30000, 65, 6)
(30000, 65, 3)
(30000, 131, 29)
(30000, 131, 29)
(30000, 131, 26)
(30000, 131, 8)
(30000, 131, 6)
(30000, 131, 6)
(30000, 131, 6)
(30000, 131, 3)
(30000, 5)


**Generate quantum hardware dataset**

hardware information dataset includes hardware topologie and noise data.

If generating is not fixed to ourense computer this will be part of `generate dataset` and deleted

In [0]:
# hardware_information input
# hardware topologie
# gate error
# qubit readout error
hardware_input = list(np.array(cmap).reshape(-1)) + \
                    list(np.array(cx_error).reshape(-1)) + \
                    u1_error + u2_error + u3_error + readout_error
hardware_in = np.array(hardware_input)
hardware_in = np.repeat(hardware_in.reshape(-1, len(hardware_in)), len(hardware_in), axis=0)

**Save dataset**

- `Total dataset` - dataset including every dataset information

In [0]:
total_dataset = {'enc_in': enc_in,
                 'enc_out_type': enc_out_type,
                 'enc_out_gate': enc_out_gate,
                 'enc_out_control': enc_out_control,
                 'enc_out_target': enc_out_target,
                 'enc_out_classic': enc_out_classic,
                 'enc_out_angle': enc_out_angle,
                 'dec_in': dec_in,
                 'dec_out_concatenated': dec_out_concatenated,
                 'dec_out_type': dec_out_type,
                 'dec_out_gate': dec_out_gate,
                 'dec_out_control': dec_out_control,
                 'dec_out_target': dec_out_target,
                 'dec_out_classic': dec_out_classic,
                 'dec_out_angle': dec_out_angle,
                 'layout_out': layout_out,
                 'hardware_in': hardware_in,
                 'coupling_map': coupling_map,
                 'u1_error': u1_error,
                 'u2_error': u2_error,
                 'u3_error': u3_error,
                 'cx_error': cx_error,
                 'readout_error': readout_error}

In [0]:
import pickle

# TODO - Choose the encoded dataset name
encoded_dataset_name = "5_qubit_encoded_dataset.pkl"
with open(os.path.join(dataset_path, encoded_dataset_name), 'wb') as f:
    pickle.dump(total_dataset, f)

#### Dataset Seperated ver.

**This is not currently used. But if you want to seperate dataset you can use this. But the codes used to load the dataset at the training files should be modified**

In [0]:
aelstm_dataset = {'enc_in': enc_in,
                 'enc_out_type': enc_out_type,
                 'enc_out_gate': enc_out_gate,
                 'enc_out_control': enc_out_control,
                 'enc_out_target': enc_out_target,
                 'enc_out_classic': enc_out_classic,
                 'enc_out_angle': enc_out_angle}

In [0]:
layout_dataset = {'enc_in': enc_in,
                  'env_in': env_in,
                  'layout_out': layout_out}

In [0]:
optimize_dataset = {'enc_in': enc_in,
                    'env_in': env_in,
                    'dec_in': dec_in,
                    'dec_out_concatenated': dec_out_concatenated,
                    'dec_out_type': dec_out_type,
                    'dec_out_gate': dec_out_gate,
                    'dec_out_control': dec_out_control,
                    'dec_out_target': dec_out_target,
                    'dec_out_classic': dec_out_classic,
                    'dec_out_angle': dec_out_angle,}

### Additional dataset util - Merge
**Merging two dataset**

If dataset was generated seperately because the colab RAM limit is 25GB then we could uses the under codes to merge two dataset

In [0]:
# Load two dataset
import pickle, os
dataset_path = '/content/drive/My Drive/QUANTUM/qaward/qc/'
with open(os.path.join(dataset_path, "5_qubit_noise_dataset.pkl"), 'rb') as f:
    dataset = pickle.load(f)
with open(os.path.join(dataset_path, "5_qubit_noise_dataset_2.pkl"), 'rb') as f:
    dataset_2 = pickle.load(f)

In [0]:
# Merge two dataset
merged_dataset = {}
for parameter in dataset.keys():
    first_data = dataset['{}'.format(parameter)]
    second_data = dataset_2['{}'.format(parameter)]
    merged_data = np.vstack(first_data, second_data)
    merged_dataset['{}'.format(parameter)] = merged_data
    print("{} : ".format(merged_data.shape)) 

In [0]:
# Save the merged dataset
# Save it to the first dataset
import pickle
dataset_path = '/content/drive/My Drive/QUANTUM/qaward/qc/'
with open(os.path.join(dataset_path, "5_qubit_noise_dataset.pkl"), 'wb') as f:
    pickle.dump(merged_dataset, f)